In [3]:
%load_ext memory_profiler



The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [4]:
#import necessary packages
import numpy as np
import sys
import math
import re
import isoprene_rates as rate
from math import exp as EXP
from copy import deepcopy
import sympy as sym
import networkx as nx
import matplotlib.pyplot as plt
#import graphviz 
#import pygraphviz as pgv
#import to_precision
import time
import csv
import pandas as pd
import random
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import dijkstra
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse.csgraph import csgraph_from_dense
from scipy.sparse import lil_matrix
from scipy.sparse.csgraph import connected_components
#from sklearn.preprocessing import normalize

In [5]:
def reduce_mechanism_full_group_ammendment_with_comments_new_fix(mechanism, background_spc_n, conditions, settings):

    default_settings = {}
    default_settings['No Group'] = []
    default_settings['Manual Groups'] = []
    default_settings['Remove Reactions'] = True
    default_settings['Remove Weak Reactions'] = False
    default_settings['Weak Reaction Cutoff'] = 0
    default_settings['Keep Cycle Reactions'] = True
    default_settings['Iterations'] = 0
    for i in default_settings:
        if i not in settings:
            settings[i] = default_settings[i]
    
    print('Stage 1: Mechanism Preprocessing')
    
    needed_concentrations = set() # any species which is the second reactant in a reaction but the concentration is not provided

    # We are duplicating the mechanism and calling it test mechanism
    # This is to avoid inadvertently modifying the original mechanism variable
    test_reactions = []
    not_rxns = set(settings['Background Rxns'])
    for i in settings['Aerosol Rxns']:
        not_rxns.add(i)
    for i in range(len(mechanism.reactions)):
        if i not in not_rxns:
            test_reactions.append(deepcopy(mechanism.reactions[i]))      
    test_mechanism = Mechanism(deepcopy(mechanism.species),test_reactions)

    # number of reactions, number of species, species name to index dictionary, species list as indices
    reac_len = len(test_mechanism.reactions)
    spec_len = len(test_mechanism.species)
    dic = {test_mechanism.species[i]:i for i in range(spec_len)}
    species_list = [i for i in range(spec_len)]

    # species that cannot be removed from the mechanism (manual input)
    protected = set(settings['Protected']).union(background_spc_n)
    protected = [dic[i] for i in protected]

    # species that cannot be placed in a group
    no_group = [dic[i] for i in settings['No Group']]

    # species that have been manually added to a group
    manual_groups = []
    man_group_set = set()
    for i in settings['Manual Groups']:
        g = []
        for j in i:
            g.append(dic[j])
            man_group_set.add(dic[j])
        manual_groups.append(g)
    print('These are the manual groups: ', manual_groups)
    # root species of the mechanism
    roots = [dic[i] for i in settings['roots']]

    # number of conditions
    l_c = len(conditions)

    # set of background species
    back_set = set()
    for i in background_spc_n:
        back_set.add(mechanism.species.index(i))

    # list of reactants, list of products, list of product coefficients, and product dictionary for each reaction
    # eg: rxn no. 7 is A + X --> 0.5 B + 0.2 C and X is a background species
    # eg cont: A has index 0, B has index 1, c has index 2, and X has index 3
    # eg cont:  reac_list_n[7] = [A,X]     prod_list_n[7] = [B,C]      prod_coeff_list[7] = [0.5, 0.2]     prod_dict[7] = {B:0.5, C:0.2}
    # eg cont:    reac_list[7] = [0,3]       prod_list[7] = [1, 2]     
    # eg cont: reac_no_back[7] = [0]      prod_no_back[7] = [1,2]
    # eg cont: rxn_reac[0] = {7...}, rxn_reac[3] = {7...}, rxn_prod[1] = {7...}, rxn_prod[2] = {7...}
    reac_list_n = []
    prod_list_n = []
    prod_coeff_list = []
    prod_dict = [{} for i in range(reac_len)]
    for i in range(reac_len):
        test_mechanism.reactions[i].rate = []
        reac_list_n.append(test_mechanism.reactions[i].reactants)
        prod_list_n.append(list(test_mechanism.reactions[i].prod_dict))
        for k in test_mechanism.reactions[i].prod_dict:
            prod_dict[i][dic[k]] = test_mechanism.reactions[i].prod_dict[k]
    reac_list, prod_list, reac_no_back, prod_no_back, rxn_reac, rxn_prod = rxn_index_convert(reac_list_n,prod_list_n,background_spc_n, reac_len, spec_len, dic)
    rxn_prod = [set(i) for i in rxn_prod]
    rxn_reac = [set(i) for i in rxn_reac]

    
    c_count = 0
    for i in conditions: 
        M = pressure_to_m(i['pressure'],i['temp'])
        TEMP = i['temp']
        SUN = i['sun']
        CFACTOR = 2.5e+19
        p_fac = M/1000000000
        double_rxns = []
        count = 0
        for j in test_mechanism.reactions:
            if j.rate_law!='null':
                if 'J' in j.rate_law:
                    j.eval_rate_law = solve_j_rate(j.rate_law, i['sza'],i['sun'])
                else:
                    j.eval_rate_law = eval(j.rate_law)
                if len(j.reactants)==1:
                    j.rate.append(j.eval_rate_law)

                elif j.reactants[1] in background_spc_n and j.reactants[1] not in settings['roots']:
                    j.rate.append(j.eval_rate_law*i[j.reactants[1]]*p_fac)

                elif j.reactants[0] in background_spc_n and j.reactants[0] not in settings['roots']:
                    j.rate.append(j.eval_rate_law*i[j.reactants[0]]*p_fac)

                elif j.reactants[1] in i and j.reactants[1] not in settings['roots']:
                    j.rate.append(j.eval_rate_law*i[j.reactants[1]]*p_fac)

                elif j.reactants[0] in i and j.reactants[0] not in settings['roots']:
                    j.rate.append(j.eval_rate_law*i[j.reactants[0]]*p_fac)

                else:
                    needed_concentrations.add(j.reactants[1])
                    root_conc = 0
                    for r in settings['roots']:
                        root_conc = i[r] + root_conc
                    j.rate.append(j.eval_rate_law*0.005*root_conc*p_fac)
                    double_rxns.append(count)
                count = count + 1
            else:
                counted_reactants = 0
                for s in j.reactants:
                    if s not in no_counts:
                        counted_reactants+=1
                if counted_reactants==1:
                    j.rate.append(j.eval_rate_law[c_count])
                elif j.reactants[1] in background_spc_n and j.reactants[1] not in settings['roots']:
                    j.rate.append(j.eval_rate_law[c_count]*i[j.reactants[1]]*p_fac)
                elif j.reactants[0] in background_spc_n and j.reactants[0] not in settings['roots']:
                    j.rate.append(j.eval_rate_law[c_count]*i[j.reactants[0]]*p_fac)
                elif j.reactants[1] in i and j.reactants[1] not in settings['roots']:
                    j.rate.append(j.eval_rate_law[c_count]*i[j.reactants[1]]*p_fac)
                elif j.reactants[0] in i and j.reactants[0] not in settings['roots']:
                    j.rate.append(j.eval_rate_law[c_count]*i[j.reactants[0]]*p_fac)
                else:
                    needed_concentrations.add(j.reactants[1])
                    root_conc = 0
                    for r in settings['roots']:
                        root_conc = i[r] + root_conc
                    j.rate.append(j.eval_rate_law[c_count]*0.05*root_conc*p_fac)

                    double_rxns.append(count)
        c_count+=1
    
    print('For better results, add concentrations for these species: ', needed_concentrations)
    
    print('Stage 2: Representing Mechanism as Graph')
    graphs = [[] for c in range(l_c)]
    in_graph = [set() for i in range(spec_len)]
    out_graph = [set() for i in range(spec_len)]
    out_graph_type = [{} for i in range(spec_len)]
    in_graph_type  = [{} for i in range(spec_len)]
    
    
    
    rxn_types = {}
    species_types = {i:set() for i in range(spec_len)}
    for i in range(spec_len):
        if i not in back_set:
            edges = [{} for c in range(l_c)]
            rate_sums = [0 for c in range(l_c)]
            
            for j in rxn_reac[i]:
                for c in range(l_c):
                    rate_sums[c] = rate_sums[c] + test_mechanism.reactions[j].rate[c]
            for j in rxn_reac[i]:
                mults = [0 for c in range(l_c)]
                for c in range(l_c):
                    mults[c] = test_mechanism.reactions[j].rate[c]/max(1e-20,rate_sums[c])
                if len(test_mechanism.reactions[j].reactants) == 1:
                    type_r = 'solo'
                else:
                    mark = True
                    for p in reac_list[j]:
                        if p in background_spc:
                            type_r = p
                            mark = False
                    if mark:
                        type_r = 'double'
                rxn_types[j] = type_r
                species_types[i].add(str(type_r))
                for k in prod_dict[j]:
                    if i in in_graph_type[k]:
                        in_graph_type[k][i].add(type_r)
                    else:
                        in_graph_type[k][i] = set([type_r])
                    for c in range(l_c):
                        if k in edges[c]:
                            if c==0:
                                edges[c][k] = edges[c][k] + prod_dict[j][k]*mults[c]
                                out_graph[i].add(k)
                                in_graph[k].add(i)
                                out_graph_type[i][k].add(type_r)
                            else:
                                edges[c][k] = edges[c][k] + prod_dict[j][k]*mults[c]

                        
                        else:
                            if c==0:
                                edges[c][k] = prod_dict[j][k]*mults[c]
                                out_graph[i].add(k)
                                in_graph[k].add(i)
                                out_graph_type[i][k] = set([type_r])
                            else:
                                edges[c][k] = prod_dict[j][k]*mults[c]
            for c in range(l_c):
                graphs[c].append(edges[c])
        else:
            for c in range(l_c):
                graphs[c].append({})
    for i in range(len(species_types)):
        species_types[i] = sorted(list(species_types[i]))
    avg_graph = []
    for i in range(spec_len):
        avg = {}
        for c in range(l_c):
            for j in graphs[c][i]:
                if j in avg:
                    avg[j]+=graphs[c][i][j]
                else:
                    avg[j] = graphs[c][i][j]
        for x in avg:
            avg[x]*=1/l_c
        avg_graph.append(avg)
    
    graph_mat_lil = lil_matrix((spec_len,spec_len), dtype=None)

    edges = []
    for i in range(len(out_graph)):
        for j in out_graph[i]:
            edges.append((i,j))

    count = 0
    for i,j in edges:
        graph_mat_lil[i,j] = 1

    graph_mat = graph_mat_lil.tocsr(copy=False)

    scc_result = connected_components(graph_mat, directed=True, connection='strong', return_labels=True)

    scc_dic = {}
    for i in range(len(scc_result[1])):
        if scc_result[1][i] in scc_dic:
            scc_dic[scc_result[1][i]].append(i)
        else:
            scc_dic[scc_result[1][i]] = [i]
    
    scc = []
    for i in scc_dic:
        if len(scc_dic[i])>1:
            scc.append(scc_dic[i])

    distance,preds,root_specs = dijkstra(graph_mat, directed=True, indices=roots, return_predecessors=True, unweighted=True, limit=np.inf, min_only=True)    
    #return distance, preds, root_specs
    # measure scc's

    scc_lens = [len(i) for i in scc]
    # seeing how many species can be joined by the common parent rule
    
    #identifying species with same parent, same incoming reaction types and same outgoing reaction types
    
    
    paths = []
    path_roots = []
    for i in range(spec_len):
        if distance[i]<100:
            if preds[i]<0:
                path = []
            else:
                marker = preds[i]
                path = [marker]
                counter = 0
                while marker not in roots and counter<100:
                    marker = preds[marker]
                    path.append(marker)
            paths.append(path)
            path_roots.append(root_specs[i])
        else:
            paths.append([])
            path_roots.append('none')
    
    paths_with_types = []
    for i in range(len(paths)):
        path = []
        count = 0
        for j in paths[i]:
            if count == 0:
                path.append(in_graph_type[i][j])
            else:
                path.append(in_graph_type[paths[i][count-1]][j])
            count = count + 1
        path.insert(0,path_roots[i])
        species_type_string = ''
        for j in species_types[i]:
            species_type_string+=j+' '
        
        #path.append(species_type_string)
        paths_with_types.append(path)

    print('10')
    path_type_strings = []
    for i in paths_with_types:
        string = str(i[0]) + '+'
        for j in i[1:]:
            for k in j:
                string = string + str(k) + ','
            string = string+'+'
        path_type_strings.append(string)

    path_sim_dict = {}
    for i in range(len(path_type_strings)):
        if i not in back_set:
            if path_type_strings[i] in path_sim_dict:
                path_sim_dict[path_type_strings[i]].append(i)
            else:
                path_sim_dict[path_type_strings[i]]= [i]
    groups = []  
    for i in path_sim_dict:
        if len(path_sim_dict[i])>1:
            groups.append(path_sim_dict[i])
    #print('this is the og grops', groups)
    #print('out 31', out_graph[31])  
    #for i in groups:
    
    del paths_with_types
    
    
    print('11')
    in_cycle_specs = []
    scc_out_specs = []
    for i in scc:
        count = count + 1
        #if count%100 == 0:
         #   print(count)
        in_cycle_spec = set()
        out_spec = set()
        for p in i:
            for k in in_graph[p]:
                if k not in i:
                    in_cycle_spec.add(p)
            for k in out_graph[p]:
                if k not in i:
                    out_spec.add(k)
        in_cycle_specs.append(in_cycle_spec)
        scc_out_specs.append(out_spec)
        
    

    scc_set = set()
    scc_dict_2 = {}
    for i in range(len(scc)):
        for j in scc[i]:
            scc_dict_2[j] = i
            scc_set.add(j)

    
    print('Stage 3: Measuring Mechanism Yields')

    county = 0
    yields = []
    yields_for_order = []
    yields_for_order_2 = []
    cycle_out_graphs = [{} for c in range(l_c)]
    cycle_dag_graphs = [{} for c in range(l_c)]
    cycle_dag_graphs_2 = [{} for c in range(l_c)]
    new_graphs = []
    new_ins = []
    new_outs = []
    for c in range(l_c):
        #print(county)
        county +=1
        
       # new_graph, new_in, new_out = copy_graph(graphs[c], in_graph, out_graph)
        
        new_graph = []
        for i in graphs[c]:
            dicy = {}
            for j in i:
                dicy[j] = i[j]
            new_graph.append(dicy)
        new_graph_2 = deepcopy(new_graph)
        new_graph_3 = deepcopy(new_graph)

        new_in = []
        for i in in_graph:
            sety = set()
            for j in i:
                sety.add(j)
            new_in.append(sety)
        

        new_out = []
        for i in out_graph:
            sety = set()
            for j in i:
                sety.add(j)
            new_out.append(sety)
        
        
        for i in range(len(scc)):
            counte = 0
            
            for x in in_cycle_specs[i]:
                counte +=1
                new_out[x] = scc_out_specs[i].union(set(scc[i]))
                leny = int(np.sqrt(len(scc[i])))
                data, in_cyc_data, out_data = cycle_simulator_3_out(scc[i], x, graphs[c],out_graph,in_graph, [leny+40,2*(leny+40)],1e-6,set(scc[i]).union(scc_out_specs[i]),scc_out_specs[i])
                cycle_out_graphs[c][x] = data
                ful_dat = {}
                for p in data:
                    ful_dat[p] = data[p]
                ful_dat_2 = deepcopy(ful_dat)
                for p in in_cyc_data:
                    ful_dat[p] = in_cyc_data[p]
                for p in out_data:
                    ful_dat_2[p] = out_data[p]
                new_graph[x]= data
                new_graph_2[x]= ful_dat
                new_graph_3[x] = deepcopy(ful_dat_2)
                cycle_dag_graphs[c][x] = ful_dat
                cycle_dag_graphs_2[c][x] = ful_dat_2

                

            for y in scc[i]:
                if y not in in_cycle_specs[i]:

                    for s in out_graph[y]:
                        if y in new_in[s]:
                            new_in[s].remove(y)
                    new_in[y] = set(in_cycle_specs[i])
                    new_out[y] = set()
                    new_graph[y] = set()
                    new_graph_2[y]= set()
        new_graphs.append(new_graph)
        new_ins.append(new_in)
        new_outs.append(new_out)
        yi = get_yields(roots, 1e-8, new_graph, new_in, new_out, back_set,scc,scc_dict_2, scc_set, spec_len)
        yi_2 = get_yields(roots, 1e-8, new_graph_2, new_in, new_out, back_set,scc,scc_dict_2, scc_set, spec_len)
        yi_3 = get_yields(roots, 1e-8, new_graph_3, new_in, new_out, back_set,scc,scc_dict_2, scc_set, spec_len)

        yields.append(yi)
        yields_for_order.append(yi_2)
        yields_for_order_2.append(yi_3)
    avg_cycle_out_graph = {}
    for c in range(l_c):
        for x in cycle_out_graphs[c]:
            
            if x in avg_cycle_out_graph:
                for j in cycle_out_graphs[c][x]:
                    if j in avg_cycle_out_graph[x]:
                        avg_cycle_out_graph[x][j]+=cycle_out_graphs[c][x][j]
                    else:
                        avg_cycle_out_graph[x][j] = cycle_out_graphs[c][x][j]
            else:
                avg_cycle_out_graph[x] = {}
                for j in cycle_out_graphs[c][x]:
                    if j in avg_cycle_out_graph[x]:
                        avg_cycle_out_graph[x][j]+=cycle_out_graphs[c][x][j]
                    else:
                        avg_cycle_out_graph[x][j] = cycle_out_graphs[c][x][j]
    
    for i in avg_cycle_out_graph:
        for j in avg_cycle_out_graph[i]:
            avg_cycle_out_graph[i][j]*=1/l_c




    avg_cycle_dag_graph = {}
    for c in range(l_c):
        for x in cycle_dag_graphs[c]:
            
            if x in avg_cycle_dag_graph:
                for j in cycle_dag_graphs[c][x]:
                    if j in avg_cycle_dag_graph[x]:
                        avg_cycle_dag_graph[x][j]+=cycle_dag_graphs[c][x][j]
                    else:
                        avg_cycle_dag_graph[x][j] = cycle_dag_graphs[c][x][j]
            else:
                avg_cycle_dag_graph[x] = {}
                for j in cycle_dag_graphs[c][x]:
                    if j in avg_cycle_dag_graph[x]:
                        avg_cycle_dag_graph[x][j]+=cycle_dag_graphs[c][x][j]
                    else:
                        avg_cycle_dag_graph[x][j] = cycle_dag_graphs[c][x][j]
    
    for i in avg_cycle_dag_graph:
        for j in avg_cycle_dag_graph[i]:
            avg_cycle_dag_graph[i][j]*=1/l_c



    

    avg_yield_o = {}
    for i in yields_for_order[0]:
        specyi = []
        for c in range(l_c):
            specyi.append(yields_for_order[c][i])
        avg_yield_o[i] = np.mean(specyi)


    avg_yield_o2 = {}
    for i in yields_for_order_2[0]:
        specyi = []
        for c in range(l_c):
            specyi.append(yields_for_order_2[c][i])
        avg_yield_o2[i] = np.mean(specyi)
        
    boost_factor = 1.5
    ordered_species = []
    ordered_yields = []
    yield_values = []
    for i in avg_yield_o:
        yield_values.append(avg_yield_o[i])
    max_yield = max(yield_values)
    for i in avg_yield_o:
        if i in protected:
            ordered_yields.append(max_yield)
        else:
            ordered_yields.append(avg_yield_o[i])
        ordered_species.append(i)


    
    avg_yield = {}
    for i in yields[0]:
        specyi = []
        for c in range(l_c):
            specyi.append(yields[c][i])
        avg_yield[i] = np.mean(specyi)
        
    
        
    ordered_species = [x for _, x in sorted(zip(ordered_yields, ordered_species))]
    ordered_yields.sort()
    spec_order_dict = {}
    count = 0
    for i in ordered_species:
        spec_order_dict[i] = count
        count+=1
    for i in groups:
        min_dict = {}
        for j in i:
            min_dict[spec_order_dict[j]] = j
        min_spec = min_dict[min(min_dict)]
        ordered_yields[spec_order_dict[min_spec]] *= boost_factor
    
    ordered_species = [x for _, x in sorted(zip(ordered_yields, ordered_species))]
    for i in protected:
        ordered_species.remove(i)
        ordered_species.append(i)
    ordered_yields.sort()

    for i in settings['Remove Species']:
        ordered_species.remove(dic[i])
        ordered_species.insert(0, dic[i])
    
    reduced_graph, reduced_in, reduced_out = copy_graph(avg_graph, in_graph, out_graph)
    reduced_reactions = []
    for i in test_mechanism.reactions:
        reduced_reactions.append(deepcopy(i))
    
    reduced_mech = Mechanism(deepcopy(test_mechanism.species),reduced_reactions)

    for i in range(reac_len):
        reduced_mech.reactions[i].prod_dict = prod_dict[i]
        reduced_mech.reactions[i].reactants = reac_list[i]
    
    avg_yield_full = avg_yield
    not_removed = set()
    remaining_specs = ordered_species[spec_len-red_mech_size:]
    
    remove_species = []
    for i in ordered_species[:spec_len-red_mech_size]:
        if i not in protected:
            remove_species.append(i)
        else:
            remaining_specs.append(i)

    print('Stage 4: Combining Grouped Species')

    new_groups = []
    man_group_set = set()
    for i in manual_groups:
        for j in i:
            man_group_set.add(j)
    for i in groups:
        new_group = []
        for j in i:
            if j not in man_group_set:
                new_group.append(j)
        if len(new_group)>1:
            new_groups.append(new_group)
    groups= deepcopy(new_groups)
    new_groups = []
    for i in groups:
        if not all([x in remaining_specs for x in i]):
            new_groups.append(i)
    groups = deepcopy(new_groups)
    

    remaining_scc = set()
    for i in remaining_specs:
        if i in scc_set:
            remaining_scc.add(scc_dict_2[i])

    removed_scc = set(range(len(scc))).difference(remaining_scc)
    scc_remove_species = set()

    for i in removed_scc:
        for j in scc[i]:
            scc_remove_species.add(j)
    
    new_groups = []
    for i in range(len(groups)):
        no_scc_group = []
        scc_groups = {}
        none_left = []
        for j in groups[i]:
            if j not in no_group:
                if j in scc_set:
                    if scc_dict_2[j] in scc_groups:
                        scc_groups[scc_dict_2[j]].append(j)
                    else:
                        scc_groups[scc_dict_2[j]] = [j]
                else:
                    no_scc_group.append(j)
        for g in scc_groups:
            scc_ind = scc_dict_2[scc_groups[g][0]]
 
            scc_specs = []
            for y in scc[scc_ind]:
                if y not in scc_groups[g]:
                    scc_specs.append(y)

            counted = 0

            if not any([x in remaining_specs for x in scc_specs]):

                for h in scc_groups:
                    if g!=h:
                        if any([x in remaining_specs for x in scc_groups[h]]) and counted==0:
                            counted+=1
                            
                            for f in scc_groups[g]:
                                scc_groups[h].append(f)
                                print('FF', f)
                                if f in remaining_specs:
                                    
                                    remaining_specs.remove(f)
                                    remove_species.append(f)
                if counted==0:
                    for f in scc_groups[g]:
                        none_left.append(f)
                scc_groups[g] = []
                        
        if len(no_scc_group)>1:
            new_groups.append(no_scc_group)
        for k in scc_groups:
            if len(scc_groups[k])>1:
                new_groups.append(scc_groups[k])
        if len(none_left)>1:
            new_groups.append(none_left)
    groups = deepcopy(new_groups)   
    
    for m in manual_groups:
        if any([x in remaining_specs for x in m]):
            if any([x in remove_species for x in m]):
                groups.append(m)     
    #groups
    group_set = set()
    grouped = set()
    group_dict = {}
    rem_groups = []
    count = 0
    for i in groups:
        if any([j in remaining_specs for j in i]) and any([k in remove_species for k in i]):
            rem_groups.append(i)
            for j in i:
                group_set.add(j)
                group_dict[j] = count
            count+=1
    groups = deepcopy(rem_groups)
    group_mapping = []
    
    
    group_dict_check = {}

    for j in range(len(groups)):
        mapping = [0]
        print(groups[j])
        group = groups[j]
        for i in ordered_species[spec_len-red_mech_size:]:
            if i in group:
                group_lead = i
                mapping[0] = group_lead
        out_reactions = {}
        removers = set()

        new_group = {group_lead}
        for i in group:
            if i not in remove_species:
                pass

            else:
                new_group.add(i)
        group = deepcopy(new_group) 
        for sp in new_group:
            group_dict_check[sp] = new_group
        total_yield = 0 #avg_yield_full[group_lead]
        grouped_map = []
        for i in group:
            #if i in remove_species:
            grouped.add(i)
            grouped_map.append(i)
            total_yield += avg_yield_o2[i]
            #else:
                #print('just checking')
        mapping.append(grouped_map)
        group_mapping.append(mapping)

        
        
        new_graph_gl = {}
        within_spec_ratio = {}
        
        #mult * = base * spec_rat * rate_type[x]/rate_sum  /rate 
        for i in group:
            for r in rxn_reac[i]:
                if any([x in group for x in reduced_mech.reactions[r].prod_dict]):
                    new_prod_dict = {}
                    
                    for p in reduced_mech.reactions[r].prod_dict:
                        if p in group:
                            if p in reduced_graph[i]:
                                del reduced_graph[i][p]
                        else:
                            new_prod_dict[p] = reduced_mech.reactions[r].prod_dict[p]
                        
                    reduced_mech.reactions[r].prod_dict = deepcopy(new_prod_dict)                      

        # new stuff above
        
        base_spec = avg_yield_o2[group_lead]/max(1e-20,total_yield)
        base_rate_type = {}
        for g in groups[j]:
            for x in out_graph_type[g]:
                for y in out_graph_type[g][x]:
                    base_rate_type[y] = 0
        

        
        base_rate = 0
        for r in rxn_reac[group_lead]:
            if rxn_types[r] in base_rate_type:
                base_rate_type[rxn_types[r]]+= np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier
            else:
                pass
                #print('no match', groups[j], reduced_mech.reactions[r].reactants)
            base_rate+=np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier
            
        for r in rxn_reac[group_lead]:
            
            mult = base_spec

            reduced_mech.reactions[r].multiplier *= mult
        for i in group:
            if i != group_lead:
                spec_rat = avg_yield_o[i]/max(1e-20,total_yield)
                rate_type = {}
                for x in out_graph_type[i]:
                    for y in out_graph_type[i][x]:
                        rate_type[y] = 0
                rate_sum = 0
                for r in rxn_reac[i]:
                    rate_type[rxn_types[r]]+= np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier
                    rate_sum+=np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier
                for r in rxn_reac[i]:
                    if rate_sum>0:
                        mult = spec_rat*base_rate/rate_sum
                        
                    else:
                        mult = 0
                    new_reactants = []
                    for x in reduced_mech.reactions[r].reactants:
                        if x in group:
                            new_reactants.append(group_lead)
                        else:
                            new_reactants.append(x)
                    reduced_mech.reactions[r].reactants = new_reactants
                    reduced_mech.reactions[r].multiplier*=mult
                    rxn_reac[group_lead].add(r)

                    for p in reduced_mech.reactions[r].prod_dict:
                        
                        reduced_out[group_lead].add(p)
                        reduced_in[p].add(group_lead)
                for r in rxn_prod[i]:
                    if i in reduced_mech.reactions[r].prod_dict:
                        if group_lead in reduced_mech.reactions[r].prod_dict:
                            reduced_mech.reactions[r].prod_dict[group_lead] += reduced_mech.reactions[r].prod_dict[i]
                        else:
                            reduced_mech.reactions[r].prod_dict[group_lead] = reduced_mech.reactions[r].prod_dict[i]
                        del reduced_mech.reactions[r].prod_dict[i]
                        rxn_prod[group_lead].add(r)
                        for re in reduced_mech.reactions[r].reactants:
                            if re not in background_spc:
                                if i in reduced_graph[re]:
                                    if group_lead in reduced_graph[re]:
                                        reduced_graph[re][group_lead]+= reduced_graph[re][i]
                                    else:
                                        reduced_graph[re][group_lead] = reduced_graph[re][i]
                for x in reduced_out[i]:
                    if i in reduced_in[x]:
                        reduced_in[x].remove(i)
                for x in reduced_in[i]:
                    if i in reduced_out[x]:
                        reduced_out[x].remove(i)
                    if i in reduced_graph[x]:
                        del reduced_graph[x][i]
              
        total_rate = 0
        new_in_gl = set()
        for r in rxn_prod[group_lead]:
            for j in reduced_mech.reactions[r].reactants:
                if j not in background_spc:
                    new_in_gl.add(j)
                    
        for r in rxn_reac[group_lead]:
            total_rate+= np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier
        new_out_gl = set()
        for r in rxn_reac[group_lead]:
            for p in reduced_mech.reactions[r].prod_dict:
                new_out_gl.add(p)
                if p in new_graph_gl:
                    new_graph_gl[p]+= reduced_mech.reactions[r].prod_dict[p]*np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier/max(1e-20,total_rate)
                else:
                    new_graph_gl[p] = reduced_mech.reactions[r].prod_dict[p]*np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier/max(1e-20,total_rate)
        reduced_graph[group_lead] = new_graph_gl
        reduced_out[group_lead] = new_out_gl
        reduced_in[group_lead] = new_in_gl
    named_rxns = []
    for i in reduced_mech.reactions:
        new_rxn = deepcopy(i)
        new_reactants = []
        for j in i.reactants:
            new_reactants.append(species_list_names_2[j])
        new_prods = {}
        for j in i.prod_dict:
            new_prods[species_list_names_2[j]]= i.prod_dict[j]
        new_rxn.prod_dict = new_prods
        new_rxn.reactants = new_reactants
        named_rxns.append(new_rxn)
    reduced_mech_test = Mechanism(species_list_names_2,named_rxns)
    print('MULTIPLIER', reduced_mech.reactions[177].multiplier)
    print('Stage 5: Converting Graph to Directed Acyclic Graph')

    removed_rxns = set()
    dont_remove = set()
    for i in remaining_specs:
        dont_remove.add(i)
    for i in grouped:
        dont_remove.add(i)
    scc_rem_removed = set()
    for i in remaining_scc:
        just_removed = set()
        for j in scc[i]:
            if j not in dont_remove:
                for r in rxn_reac[j]:
                    removed_rxns.add(r)
                for ins in reduced_in[j]:
                    
                    if ins in reduced_graph[j]:
                        scale_factor = abs(1/max(1e-20,1-reduced_graph[j][ins]))
                        if j==126:
                            print('SCALRE FActor', scale_factor)
                    else:
                        scale_factor = 1
                    
                    if j in reduced_graph[ins]: 
                        mark = False
                        if ins in group_dict_check:
                            if any([t in scc[i] for t in group_dict_check[ins]]):
                                mark = True

                        if ins in scc[i] or mark==True:
                            for sp in reduced_graph[j]:
                                if sp!= ins:
                                    if sp in reduced_graph[ins]:
                                        
                                        reduced_graph[ins][sp]+= reduced_graph[j][sp]*reduced_graph[ins][j]*scale_factor
                                    else:
                                        reduced_graph[ins][sp] = reduced_graph[j][sp]*reduced_graph[ins][j]*scale_factor
                                    reduced_out[ins].add(sp)
                                    reduced_in[sp].add(ins)
                        else:
                            for sp in avg_cycle_out_graph[j]:
 
                                if sp in reduced_graph[ins]:
                                    reduced_graph[ins][sp]+= avg_cycle_out_graph[j][sp]*reduced_graph[ins][j]
                                else:

                                    reduced_graph[ins][sp] = avg_cycle_out_graph[j][sp]*reduced_graph[ins][j]
                                reduced_out[ins].add(sp)
                                reduced_in[sp].add(ins)
            
                            
                                
                    reduced_out[ins].remove(j)
                    del reduced_graph[ins][j]
                for r in rxn_prod[j]:
                    if j in reduced_mech.reactions[r].prod_dict:
                        
                        mark = False
                        for spe in reduced_mech.reactions[r].reactants:
                            if spe in group_dict_check:
                                if any([t in scc[i] for t in group_dict_check[spe]]):
                                    mark = True
                        mark = True
                        if any([f in scc[i] for f in reduced_mech.reactions[r].reactants]) or mark==True:
                            scale_factor = 1
                            for rec in reduced_mech.reactions[r].reactants:
                                if rec in scc[i]:
                                    if rec in reduced_graph[j]:
                                        scale_factor = abs(1/max(1e-20,1-reduced_graph[j][rec]))
                            for pro in reduced_graph[j]:
                                if pro not in reduced_mech.reactions[r].reactants:
                                    if pro in reduced_mech.reactions[r].prod_dict:
                                        reduced_mech.reactions[r].prod_dict[pro] += reduced_graph[j][pro]*reduced_mech.reactions[r].prod_dict[j]*scale_factor
                                        if j==8:
                                            print(scale_factor, reduced_graph[j][pro], reduced_mech.reactions[r].prod_dict[j])
                                    else:
                                        reduced_mech.reactions[r].prod_dict[pro] = reduced_graph[j][pro]*reduced_mech.reactions[r].prod_dict[j]*scale_factor
                                        if j==8:
                                            print(scale_factor, reduced_graph[j][pro], reduced_mech.reactions[r].prod_dict[j])
                                    rxn_prod[pro].add(r)
                            
                        else:
                            for pro in avg_cycle_out_graph[j]:
                                if pro not in reduced_mech.reactions[r].reactants:
                                    if pro in reduced_mech.reactions[r].prod_dict:
                                        reduced_mech.reactions[r].prod_dict[pro] += avg_cycle_out_graph[j][pro]*reduced_mech.reactions[r].prod_dict[j]
                                    else:
                                        reduced_mech.reactions[r].prod_dict[pro] = avg_cycle_out_graph[j][pro]*reduced_mech.reactions[r].prod_dict[j]
                                    rxn_prod[pro].add(r)
                        del reduced_mech.reactions[r].prod_dict[j]
                scc_rem_removed.add(j)  

    
    
    marked = set(background_spc).union(remaining_specs)
    dag_graph = [{} for i in range(spec_len)]
    dag_out = [set() for i in range(spec_len)]
    dag_in = [set() for i in range(spec_len)]
    count = 0
    
    for i in in_cycle_specs:
        for j in i:
            dicy = {}
            for k in avg_cycle_out_graph[j]:
                if avg_cycle_out_graph[j][k]!=0:
                    dicy[k]=avg_cycle_out_graph[j][k]
                    dag_out[j].add(k)
                    dag_in[k].add(j)
            dag_graph[j] = dicy
            if j==29:
                print('29 stuff',dag_graph[j])
    for i in range(spec_len):
        count+=1
        if i not in scc_set:
            dicy = {}
            for j in reduced_graph[i]:
                if reduced_graph[i][j]!=0:
                    dicy[j]=reduced_graph[i][j]
                    dag_out[i].add(j)
                    dag_in[j].add(i)
            dag_graph[i] = dicy

    remove_these_rxns = set()
    avg_cycle_out_graph_2 = {}
    for i in avg_cycle_out_graph:
        dicy = {}
        for j in avg_cycle_out_graph[i]:
            if avg_cycle_out_graph[i][j]!=0:
                dicy[j]= avg_cycle_out_graph[i][j]
        avg_cycle_out_graph_2[i] = dicy

    for i in range(len(reduced_graph)):
        dicy = {}
        for j in reduced_graph[i]:
            if reduced_graph[i][j]!=0:
                dicy[j]= reduced_graph[i][j]
        reduced_graph[i]=dicy

    print('Stage 6: Creating Category Species')

    
    new_dict = deepcopy(dic)
    
    categories = []
    cat_names = []
    new_species = deepcopy(mechanism.species)
    cat_species_indices = []
    for i in settings['Categories']:
        caty = []
        for j in i[1]:
            caty.append(dic[j])
        categories.append(caty)
        cat_names.append(i[0])
        new_species.append(i[0])
        new_dict[i[0]] = len(new_species)-1
        cat_species_indices.append(len(new_species)-1)
    cat_dict = {}
    cat_set = set()
    count = 0
    for i in categories:
        for j in i:
            cat_dict[j] = count
            cat_set.add(j)
        count+=1
    print('MULTIPLIER', reduced_mech.reactions[177].multiplier)
    cat_specs = []
    cat_mapping = []
    
    #category rate multiplier: do the exact same method as the groups 
    #Basically, take the rate of reaction, multiply it by the % contribution of the species
    # and the % contribution of the rxn type compared to a base rate which is selected as the highest
    # contributor to the category
    # if a product in a reaction is in the category, what ever the moles to in-cat is the fraction
    # within cat, and the rate should be multiplied by 1 - frac, and all products should be multiplied 
    # by 1/(1-frac) except for the within cat which is removed
    all_cat_set = set()
    for i in categories:
        for j in i:
            all_cat_set.add(j)
    multses = []
    mult_list_list = []
    cat_data = []
    multy = []

    for j in range(len(categories)):
        mapping = []
        spec_len = spec_len+1
        dag_out.append(set())
        dag_in.append(set())
        dag_graph.append({})
        new_spec = cat_species_indices[j]
        mapping.append(new_spec)
        cat_specs.append(new_spec)
        all_cat_set.add(new_spec)
        caty = categories[j]
        reduced_mech.species.append(cat_names[j])
        removers = set()
        rxy_prod = set()
        rxy_reac = set()
        net_yield = 0
        net_out_yield = 0
        out_reactions = {}
        multies = []
        data_c = []
        for i in caty:
            
            if i in remaining_specs:
                removers.add(i)
                
            if i in grouped:
                removers.add(i)

        for i in removers:
            caty.remove(i)
        # check all reactions
        #for i in categories[j]:
        cat_map = []
        caty_yields = {}
        caty_yields_all = {}
        caty_yields_cyc = {}
        caty_yields_cyc_all = {}
        max_yi = 0
        if len(caty)==0:
            break
        for i in caty:
            spec_yi = 0
            spec_yi_c = 0
            for x in dag_in[i]:
                if x not in caty:
                    if i in dag_graph[x]:
                        
                        spec_yi += dag_graph[x][i]*avg_yield_o[x]
                        if x in avg_yield_o:
                            spec_yi_c += dag_graph[x][i]*avg_yield_o[x]
                        
            caty_yields[i] = spec_yi
            caty_yields_cyc[i] = spec_yi_c
            caty_yields_all[i] = avg_yield_o[i]
            caty_yields_cyc_all[i] = avg_yield_o[i]
            
            if spec_yi>max_yi:
                max_yi = deepcopy(spec_yi)
                cat_lead = i
        cat_full_yield = sum(caty_yields.values())
        print('Cat full yield', cat_full_yield)#, cat_full_yield_all, cat_full_yield_cyc, cat_full_yield_cyc_all)
        cat_rel_rate = 0
        avg_yield_full[new_spec] = cat_full_yield
        avg_yield_o[new_spec] = cat_full_yield
        cat_rel_out = 0
        cat_weight_list = []
        cat_num_list = []
        for i in caty:
            rates_c = []
            for r in rxn_reac[i]:
                delta = 1
                for x in reduced_mech.reactions[r].prod_dict:
                    if x in caty:
                        delta = delta - reduced_mech.reactions[r].prod_dict[x]
                cat_rel_rate+=np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier*caty_yields[i]/cat_full_yield
                cat_rel_out+= np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier*max(0,delta)*caty_yields[i]/cat_full_yield
                cat_weight_list.append(caty_yields[i]/max(1e-20,cat_full_yield))
                cat_num_list.append(np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier*max(0,delta))
                
                rates_c.append([i,r, reduced_mech.reactions[r].reactants,np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier])
            data_c.append(rates_c)
        #cat_rel_rate = cat_rel_rate/len(caty)
        #cat_rel_out = cat_rel_out/len(caty)
        cdata = pd.DataFrame({'num':cat_num_list,'weight':cat_weight_list})
        if len(cdata)>0:
            cat_median_rate = weighted_median(cdata,'num', 'weight')
        else:
            cat_median_rate = 1e-5
        data_c.append(cat_rel_rate)
        mult_list = []
        for i in caty:
            spec_yield_frac = caty_yields[i]/max(1e-20,cat_full_yield)
            species_rate = 0
            for r in rxn_reac[i]:
                species_rate+= np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier
            if species_rate>0:  
                multies.append([i, spec_yield_frac, cat_rel_rate, species_rate])
                mult = spec_yield_frac*(cat_median_rate)/species_rate
                mult_list.append([i,mult, cat_rel_rate,species_rate, spec_yield_frac])
                
            else:
                mult = 0            
            cat_map.append(i)
            for k in dag_in[i]:
                dag_out[k].add(new_spec)
                dag_in[new_spec].add(k)
                if i in dag_graph[k]:
                    if new_spec not in dag_graph[k]:
                        dag_graph[k][new_spec] = dag_graph[k][i]
                    else:
                        dag_graph[k][new_spec] += dag_graph[k][i]
                else:
                    pass
                    #print('matchning issue', k, i)
                if i in dag_out[k]:
                    dag_out[k].remove(i)
                    if i in dag_graph[k]:    
                        del dag_graph[k][i]
                    else:
                        pass
            skip_r = set()

            for r in rxn_prod[i]:
                
                
                if not any([x in caty for x in reduced_mech.reactions[r].reactants]) and r not in skip_r:
                    

                    rxy_prod.add(r)
                    
                    if i in reduced_mech.reactions[r].prod_dict:
                        if new_spec in reduced_mech.reactions[r].prod_dict:
                            reduced_mech.reactions[r].prod_dict[new_spec] += reduced_mech.reactions[r].prod_dict[i]
                        else:
                            reduced_mech.reactions[r].prod_dict[new_spec] = reduced_mech.reactions[r].prod_dict[i]
                else:
                    remove_these_rxns.add(r)
                
            for r in rxn_reac[i]:
                if any([x not in caty for x in reduced_mech.reactions[r].prod_dict]) and r not in skip_r:
                    delta = 1
                     
                    for x in reduced_mech.reactions[r].prod_dict:
                        if x in caty:
                            delta = delta - reduced_mech.reactions[r].prod_dict[x]
                    if delta>0:
                        out_reactions[r] = delta*avg_yield_full[i]
                        net_out_yield = delta*avg_yield_full[i]
                        rxy_reac.add(r)
                        reduced_mech.reactions[r].multiplier *= mult*delta
                        new_prods = {}
                        p_count = 0
                        for p in reduced_mech.reactions[r].prod_dict:
                            #if i not in scc_set:
                            if p not in caty:
                                p_count+=reduced_mech.reactions[r].prod_dict[p]

                        for p in reduced_mech.reactions[r].prod_dict:
                            if p not in caty:
                                new_prods[p] = reduced_mech.reactions[r].prod_dict[p] + (1 - delta)*(reduced_mech.reactions[r].prod_dict[p]/max(p_count,1e-20))
                            if (1/delta)*reduced_mech.reactions[r].prod_dict[p]>3:
                                #print('maxed it')
                                pass
                        if p_count == 0 and 'XCLOST' in species_list_names:
                                
                            if dic['XCLOST'] in new_prods:
                                new_prods[dic['XCLOST']] += 1-delta
                            else:
                                new_prods[dic['XCLOST']] = 1-delta
                        reduced_mech.reactions[r].prod_dict = new_prods
                    else:
                        remove_these_rxns.add(r)
                    reduced_mech.reactions[r].reactants.append(new_spec)
                    if i in reduced_mech.reactions[r].reactants:
                        reduced_mech.reactions[r].reactants.remove(i)
                    else:
                        print('not in ', r, i)
                elif all([x not in caty for x in reduced_mech.reactions[r].prod_dict]) and r not in skip_r:
                    out_reactions[r] = avg_yield_full[i]
                    net_out_yield = avg_yield_full[i]
                    rxy_reac.add(r)
                    reduced_mech.reactions[r].multiplier *= mult
                    multy.append([r, reduced_mech.reactions[r].multiplier, mult])
                    new_prods = {}
                    if i in scc_set:
                        new_prods = dag_graph[i]
                        reduced_mech.reactions[r].prod_dict = new_prods
                    reduced_mech.reactions[r].reactants.append(new_spec)
                    if i in reduced_mech.reactions[r].reactants:
                        reduced_mech.reactions[r].reactants.remove(i)
                else:
                    
                    for p in reduced_mech.reactions[r].prod_dict:
                        if r in rxn_prod[p]:
                            rxn_prod[p].remove(r)
                    remove_these_rxns.add(r)
        mult_list_list.append(mult_list)
        mapping.append(cat_map)
        net_out_graph = {}
        multses.append(multies)
        # out reactions + graph outs
        tot_cat_rate = 0
        for r in out_reactions:
            tot_cat_rate += np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier
        for r in out_reactions:
            r_eff = np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier/max(1e-20,tot_cat_rate)
            for p in reduced_mech.reactions[r].prod_dict:
                if p in net_out_graph:
                    net_out_graph[p]+= r_eff*reduced_mech.reactions[r].prod_dict[p]
                else:
                    net_out_graph[p] = r_eff*reduced_mech.reactions[r].prod_dict[p]

        for i in caty:
            for k in dag_out[i]:
                if k not in caty:
                    dag_in[k].add(new_spec)
                    dag_out[new_spec].add(k)
                    if i in dag_in[k]:
                        dag_in[k].remove(i)
            for k in dag_in[i]:
                if i in dag_graph[k]:
                    del dag_graph[k][i]
                if i in dag_out[k]:
                    dag_out[k].remove(i)
            dag_in[i] = set()
        dag_graph[new_spec] = net_out_graph
        rxn_reac.append(rxy_reac)
        rxn_prod.append(rxy_prod)
        cat_mapping.append(mapping)
        cat_data.append(data_c)
  
    print('Stage 7: Creating Species Removal Tiers')


    group_set = set()
    group_dict = {}
    for i in range(len(groups)):
        for j in groups[i]:
            group_dict[j] = i
            group_set.add(j)
    remaining_scc = set()
    removed_scc = set()
    remaining_groups = set()
    removed_groups = set()
    group_lead = {}
    reduced_mech_1 = deepcopy(reduced_mech)
    
    #print('remaining', remaining_specs, protected)
    for i in cat_specs:
        remaining_specs.append(i)
    
    for i in remaining_specs:
        if i in scc_set:
            remaining_scc.add(scc_dict_2[i])
        if i in group_set:
            remaining_groups.add(group_dict[i])
            group_lead[group_dict[i]] = i
    #print('REM', remaining_scc)
    for i in grouped:
        if i in scc_set:
            remaining_scc.add(scc_dict_2[i])
    #print('REM', remaining_scc)
    removed_scc = set(range(len(scc))).difference(remaining_scc)
    removed_groups = set(range(len(groups))).difference(remaining_groups)
    #print('scc_removed',[[scc[i],i] for i in removed_scc])
    #print('removed groups', removed_groups)
    
    #print('REACTOIN 141',  reduced_mech.reactions[141].reactants, reduced_mech.reactions[141].prod_dict)
    tiers = [[]]
    cat_set = set()
    group_set = set()
    tier_set = set(background_spc).union(remaining_specs)
    for i in grouped:
        tier_set.add(i)
    #for i in scc_rem_removed:
    #    tier_set.add(i)
    '''for i in range(len(dag_out)):
        ndo = set()
        ndg = {}
        for j in dag_out[i]:
            if j not in background_spc:
                ndo.add(j)
        for j in dag_graph[i]:
            if j not in background_spc:
                ndg[j] = dag_graph[i][j]
        
        dag_out[i] = ndo
        dag_graph[i] = ndg'''
    delta = 1
 
    in_cycle_spec_set = set()
    for i in in_cycle_specs:
        for j in i:
            in_cycle_spec_set.add(j)
    remaining_search = {i for i in range(spec_len) if i not in tier_set}
    #print('SCC REMOVE SPECIES', scc_remove_species)
    for i in scc_remove_species:
        if i not in in_cycle_spec_set:
            if i in remaining_search:
                remaining_search.remove(i)
    for i in cat_set:
        if i not in not_removed and i in remaining_search:
            remaining_search.remove(i)
    for i in group_set:
        if i not in not_removed and i in remaining_search:
            remaining_search.remove(i)
    scc_remove_no_in = set()
    for i in scc_remove_species:
        if i not in in_cycle_spec_set:
            scc_remove_no_in.add(i)
    
    #print('1282',remove_these_rxns)                

    remove_1 = deepcopy(remove_these_rxns)
    #print('REM SEARCH', remaining_search)
    #print('12 DAG out', dag_out[12])
    while delta>0:
        new_to_add = set()
        tier = set()
        to_remove = set()
        for j in remaining_search:
            if all([x in tier_set for x in dag_out[j]]):
                new_to_add.add(j)
                to_remove.add(j)
                if j not in cat_set and j not in group_set and j not in scc_remove_no_in:
                    tier.add(j)
        for p in to_remove:
            remaining_search.remove(p)
        delta = len(tier)
        tiers.append(tier)
        tier_set = tier_set.union(new_to_add)
        #print('TO REMOVE', to_remove)
    if len(remaining_search)>0:
        tiers.append(remaining_search)
    #return tiers
    t1 = time.time()
    #print('Tiers Finished', t1-t0)
    #remove_species = set(ordered_species[:-red_mech_size]).difference(scc_remove_species)
    
    dag_graph_save = deepcopy(dag_graph)
    #return graphs
    # first do scc species, because they don't go one at a time
    # second do other non full scc species. Both of these require updates to both the reduced_graph and the reaction list
    # grouping species does not require updates to the reduced_graph, and can be done last. 
    tt1 = time.time()
    #print('grouped',grouped)
    #removed_rxns = set()
    #print('to in cycle specs')
    count = 0
    dag_graph_save = deepcopy(dag_graph)

    #return dag_graph, dag_in, dag_out, tiers, scc, in_cycle_specs
    dag_graph_save_2 = deepcopy(dag_graph)
    t2 = time.time()
    #print('cycle time',t2-t1)
    remove_2 = deepcopy(remove_these_rxns)
    #remove_species = set(ordered_species[:-red_mech_size])
    normal_remove = deepcopy(remove_species)
    normal_remove = set(range(spec_len))
    print('this far')
    removed_species = set()
    true_count = 0
    false_count =0
    remaining_specs = set(remaining_specs)
    T1 = 0
    T2 = 0
    T3 = 0
    T4 = 0
    T5 = 0

    remove_3 = deepcopy(remove_these_rxns)
    
    print('Stage 8: Removing Remaining Species')

    reduced_mech_1 = deepcopy(reduced_mech)
    for t in tiers:
        for i in t:
            if i in scc_set:
                scc_indy = scc_dict_2[i]
                if i not in grouped:
    
                    count+=1
                    if count%1000==0:
                        print(count)
                    t1 = time.time()
                    for k in dag_in[i]:
                        if k not in scc[scc_indy]:
    
                            if i in dag_graph[k]:
                                mult = dag_graph[k][i]
                            else:
                                mult = 0
    
                            for p in dag_graph[i]:
                                dag_in[p].add(k)
                                dag_out[k].add(p)
                                if p not in dag_graph[k]:
                                    dag_graph[k][p] = mult*dag_graph[i][p]
                                else:
                                    dag_graph[k][p] += mult*dag_graph[i][p]
    
                            if i in dag_out[k]:
                                dag_out[k].remove(i)
                            if i in dag_graph[k]:
                                del dag_graph[k][i]    

    
                    for r in rxn_prod[i]:
                        if r not in removed_rxns:
                            if not any([x in scc[scc_indy] for x in reduced_mech.reactions[r].reactants]):
                                if i in reduced_mech.reactions[r].prod_dict:
                                    mult = reduced_mech.reactions[r].prod_dict[i]
                                else:
                                    mult = 0
                                for p in dag_graph[i]:
                                    if p not in reduced_mech.reactions[r].reactants and p not in scc[scc_indy]:
                                        if p in reduced_mech.reactions[r].prod_dict:
                                            reduced_mech.reactions[r].prod_dict[p]+= mult*dag_graph[i][p]
                                        else:
                                            reduced_mech.reactions[r].prod_dict[p] = mult*dag_graph[i][p]
                                        rxn_prod[p].add(r)
                    for r in rxn_reac[i]:
                        removed_rxns.add(r)
            else:

                removed_species.add(i)
                count+=1
    
                removers = set()
                for j in dag_in[i]:
                    t1 = time.time()
                    if i in dag_out[j]:
                        dag_out[j].remove(i)
                    if i in dag_graph[j]:
                        mult = dag_graph[j][i]
                        del dag_graph[j][i]
                    else:
                        mult = 0
    
                    t2 = time.time()
                    for k in dag_out[i]:
                        
                        if k!=j and dag_graph[i][k]*mult!=0:
                            t4 = time.time()
                            dag_out[j].add(k)
                            t5 = time.time()
                            dag_in[k].add(j)
                            t6 = time.time()
                            if k in dag_graph[j]:
                                dag_graph[j][k] += dag_graph[i][k]*mult
                            else:
                                dag_graph[j][k] = dag_graph[i][k]*mult
                            t7 = time.time()
                            T3 += t5-t4
                            T4 += t6-t5
                            T5 += t7-t6
     
                    removers.add(j)
                    
                    t3 = time.time()
                    T1+= t2-t1
                    T2+=t3-t2
    
                if any([x in remaining_specs for x in dag_in[i]]):
                    true_count+=1
                    
                    for r in rxn_prod[i]:
    
                        if r not in removed_rxns:
    
                            if i in reduced_mech.reactions[r].prod_dict:
                                mult = reduced_mech.reactions[r].prod_dict[i]
                            else:
                                mult = 0
                            for j in dag_graph[i]:
    
                                if j not in reduced_mech.reactions[r].reactants and mult*dag_graph[i][j]!=0:
    
                                    rxn_prod[j].add(r)
                                    if j in reduced_mech.reactions[r].prod_dict:
                                        reduced_mech.reactions[r].prod_dict[j] += mult*dag_graph[i][j]
                                    else:
                                        reduced_mech.reactions[r].prod_dict[j] = mult*dag_graph[i][j]
                    for r in rxn_reac[i]:
    
                        removed_rxns.add(r)
                for p in removers:
                    dag_in[i].remove(p)



    reduced_mech_3 = deepcopy(reduced_mech)
    
    remove_4 = deepcopy(remove_these_rxns)
    t4 = time.time()
    for i in reduced_mech.reactions:
        if 10 in i.reactants:
            print(i.reactants, i.multiplier)

    print('Stage 9: Mechanism Clean Up')
    remove_set = set(remove_species)
    back_set_set = set(back_set)
    full_set = remove_set.union(back_set_set)
    
    for i in range(len(reduced_mech.reactions)):
        
        if i%10000==0:
            print(i)
        for j in reduced_mech.reactions[i].reactants:
            if j in remove_set:
                remove_these_rxns.add(i)

    new_reactions = []
    count = 0
    for i in range(len(reduced_mech.reactions)):
        if i not in remove_these_rxns:
            
            if i==141:
                indy = count
            new_reactions.append(deepcopy(reduced_mech.reactions[i]))
            count+=1

    for i in new_reactions:
        if 10 in i.reactants:
            print(i.reactants)
    tx = 0
    ty = 0
    tz = 0

    remove_species = set(remove_species)

    for i in range(len(new_reactions)):
        if i%1000==0:
            print(i)
        t1 = time.time()
        new_reactants = deepcopy(new_reactions[i].reactants)
  
        t2 = time.time()
        new_prod_dict = {}
        for j in new_reactions[i].prod_dict:
            if new_reactions[i].prod_dict[j]>0.001 and j not in remove_species:

                new_prod_dict[j] = new_reactions[i].prod_dict[j]
        new_reactions[i].prod_dict = new_prod_dict
        t3 = time.time()
        tx+=t2-t1
        ty+=t3-t2

    
    for i in new_reactions:
        if 10 in i.reactants:
            print(i.reactants)

    new_new_reactions = []
    reduced_mech_1 = deepcopy(reduced_mech)
    for i in new_reactions:
        if i.multiplier>0:
            new_new_reactions.append(i)

    new_reactions = deepcopy(new_new_reactions)
    
    remove_5 = deepcopy(remove_these_rxns)

    reduced_mech.reactions = deepcopy(new_reactions)
   


    accounted_specs = set()
    new_cat_mapping = []
    map_dict = {}
    for i in background_spc:
        map_dict[i] = i
    for i in cat_mapping:
        if len(i[1])>0:
            accounted_specs.add(i[0])
            new_cat_mapping.append(i)
            for k in i[1]:
                map_dict[k] = i[0]
    new_group_mapping = []
    group_map_dict = {}
    for i in group_mapping:
        if len(i[1])>0:
            accounted_specs.add(i[0])
            new_group_mapping.append(i)
            group_map_dict[i[0]] = i[1]
            for k in i[1]:
                map_dict[k] = i[0]
        map_dict[i[0]] = i[0]
            
    normal_specs = []
    for i in remaining_specs:
        if i not in accounted_specs:
            normal_specs.append(i)
            map_dict[i] = i
    represented_specs = []
    for i in new_group_mapping:
        represented_specs.append(i[0])
        for j in i[1]:
            represented_specs.append(j)
    for i in new_cat_mapping:
        for j in i[1]:
            represented_specs.append(j)
    for i in normal_specs:
        represented_specs.append(i)
    full_map = [new_group_mapping, new_cat_mapping, normal_specs]
    map_yields = [[] for i in range(len(represented_specs))]
    inv_map_dict = {}
    for i in map_dict:
        if map_dict[i] in inv_map_dict:
            inv_map_dict[map_dict[i]].add(i)
        else:
            inv_map_dict[map_dict[i]] = {i}
    #print('70721 STUFF', map_dict[70721])
    represented_set = set(represented_specs)
    
    


    print('Stage 10: Analyzing Reduced Mechanism')

        
    red_graphs = [[{} for i in range(spec_len)] for c in range(l_c)]
    red_in_graph = [set() for i in range(spec_len)]
    red_out_graph = [set() for i in range(spec_len)]

    
    red_dag_out = []
    new_rxns = []
    for i in reduced_mech.reactions:
        if i.prod_dict!={}:
            new_rxns.append(i)

    reduced_mech.reactions = deepcopy(new_rxns)
    red_specs = []
    for i in new_dict:
        if new_dict[i] in remaining_specs:
            red_specs.append(new_dict[i])

    red_rxn_reac = {}
    red_rxn_prod = {}
    red_spec_dict = {}
    for i in range(len(red_specs)):
        red_spec_dict[red_specs[i]] = i
    for i in range(len(reduced_mech.reactions)):
        for j in reduced_mech.reactions[i].reactants:
            if j not in back_set:
                if j in red_rxn_reac:  
                    red_rxn_reac[j].add(i)
                else:
                    red_rxn_reac[j] = {i}
        for p in reduced_mech.reactions[i].prod_dict:
            if p not in back_set:
                if p in red_rxn_prod:
                    red_rxn_prod[p].add(i)
                else:
                    red_rxn_prod[p] = {i}
    for i in red_rxn_reac:
        if i not in back_set:
            edges = [{} for c in range(l_c)]
            rate_sums = [0 for c in range(l_c)]
            
            for j in red_rxn_reac[i]:

                for c in range(l_c):
                    rate_sums[c] = rate_sums[c] + reduced_mech.reactions[j].rate[c]
            for j in red_rxn_reac[i]:
                mults = [0 for c in range(l_c)]
                for c in range(l_c):
                    mults[c] = reduced_mech.reactions[j].rate[c]/max(1e-20,rate_sums[c])
                if len(reduced_mech.reactions[j].reactants) == 1:
                    type_r = 'solo'
                else:
                    mark = True
                    for p in reduced_mech.reactions[j].reactants:
                        if p in background_spc:
                            type_r = p
                            mark = False
                    if mark:
                        type_r = 'double'
                for k in reduced_mech.reactions[j].prod_dict:
                    
                    for c in range(l_c):
                        if k in edges[c]:
                            if c==0:
                                edges[c][k] = edges[c][k] + reduced_mech.reactions[j].prod_dict[k]*mults[c]
                                red_out_graph[i].add(k)
                                red_in_graph[k].add(i)
                            else:
                                edges[c][k] = edges[c][k] + reduced_mech.reactions[j].prod_dict[k]*mults[c]

                        
                        else:
                            if c==0:
                                edges[c][k] = reduced_mech.reactions[j].prod_dict[k]*mults[c]
                                red_out_graph[i].add(k)
                                red_in_graph[k].add(i)
                            else:
                                edges[c][k] = reduced_mech.reactions[j].prod_dict[k]*mults[c]
            for c in range(l_c):
                red_graphs[c][i] = edges[c]
    
    
    

    for i in red_rxn_reac:
        if i not in back_set:
            edges = [{} for c in range(l_c)]
            rate_sums = [0 for c in range(l_c)]
            
            for j in red_rxn_reac[i]:

                for c in range(l_c):
                    rate_sums[c] = rate_sums[c] + reduced_mech.reactions[j].rate[c]*reduced_mech.reactions[j].multiplier
            for j in red_rxn_reac[i]:
                mults = [0 for c in range(l_c)]
                for c in range(l_c):
                    mults[c] = reduced_mech.reactions[j].multiplier*reduced_mech.reactions[j].rate[c]/max(1e-20,rate_sums[c])
                if len(reduced_mech.reactions[j].reactants) == 1:
                    type_r = 'solo'
                else:
                    mark = True
                    for p in reduced_mech.reactions[j].reactants:
                        if p in background_spc:
                            type_r = p
                            mark = False
                    if mark:
                        type_r = 'double'
                for k in reduced_mech.reactions[j].prod_dict:
                    
                    for c in range(l_c):
                        if k in edges[c]:
                            if c==0:
                                edges[c][k] = edges[c][k] + reduced_mech.reactions[j].prod_dict[k]*mults[c]
                                red_out_graph[i].add(k)
                                red_in_graph[k].add(i)
                            else:
                                edges[c][k] = edges[c][k] + reduced_mech.reactions[j].prod_dict[k]*mults[c]

                        
                        else:
                            if c==0:
                                edges[c][k] = reduced_mech.reactions[j].prod_dict[k]*mults[c]
                                red_out_graph[i].add(k)
                                red_in_graph[k].add(i)
                            else:
                                edges[c][k] = reduced_mech.reactions[j].prod_dict[k]*mults[c]
            for c in range(l_c):
                red_graphs[c][i] = edges[c]
    
    red_graph_mat_lil = lil_matrix((spec_len,spec_len), dtype=None)
    

        
    red_edges = []
    for i in range(len(red_out_graph)):
        for j in red_out_graph[i]:
            red_edges.append((i,j))
    count = 0
    for i,j in red_edges:
        red_graph_mat_lil[i,j] = 1
    reduced_mech_4 = deepcopy(reduced_mech)
    red_graph_mat = red_graph_mat_lil.tocsr(copy=False)

    red_scc_result = connected_components(red_graph_mat, directed=True, connection='strong', return_labels=True)

    red_scc_dic = {}
    for i in range(len(red_scc_result[1])):
        if red_scc_result[1][i] in red_scc_dic:
            red_scc_dic[red_scc_result[1][i]].append(i)
        else:
            red_scc_dic[red_scc_result[1][i]] = [i]
    #print('red scc dic', red_scc_dic)
            # tbd whether we want cateogires to participate in cycles with each other, such as mid vol <--> high vol
    red_scc = []
    
    for i in red_scc_dic:
        if len(red_scc_dic[i])>1:
            if not any([x in cat_specs for x in red_scc_dic[i]]):
                pass
            else:
                c_specs = []
                for x in red_scc_dic[i]:
                    if x in cat_specs:
                        c_specs.append(x)
                not_allowed = set()
                for x in red_scc_dic[i]:
                    if x not in c_specs:
                        not_allowed.add(x)
                for c in c_specs:
                    new_out_g = set()
                    for j in red_out_graph[c]:
                        if j not in not_allowed:
                            new_out_g.add(j)
                        else:
                            if c in red_in_graph[j]:
                                red_in_graph[j].remove(c)
                    red_out_graph[c] = new_out_g

                    for cond in range(l_c):
                        new_graph_c = {}
                        for j in red_graphs[cond][c]:
                            if j not in not_allowed:
                                new_graph_c[j] = red_graphs[cond][c][j]
                        red_graphs[cond][c] = deepcopy(new_graph_c)
                    
                    for r in red_rxn_reac[c]:
                        new_p = {}
                        for p in reduced_mech.reactions[r].prod_dict:
                            if p not in not_allowed:
                                new_p[p] = reduced_mech.reactions[r].prod_dict[p]
                            else:
                                red_rxn_prod[p].remove(r)
                        reduced_mech.reactions[r].prod_dict = new_p

    red_graph_mat_lil = lil_matrix((spec_len,spec_len), dtype=None)
    red_edges = []
    for i in range(len(red_out_graph)):
        for j in red_out_graph[i]:
            red_edges.append((i,j))
    count = 0
    for i,j in red_edges:
        red_graph_mat_lil[i,j] = 1
    reduced_mech_4 = deepcopy(reduced_mech)
    red_graph_mat = red_graph_mat_lil.tocsr(copy=False)

    red_scc_result = connected_components(red_graph_mat, directed=True, connection='strong', return_labels=True)

    red_scc_dic = {}
    for i in range(len(red_scc_result[1])):
        if red_scc_result[1][i] in red_scc_dic:
            red_scc_dic[red_scc_result[1][i]].append(i)
        else:
            red_scc_dic[red_scc_result[1][i]] = [i]
    for i in red_scc_dic:
        if len(red_scc_dic[i])>1:
            red_scc.append(red_scc_dic[i])
            
 

    red_scc_dict_2 = {}
    red_scc_set = set()
    for i in range(len(red_scc)):
        for j in red_scc[i]:
            red_scc_dict_2[j] = i
            red_scc_set.add(j)
    red_scc_map = {}
    for i in red_scc_set:
        red_scc_map[i] = inv_map_dict[i]
    

    print('Stage 11: Optimizing Cyclical Rate Constants')

    scc_ref_data = {}
    represented_scc = set()
    for i in red_scc_map:
        for j in red_scc_map[i]:
            if j in scc_dict_2:
                represented_scc.add(scc_dict_2[j])
    if settings['Iterations']>0:
        for i in represented_scc:
            print(i)
            ref_data = []
            for c in range(l_c):
                print('len', set(scc[i]).union(scc_out_specs[i]),scc_out_specs[i])
                in_specs = []
                for k in in_cycle_specs[i]:
                    k_start = 0
                    for s in in_graph[k]:
                       
                        if s not in scc[i]:
                            if k in graphs[c][s]:
                                k_start+=graphs[c][s][k]*yields[c][s]
                    in_specs.append([k,k_start])
                    
                data, within_data, out_data = cycle_simulator_3_modified(scc[i], in_specs, graphs[c],out_graph,in_graph, [leny+40,2*(leny+40)],1e-6,set(scc[i]).union(scc_out_specs[i]),scc_out_specs[i])
    
                
                new_data = {}
                for d in data:
                    if d not in represented_set:
                        
                        yi = get_yields_modified([d], represented_set, 1e-10,new_graphs[c], new_ins[c], new_outs[c], back_set,scc, scc_dict_2, scc_set)
                        for y in yi:
                            if yi[y]!=0:
                                if y in new_data:
                                    new_data[y]+=data[d]*yi[y]
                                else:
                                    new_data[y]=data[d]*yi[y]
                    else:
                        if d in new_data:
                            new_data[d]+= data[d]
                        else:
                            new_data[d]= data[d]
    
                ref_data.append([new_data, within_data, out_data])
            scc_ref_data[i] = ref_data

    red_dag_out = deepcopy(red_out_graph)
    red_dag_in = deepcopy(red_in_graph)
    xspecs = []
    for i in red_scc:
 
        xspec = i[0]
        xspecs.append(i[0])
        for j in i:
            if j != xspec:
                for x in red_dag_out[j]:
                    if x not in i:
                        red_dag_out[xspec].add(x)
                        red_dag_in[x].add(xspec)
                    if j in red_dag_in[x]:
                        red_dag_in[x].remove(j)
                for x in red_dag_in[j]:
                    red_dag_out[x].add(xspec)
                    red_dag_in[xspec].add(x)
                    red_dag_out[x].remove(j)
                red_dag_out[j] = set()
                red_dag_in[j] = set()

    for i in range(len(red_dag_in)):
        if i in red_dag_in[i]:
            red_dag_in[i].remove(i)
        if i in red_dag_out[i]:
            red_dag_out[i].remove(i)

    searcher = set(deepcopy(roots))
    dont_visit = set()
    for i in range(len(red_dag_in)):
        if len(red_dag_in[i])==0:
            searcher.add(i)
            dont_visit.add(i)
    visited = []
    visit_set = set()
    while len(searcher)>0:
        
        new_searcher = deepcopy(searcher)
        for i in searcher:
            if i not in visit_set:
                if i not in dont_visit:
                    visited.append(i)
                visit_set.add(i)
            new_searcher.remove(i)
            for j in red_dag_out[i]:
                if all([x in visit_set for x in red_dag_in[j]]):
                    if j not in back_set and j not in visit_set:
                        new_searcher.add(j)
        searcher = deepcopy(new_searcher)
    
 
    red_scc_ins = []
    for i in red_scc:
        in_specy = set()
        for k in i:
            for j in red_in_graph[k]:
                if j not in i:
                    in_specy.add(j)
        red_scc_ins.append(in_specy)
        
    in_specy_set = set()
    for i in range(len(red_scc_ins)):
        for j in red_scc_ins[i]:
            in_specy_set.add(j)

    red_scc_order = []

    count = 0

    for i in visited:
        if i in red_scc_set:
            print('yes')
            red_scc_order.append(red_scc_dict_2[i])
    
    red_scc_ins_2 = []
    for i in red_scc:
        insy = set()
        for j in i:
            mark = False
            for k in red_in_graph[j]:
                if k not in i:
                    mark = True
            if mark:
                insy.add(j)
        red_scc_ins_2.append(insy)
    
    
    scc_ref_data_2 = {}

    new_red_scc_order = []
    for i in red_scc_order:
        if not any([x in cat_specs for x in red_scc[i]]):
            new_red_scc_order.append(i)
    red_scc_order = deepcopy(new_red_scc_order)

    for m in range(len(red_scc_order)):
        m_para = {}
        for n in range(len(reduced_mech.reactions)):
            if any([f in reduced_mech.reactions[n].reactants for f in red_scc[red_scc_order[m]]]):
                mark = True
                
                if mark:
                    m_para[n] = reduced_mech.reactions[n].multiplier
        print('Constants to Optimize: ',m,len(m_para))

    
    scc_sets = []
    for i in scc:
        scc_sets.append(set(i))
    red_scc_sets = []
    for i in red_scc:
        red_scc_sets.append(set(i))
    for i in range(len(red_scc)):
        if red_scc_sets[i] in scc_sets:
            if i in red_scc_order:
                red_scc_order.remove(i)
    if settings['Iterations']>0:
        for m in range(len(red_scc_order)):
            
            scc_rem = red_scc_order[m]
            stop_specs = set()
            for x in range(len(red_dag_out)):
                if len(red_dag_out[x])==0:
                    stop_specs.add(x)
            for x in red_scc_order[m:]:
                for j in red_scc[x]:
                    stop_specs.add(j)
            rep_sccs = set()
            for j in red_scc[red_scc_order[m]]:
                for k in red_scc_map[j]:
                    if k in scc_dict_2:
                        rep_sccs.add(scc_dict_2[k])
            data = []
            for c in range(l_c):
                daty = [{}, {}, {}]
                for f in rep_sccs:
                    for p in scc_ref_data[f][c][0]:
    
                        if p in daty:
                            daty[0][p]+= scc_ref_data[f][c][0][p]
                        else:
                            daty[0][p]= scc_ref_data[f][c][0][p] 
                    for p in scc_ref_data[f][c][1]:
    
                        if p in daty:
                            daty[1][p]+= scc_ref_data[f][c][1][p]
                        else:
                            daty[1][p]= scc_ref_data[f][c][1][p] 
                    for p in scc_ref_data[f][c][2]:
    
                        if p in daty:
                            daty[2][p]+= scc_ref_data[f][c][2][p]
                        else:
                            daty[2][p]= scc_ref_data[f][c][2][p] 
                for sp in red_scc[red_scc_order[m]]:
                    if sp not in daty[0]:
                        daty[0][sp] = 0
                    if sp not in daty[1]:
                        daty[1][sp] = 0
                    if sp not in daty[2]:
                        daty[2][sp] = 0
                data.append(daty)
            scc_ref_data_2[red_scc_order[m]] = data
            #print(scc_ref_data_2[red_scc_order[i]])
            #need to get avg red graph
            #print('here 1')
    
            all_starts = []
            for c in range(l_c):
                new_graph = []
                for p in red_graphs[c]:
                    dicy = {}
                    for j in p:
                        dicy[j] = p[j]
                    new_graph.append(dicy)
    
    
                new_in = []
                for p in red_in_graph:
                    sety = set()
                    for j in p:
                        sety.add(j)
                    new_in.append(sety)
    
                new_out = []
                for p in red_out_graph:
                    sety = set()
                    for j in p:
                        sety.add(j)
                    new_out.append(sety)
    
    
                for n in range(len(red_scc)):
                    counte = 0
                    out_specs = set()
                    all_specs = set(red_scc[n])
                    for x in red_scc[n]:
                        for j in red_out_graph[x]:
                            if j not in red_scc[n]:
                                out_specs.add(j)
                                all_specs.add(j)
                    for x in red_scc_ins_2[n]:
                        counte +=1
                        new_out[x] = all_specs
                        leny = int(np.sqrt(len(red_scc[n])))
                        data, in_cyc_data = cycle_simulator_3(red_scc[n], x, red_graphs[c],red_out_graph,red_in_graph, [leny+40,2*(leny+40)],1e-6,all_specs,out_specs)
                        cycle_out_graphs[c][x] = data
                        ful_dat = {}
                        for p in data:
                            ful_dat[p] = data[p]
                        for p in in_cyc_data:
                            ful_dat[p] = in_cyc_data[p]
                        new_graph[x] = ful_dat
    
                    for y in red_scc[m]:
                        if y not in red_scc_ins_2[red_scc_order[m]]:
    
                            for s in out_graph[y]:
                                if y in new_in[s]:
                                    new_in[s].remove(y)
                            new_in[y] = set(red_scc_ins_2[red_scc_order[m]])
                            new_out[y] = set()
                            new_graph[y] = set()
                            
                graph_mat_lil = lil_matrix((spec_len,spec_len), dtype=None)
    
                edges = []
                for i in range(len(new_out)):
                    for j in new_out[i]:
                        edges.append((i,j))
                #print('DAG OUT 235', dag_graph[235])
                count = 0
                for i,j in edges:
                    graph_mat_lil[i,j] = 1
            
                graph_mat = graph_mat_lil.tocsr(copy=False)
            
                scc_result = connected_components(graph_mat, directed=True, connection='strong', return_labels=True)
            
                scc_dic = {}
                for i in range(len(scc_result[1])):
                    if scc_result[1][i] in scc_dic:
                        scc_dic[scc_result[1][i]].append(i)
                    else:
                        scc_dic[scc_result[1][i]] = [i]
    
                yi = get_yields_modified(roots, stop_specs, 1e-10, new_graph, new_in, new_out, back_set,red_scc, red_scc_dict_2, red_scc_set)
               

                starters = []
                for j in red_scc_ins_2[red_scc_order[m]]:
                    start_val = 0
                    for k in red_in_graph[j]:
                        if k not in red_scc[red_scc_order[m]]:
                            if j in red_graphs[c][k] and k in yi:
                                start_val+= red_graphs[c][k][j]*yi[k]
                    
                    starters.append([j, start_val])
                all_starts.append(starters)
            data_map = []
            for j in red_scc[red_scc_order[m]]:
    
                mapper1 = [1]
                mapper2 = [0]
                mapper1.append(j)
                mapper2.append(j)
                comps = []
                
                for k in red_scc_map[j]:
                    comps.append([k,1])

                mapper1.append(comps)
                mapper2.append(comps)
                data_map.append(mapper1)
            out_specs = set()
            all_specs = set(red_scc[red_scc_order[m]])
            for x in red_scc[red_scc_order[m]]:
                for j in red_out_graph[x]:
                    if j not in red_scc[red_scc_order[m]]:
                        out_specs.add(j)
                        all_specs.add(j)
    
            for j in out_specs:
    
                mapper2 = [0]
                mapper2.append(j)
                comps = []
                denom = 0
                for k in inv_map_dict[j]:
                    denom+=avg_yield_o[k]
                for k in inv_map_dict[j]:
                    comps.append([k,avg_yield_o[k]/max(1e-20,denom)])

                mapper2.append(comps)
                data_map.append(mapper2)
            
            cycle_info = [red_scc[red_scc_order[m]], all_starts, all_specs, out_specs]
            settings_des = {}
            settings_des['learn_rate'] = 0.0002
            settings_des['steps'] = settings['Iterations']
            settings_des['grad_step'] = 1.001

            m_para_map = {}
            m_para = {}
            for n in range(len(reduced_mech.reactions)):
                if any([f in reduced_mech.reactions[n].reactants for f in red_scc[red_scc_order[m]]]):
                    mark = True

                    if mark:
                        m_para_map[n] = [n]
                        m_para[n] = reduced_mech.reactions[n].multiplier
    
            reduced_mech = grad_descent_cycles(m_para, m_para_map, reduced_mech, scc_ref_data_2[red_scc_order[m]], data_map, cycle_info, settings_des, conditions, red_specs, back_set, len(new_species), red_scc[scc_rem])
            
            red_graphs = [[{} for i in range(spec_len)] for c in range(l_c)]
            red_in_graph = [set() for i in range(spec_len)]
            red_out_graph = [set() for i in range(spec_len)]
            for nn in red_rxn_reac:
                if nn not in back_set:
                    edges = [{} for c in range(l_c)]
                    rate_sums = [0 for c in range(l_c)]
    
                    for j in red_rxn_reac[nn]:
    
                        for c in range(l_c):
                            rate_sums[c] = rate_sums[c] + reduced_mech.reactions[j].multiplier*reduced_mech.reactions[j].rate[c]
                    for j in red_rxn_reac[nn]:
                        mults = [0 for c in range(l_c)]
                        for c in range(l_c):
                            mults[c] = reduced_mech.reactions[j].multiplier*reduced_mech.reactions[j].rate[c]/max(1e-20,rate_sums[c])
                        if len(reduced_mech.reactions[j].reactants) == 1:
                            type_r = 'solo'
                        else:
                            mark = True
                            for p in reduced_mech.reactions[j].reactants:
                                if p in background_spc:
                                    type_r = p
                                    mark = False
                            if mark:
                                type_r = 'double'
                        for k in reduced_mech.reactions[j].prod_dict:
    
                       
                            for c in range(l_c):
                                if k in edges[c]:
                                    if c==0:
                                        edges[c][k] = edges[c][k] + reduced_mech.reactions[j].prod_dict[k]*mults[c]
                                        red_out_graph[nn].add(k)
                                        red_in_graph[k].add(nn)
                          
                                    else:
                                        edges[c][k] = edges[c][k] + reduced_mech.reactions[j].prod_dict[k]*mults[c]
    
    
                                else:
                                    if c==0:
                                        edges[c][k] = reduced_mech.reactions[j].prod_dict[k]*mults[c]
                                        red_out_graph[nn].add(k)
                                        red_in_graph[k].add(nn)
                                      
                                    else:
                                        edges[c][k] = reduced_mech.reactions[j].prod_dict[k]*mults[c]
                    for c in range(l_c):
                        red_graphs[c][nn] = edges[c]
    
    print('Stage 12: Getting Reduced Mechanism Yield')

    
    red_yields = []     
    for c in range(l_c):
        new_graph = []
        for p in red_graphs[c]:
            dicy = {}
            for j in p:
                dicy[j] = p[j]
            new_graph.append(dicy)


        new_in = []
        for p in red_in_graph:
            sety = set()
            for j in p:
                sety.add(j)
            new_in.append(sety)

        new_out = []
        for p in red_out_graph:
            sety = set()
            for j in p:
                sety.add(j)
            new_out.append(sety)


        for n in range(len(red_scc)):
            counte = 0
            out_specs = set()
            all_specs = set(red_scc[n])
            for x in red_scc[n]:
                for j in red_out_graph[x]:
                    if j not in red_scc[n]:
                        out_specs.add(j)
                        all_specs.add(j)
            for x in red_scc_ins_2[n]:
                counte +=1
                new_out[x] = all_specs
                leny = int(np.sqrt(len(red_scc[n])))
                data, in_cyc_data = cycle_simulator_3(red_scc[n], x, red_graphs[c],red_out_graph,red_in_graph, [leny+40,2*(leny+40)],1e-6,all_specs,out_specs)
                cycle_out_graphs[c][x] = data
                ful_dat = {}
                for p in data:
                    ful_dat[p] = data[p]

                new_graph[x] = ful_dat

            for y in red_scc[n]:
                if y not in red_scc_ins_2[n]:

                    for s in red_out_graph[y]:
                        if y in new_in[s]:
                            new_in[s].remove(y)
                    new_in[y] = set(red_scc_ins_2[n])
                    new_out[y] = set()
                    new_graph[y] = set()


        yi = get_yields_modified(roots, [], 0, new_graph, new_in, new_out, back_set,red_scc, red_scc_dict_2, red_scc_set)
        red_yields.append(yi)

    if settings['Reduce Stiffness'] == True:
        
        real_rates = []
        count = 0
        for i in reduced_mech.reactions:
            real_rates.append([count, np.mean(i.rate)*i.multiplier])
            count+=1
        sorted_rates = sorted(real_rates, key=lambda x: x[1]) 
        sorted_rates_only = [i[1] for i in sorted_rates]
        if 'Reference Rate' in settings:
            ref = settings['Reference Rate']
        else:
            ref = np.median(sorted_rates_only)
        
        differences = [math.log10(max(i[1], 1e-20))- math.log10(ref) for i in sorted_rates]
        slow_these = []
        slow_these_rxns = set()
        accounted = set()
        for i in range(len(differences)):
            if differences[i]>settings['Stiffness Threshold']:
                slow_these.append(sorted_rates[i])
                slow_these_rxns.add(sorted_rates[i][0])
        #print('slow these', slow_these_rxns)
        numba = 10000000000000
        for s in red_rxn_reac:
            check = [x in slow_these_rxns for x in red_rxn_reac[s]]
            if all(check):
                rates_check = [real_rates[r][1] for r in red_rxn_reac[s]]
                rate_mult = (10**settings['Stiffness Threshold'])*ref/max(rates_check)
                for r in red_rxn_reac[s]:
                    if r not in accounted:
                        accounted.add(r)
                        reduced_mech.reactions[r].multiplier*= rate_mult
                        #print('RATE MULT', rate_mult, reduced_mech.reactions[r].reactants)
            '''elif any(check):
                pass
                print('checking')
                rates_check = []
                for x in red_rxn_reac[s]:
                    if x not in accounted:
                        rates_check.append(real_rates[x])
                        accounted.add(x)
                sorted_check = sorted(rates_check, key=lambda x: x[1], reverse = True) 
                marks = []
                multer = []
                print(sorted_check)
                for j in range(len(sorted_check)):
                    if sorted_check[j][0] in slow_these_rxns:
                        if j>0 and sorted_check[j][1]/max(1e-20,sorted_check[j-1][1])<1/numba:
                            marks.append(j)
                            min_mult = numba*sorted_check[j][1]/max(1e-20,sorted_check[j-1][1])
                            multer.append(min_mult)
                        print('in slow', sorted_check[j][1])
                    elif j>0 and sorted_check[j][1]/max(1e-20,sorted_check[j-1][1])>1/numba:
                        print('is close', sorted_check[j][1], sorted_check[j][1]/max(1e-20,sorted_check[j-1][1]))
                    else:
                        marks.append(j)
                        min_mult = numba*sorted_check[j][1]/max(1e-20,sorted_check[j-1][1])
                        multer.append(min_mult)
                        print('max mult', min_mult)
                        break
                multer_2 = []
                for i in range(len(multer)):
                    multer_2.append(np.prod(multer[i:]))
                multer = deepcopy(multer_2)
                print('multer', marks, multer)
                count = 0
                if len(multer)>0:
                    for j in range(len(sorted_check)):
                        print(j, count)
                        if j<marks[count]:
                            reduced_mech.reactions[sorted_check[j][0]].multiplier *= multer[count]
                            print('sorty', sorted_check[j][1], multer[count])
                        elif count+1<len(marks):
                            count+=1
                            reduced_mech.reactions[sorted_check[j][0]].multiplier *= multer[count]
                            print('sorty', sorted_check[j][1], multer[count])
                        else:
                            break'''

    
    print('Stage 13: Removing Reactions')

    
    for i in reduced_mech.reactions:
        if 11 in i.reactants:
            print(i.reactants, i.prod_dict)
    cat_species_set = set(cat_species_indices)
    
    avoid_these_rxns = set()
    for i in range(len(reduced_mech.reactions)):
        for j in reduced_mech.reactions[i].reactants:
            if j in red_scc_set:
                if j not in cat_species_set:
                    scc_indy = red_scc_dict_2[j]
                    for k in reduced_mech.reactions[i].prod_dict:
                        if k in red_scc[scc_indy]:
                            avoid_these_rxns.add(i)
    
    
    
    shortened_reactions = []
    reaction_type_dict = {}
    count = 0
    for i in reduced_mech.reactions:
        if settings['Keep Cycle Reactions']==True:
            if count not in avoid_these_rxns:
                if len(i.reactants) == 2:
                    reac_str = str(i.reactants[0]) + ' ' + str(i.reactants[1])
                    if reac_str in reaction_type_dict:
                        reaction_type_dict[reac_str].append(i)
                    else:
                        reaction_type_dict[reac_str] = [i]
                elif 'HV' in i.rate_string:
                    reac_str = str(i.reactants[0]) + ' ' + 'HV'
                    if reac_str in reaction_type_dict:
                        reaction_type_dict[reac_str].append(i)
                    else:
                        reaction_type_dict[reac_str] = [i]
                else:
                    reac_str = str(i.reactants[0]) + ' ' + 'OTH'
                    if reac_str in reaction_type_dict:
                        reaction_type_dict[reac_str].append(i)
                    else:
                        reaction_type_dict[reac_str] = [i]
        else:
            if len(i.reactants) == 2:
                reac_str = str(i.reactants[0]) + ' ' + str(i.reactants[1])
                if reac_str in reaction_type_dict:
                    reaction_type_dict[reac_str].append(i)
                else:
                    reaction_type_dict[reac_str] = [i]
            elif 'HV' in i.rate_string:
                reac_str = str(i.reactants[0]) + ' ' + 'HV'
                if reac_str in reaction_type_dict:
                    reaction_type_dict[reac_str].append(i)
                else:
                    reaction_type_dict[reac_str] = [i]
            else:
                reac_str = str(i.reactants[0]) + ' ' + 'OTH'
                if reac_str in reaction_type_dict:
                    reaction_type_dict[reac_str].append(i)
                else:
                    reaction_type_dict[reac_str] = [i]
        count+=1
    
    for i in reaction_type_dict:
        total_rates = []
        multi = []
        for j in reaction_type_dict[i]:
            if j not in avoid_these_rxns:
                total_rates.append(np.mean(j.rate)*j.multiplier)
                multi.append(j.multiplier)
            
        total_rate = sum(total_rates)
        avg_rate = np.mean(total_rates)
        max_reac = total_rates.index(max(total_rates))
        base_rxn = reaction_type_dict[i][max_reac]
        base_rate = np.mean(reaction_type_dict[i][max_reac].rate)
        #print(base_rate)
        # base_rate*mult = total_rate
        mult = total_rate/max(base_rate, 1e-20)
        new_prod_dict = {}
        for j in reaction_type_dict[i]:
            factor = np.mean(j.rate)*j.multiplier/max(1e-20,total_rate)
            for k in j.prod_dict:
                if k in new_prod_dict:
                    new_prod_dict[k] += j.prod_dict[k]*factor
                else:
                    new_prod_dict[k] = j.prod_dict[k]*factor
        new_rxn = deepcopy(base_rxn)
        new_rxn.prod_dict = new_prod_dict
        new_rxn.multiplier = mult
        shortened_reactions.append(new_rxn)
        new_name_reactions =[]
    if settings['Keep Cycle Reactions']==True:
        for i in avoid_these_rxns:
            shortened_reactions.append(reduced_mech.reactions[i])

    if settings['Remove Weak Reactions']==True:
        short_rxn_reac = {}
        for i in range(len(shortened_reactions)):
            for j in shortened_reactions[i].reactants:
                if j not in back_set:
                    if j in short_rxn_reac:
                        short_rxn_reac[j].add(i)
                    else:
                        short_rxn_reac[j] = set([i])
    
        include_set = set()
        for i in short_rxn_reac:
            total_rate = 0
            for j in short_rxn_reac[i]:
                total_rate+= np.mean(shortened_reactions[j].rate)*shortened_reactions[j].multiplier
            for j in short_rxn_reac[i]:
                if np.mean(shortened_reactions[j].rate)*shortened_reactions[j].multiplier > total_rate*settings['Weak Reaction Cutoff']:
                    include_set.add(j)
        red_shortened_reactions = [shortened_reactions[i] for i in include_set]
        shortened_reactions = deepcopy(red_shortened_reactions)
    new_name_reactions = []
    if settings['Remove Reactions']== True:
        
        spec_set = set()
        for i in shortened_reactions:
        
            new_reacs = []
            for j in i.reactants:
                if species_list_names_2[j] not in background_spc_n:
                    new_reacs.append(species_list_names_2[j])
                    spec_set.add(species_list_names_2[j])
            for j in i.reactants:
                if species_list_names_2[j] in background_spc_n:
                    new_reacs.append(species_list_names_2[j])
                    spec_set.add(species_list_names_2[j])
            new_prods = {}
            for j in i.prod_dict:
                if i.prod_dict[j]>0.001:
                    new_prods[species_list_names_2[j]] = i.prod_dict[j]
                    spec_set.add(species_list_names_2[j])
            j = deepcopy(i)
            j.prod_dict = new_prods
            j.reactants = new_reacs
            if len(new_prods)>0:
                new_name_reactions.append(j)
    else:
        spec_set = set()
        for i in reduced_mech.reactions:
        
            new_reacs = []
            for j in i.reactants:
                if species_list_names_2[j] not in background_spc_n:
                    new_reacs.append(species_list_names_2[j])
                    spec_set.add(species_list_names_2[j])
            for j in i.reactants:
                if species_list_names_2[j] in background_spc_n:
                    new_reacs.append(species_list_names_2[j])
                    spec_set.add(species_list_names_2[j])
            new_prods = {}
            for j in i.prod_dict:
                if i.prod_dict[j]>0.001:
                    new_prods[species_list_names_2[j]] = i.prod_dict[j]
                    spec_set.add(species_list_names_2[j])
            j = deepcopy(i)
            j.prod_dict = new_prods
            j.reactants = new_reacs
            if len(new_prods)>0:
                new_name_reactions.append(j)
 
    reduced_mech.reactions = new_name_reactions
    print('Reduction Complete!')

    return reduced_mech, categories, test_mechanism, remaining_specs, avg_yield, yields, graphs, red_yields, red_graphs, new_graph, scc, red_scc, groups, species_list, dic, tiers, dag_graph_save, dag_graph_save_2, inv_map_dict, mult_list_list, cat_data, multy, ordered_species, avg_yield_o, avg_yield_o2
    
    #m_para, m_para_map, reduced_mech, scc_ref_data[1], data_map, cycle_info, settings_des, conditions, red_specs, back_set, len(new_species), rem_sccs_2
    # optimize rate coefficients and non-unit internal rate coefficients
    # in terms of outgoing amounts, internal counts, 
    
    
    
                

In [6]:
def AMORE_mechanism_reduction(mechanism, background_spc_n, conditions, settings):
    # Version 2.0
    default_settings = {}
    default_settings['No Group'] = []
    default_settings['Manual Groups'] = []
    default_settings['Remove Reactions'] = True
    default_settings['Remove Weak Reactions'] = False
    default_settings['Weak Reaction Cutoff'] = 0
    default_settings['Keep Cycle Reactions'] = True
    default_settings['Print Progress'] = True
    default_settings['Iterations'] = 0
    for i in default_settings:
        if i not in settings:
            settings[i] = default_settings[i]
    
    if settings['Print Progress'] == True:
        print('Stage 1: Mechanism Preprocessing')
    
    needed_concentrations = set() # any species which is the second reactant in a reaction but the concentration is not provided

    # We are duplicating the mechanism and calling it test mechanism
    # This is to avoid inadvertently modifying the original mechanism variable
    test_reactions = []
    not_rxns = set(settings['Background Rxns'])
    for i in settings['Aerosol Rxns']:
        not_rxns.add(i)
    for i in range(len(mechanism.reactions)):
        if i not in not_rxns:
            test_reactions.append(deepcopy(mechanism.reactions[i]))      
    test_mechanism = Mechanism(deepcopy(mechanism.species),test_reactions)

    # number of reactions, number of species, species name to index dictionary, species list as indices
    reac_len = len(test_mechanism.reactions)
    spec_len = len(test_mechanism.species)
    dic = {test_mechanism.species[i]:i for i in range(spec_len)}
    species_list = [i for i in range(spec_len)]

    # species that cannot be removed from the mechanism (manual input)
    protected = set(settings['Protected']).union(background_spc_n)
    protected = [dic[i] for i in protected]

    # species that cannot be placed in a group
    no_group = [dic[i] for i in settings['No Group']]

    # species that have been manually added to a group
    manual_groups = []
    man_group_set = set()
    for i in settings['Manual Groups']:
        g = []
        for j in i:
            g.append(dic[j])
            man_group_set.add(dic[j])
        manual_groups.append(g)
    #print('These are the manual groups: ', manual_groups)
    # root species of the mechanism
    roots = [dic[i] for i in settings['roots']]

    # number of conditions
    l_c = len(conditions)

    # set of background species
    back_set = set()
    for i in background_spc_n:
        back_set.add(mechanism.species.index(i))

    # list of reactants, list of products, list of product coefficients, and product dictionary for each reaction
    # eg: rxn no. 7 is A + X --> 0.5 B + 0.2 C and X is a background species
    # eg cont: A has index 0, B has index 1, c has index 2, and X has index 3
    # eg cont:  reac_list_n[7] = [A,X]     prod_list_n[7] = [B,C]      prod_coeff_list[7] = [0.5, 0.2]     prod_dict[7] = {B:0.5, C:0.2}
    # eg cont:    reac_list[7] = [0,3]       prod_list[7] = [1, 2]     
    # eg cont: reac_no_back[7] = [0]      prod_no_back[7] = [1,2]
    # eg cont: rxn_reac[0] = {7...}, rxn_reac[3] = {7...}, rxn_prod[1] = {7...}, rxn_prod[2] = {7...}
    reac_list_n = []
    prod_list_n = []
    prod_coeff_list = []
    prod_dict = [{} for i in range(reac_len)]
    for i in range(reac_len):
        test_mechanism.reactions[i].rate = []
        reac_list_n.append(test_mechanism.reactions[i].reactants)
        prod_list_n.append(list(test_mechanism.reactions[i].prod_dict))
        for k in test_mechanism.reactions[i].prod_dict:
            prod_dict[i][dic[k]] = test_mechanism.reactions[i].prod_dict[k]
    reac_list, prod_list, reac_no_back, prod_no_back, rxn_reac, rxn_prod = rxn_index_convert(reac_list_n,prod_list_n,background_spc_n, reac_len, spec_len, dic)
    rxn_prod = [set(i) for i in rxn_prod]
    rxn_reac = [set(i) for i in rxn_reac]

    
    c_count = 0
    for i in conditions: 
        M = pressure_to_m(i['pressure'],i['temp'])
        TEMP = i['temp']
        SUN = i['sun']
        CFACTOR = 2.5e+19
        p_fac = M/1000000000
        double_rxns = []
        count = 0
        for j in test_mechanism.reactions:
            if j.rate_law!='null':
                if 'J' in j.rate_law:
                    j.eval_rate_law = solve_j_rate(j.rate_law, i['sza'],i['sun'])
                else:
                    j.eval_rate_law = eval(j.rate_law)
                if len(j.reactants)==1:
                    j.rate.append(j.eval_rate_law)

                elif j.reactants[1] in background_spc_n and j.reactants[1] not in settings['roots']:
                    j.rate.append(j.eval_rate_law*i[j.reactants[1]]*p_fac)

                elif j.reactants[0] in background_spc_n and j.reactants[0] not in settings['roots']:
                    j.rate.append(j.eval_rate_law*i[j.reactants[0]]*p_fac)

                elif j.reactants[1] in i and j.reactants[1] not in settings['roots']:
                    j.rate.append(j.eval_rate_law*i[j.reactants[1]]*p_fac)

                elif j.reactants[0] in i and j.reactants[0] not in settings['roots']:
                    j.rate.append(j.eval_rate_law*i[j.reactants[0]]*p_fac)

                else:
                    needed_concentrations.add(j.reactants[1])
                    root_conc = 0
                    for r in settings['roots']:
                        root_conc = i[r] + root_conc
                    j.rate.append(j.eval_rate_law*0.005*root_conc*p_fac)
                    double_rxns.append(count)
                count = count + 1
            else:
                counted_reactants = 0
                for s in j.reactants:
                    if s not in no_counts:
                        counted_reactants+=1
                if counted_reactants==1:
                    j.rate.append(j.eval_rate_law[c_count])
                elif j.reactants[1] in background_spc_n and j.reactants[1] not in settings['roots']:
                    j.rate.append(j.eval_rate_law[c_count]*i[j.reactants[1]]*p_fac)
                elif j.reactants[0] in background_spc_n and j.reactants[0] not in settings['roots']:
                    j.rate.append(j.eval_rate_law[c_count]*i[j.reactants[0]]*p_fac)
                elif j.reactants[1] in i and j.reactants[1] not in settings['roots']:
                    j.rate.append(j.eval_rate_law[c_count]*i[j.reactants[1]]*p_fac)
                elif j.reactants[0] in i and j.reactants[0] not in settings['roots']:
                    j.rate.append(j.eval_rate_law[c_count]*i[j.reactants[0]]*p_fac)
                else:
                    needed_concentrations.add(j.reactants[1])
                    root_conc = 0
                    for r in settings['roots']:
                        root_conc = i[r] + root_conc
                    j.rate.append(j.eval_rate_law[c_count]*0.05*root_conc*p_fac)

                    double_rxns.append(count)
        c_count+=1
    
    if len(needed_concentrations)>0:
        print('For better results, add concentrations for these species: ', needed_concentrations)
    
    if settings['Print Progress'] == True:
        print('Stage 2: Representing Mechanism as Graph')
    graphs = [[] for c in range(l_c)]
    in_graph = [set() for i in range(spec_len)]
    out_graph = [set() for i in range(spec_len)]
    out_graph_type = [{} for i in range(spec_len)]
    in_graph_type  = [{} for i in range(spec_len)]
    
    
    
    rxn_types = {}
    species_types = {i:set() for i in range(spec_len)}
    for i in range(spec_len):
        if i not in back_set:
            edges = [{} for c in range(l_c)]
            rate_sums = [0 for c in range(l_c)]
            
            for j in rxn_reac[i]:
                for c in range(l_c):
                    rate_sums[c] = rate_sums[c] + test_mechanism.reactions[j].rate[c]
            for j in rxn_reac[i]:
                mults = [0 for c in range(l_c)]
                for c in range(l_c):
                    mults[c] = test_mechanism.reactions[j].rate[c]/max(1e-20,rate_sums[c])
                if len(test_mechanism.reactions[j].reactants) == 1:
                    type_r = 'solo'
                else:
                    mark = True
                    for p in reac_list[j]:
                        if p in background_spc:
                            type_r = p
                            mark = False
                    if mark:
                        type_r = 'double'
                rxn_types[j] = type_r
                species_types[i].add(str(type_r))
                for k in prod_dict[j]:
                    if i in in_graph_type[k]:
                        in_graph_type[k][i].add(type_r)
                    else:
                        in_graph_type[k][i] = set([type_r])
                    for c in range(l_c):
                        if k in edges[c]:
                            if c==0:
                                edges[c][k] = edges[c][k] + prod_dict[j][k]*mults[c]
                                out_graph[i].add(k)
                                in_graph[k].add(i)
                                out_graph_type[i][k].add(type_r)
                            else:
                                edges[c][k] = edges[c][k] + prod_dict[j][k]*mults[c]

                        
                        else:
                            if c==0:
                                edges[c][k] = prod_dict[j][k]*mults[c]
                                out_graph[i].add(k)
                                in_graph[k].add(i)
                                out_graph_type[i][k] = set([type_r])
                            else:
                                edges[c][k] = prod_dict[j][k]*mults[c]
            for c in range(l_c):
                graphs[c].append(edges[c])
        else:
            for c in range(l_c):
                graphs[c].append({})
    for i in range(len(species_types)):
        species_types[i] = sorted(list(species_types[i]))
    avg_graph = []
    for i in range(spec_len):
        avg = {}
        for c in range(l_c):
            for j in graphs[c][i]:
                if j in avg:
                    avg[j]+=graphs[c][i][j]
                else:
                    avg[j] = graphs[c][i][j]
        for x in avg:
            avg[x]*=1/l_c
        avg_graph.append(avg)
    
    graph_mat_lil = lil_matrix((spec_len,spec_len), dtype=None)

    edges = []
    for i in range(len(out_graph)):
        for j in out_graph[i]:
            edges.append((i,j))

    count = 0
    for i,j in edges:
        graph_mat_lil[i,j] = 1

    graph_mat = graph_mat_lil.tocsr(copy=False)

    scc_result = connected_components(graph_mat, directed=True, connection='strong', return_labels=True)

    scc_dic = {}
    for i in range(len(scc_result[1])):
        if scc_result[1][i] in scc_dic:
            scc_dic[scc_result[1][i]].append(i)
        else:
            scc_dic[scc_result[1][i]] = [i]
    
    scc = []
    for i in scc_dic:
        if len(scc_dic[i])>1:
            scc.append(scc_dic[i])

    distance,preds,root_specs = dijkstra(graph_mat, directed=True, indices=roots, return_predecessors=True, unweighted=True, limit=np.inf, min_only=True)    
    #return distance, preds, root_specs
    # measure scc's

    scc_lens = [len(i) for i in scc]
    # seeing how many species can be joined by the common parent rule
    
    #identifying species with same parent, same incoming reaction types and same outgoing reaction types
    
    
    paths = []
    path_roots = []
    for i in range(spec_len):
        if distance[i]<100:
            if preds[i]<0:
                path = []
            else:
                marker = preds[i]
                path = [marker]
                counter = 0
                while marker not in roots and counter<100:
                    marker = preds[marker]
                    path.append(marker)
            paths.append(path)
            path_roots.append(root_specs[i])
        else:
            paths.append([])
            path_roots.append('none')
    
    paths_with_types = []
    for i in range(len(paths)):
        path = []
        count = 0
        for j in paths[i]:
            if count == 0:
                path.append(in_graph_type[i][j])
            else:
                path.append(in_graph_type[paths[i][count-1]][j])
            count = count + 1
        path.insert(0,path_roots[i])
        species_type_string = ''
        for j in species_types[i]:
            species_type_string+=j+' '
        
        #path.append(species_type_string)
        paths_with_types.append(path)

    path_type_strings = []
    for i in paths_with_types:
        string = str(i[0]) + '+'
        for j in i[1:]:
            for k in j:
                string = string + str(k) + ','
            string = string+'+'
        path_type_strings.append(string)

    path_sim_dict = {}
    for i in range(len(path_type_strings)):
        if i not in back_set:
            if path_type_strings[i] in path_sim_dict:
                path_sim_dict[path_type_strings[i]].append(i)
            else:
                path_sim_dict[path_type_strings[i]]= [i]
    groups = []  
    for i in path_sim_dict:
        if len(path_sim_dict[i])>1:
            groups.append(path_sim_dict[i])
    
    del paths_with_types
    
    
    in_cycle_specs = []
    scc_out_specs = []
    for i in scc:
        count = count + 1
        in_cycle_spec = set()
        out_spec = set()
        for p in i:
            for k in in_graph[p]:
                if k not in i:
                    in_cycle_spec.add(p)
            for k in out_graph[p]:
                if k not in i:
                    out_spec.add(k)
        in_cycle_specs.append(in_cycle_spec)
        scc_out_specs.append(out_spec)
        
    

    scc_set = set()
    scc_dict_2 = {}
    for i in range(len(scc)):
        for j in scc[i]:
            scc_dict_2[j] = i
            scc_set.add(j)

    
    if settings['Print Progress'] == True:
        print('Stage 3: Measuring Mechanism Yields')

    county = 0
    yields = []
    yields_for_order = []
    yields_for_order_2 = []
    cycle_out_graphs = [{} for c in range(l_c)]
    cycle_dag_graphs = [{} for c in range(l_c)]
    cycle_dag_graphs_2 = [{} for c in range(l_c)]
    new_graphs = []
    new_ins = []
    new_outs = []
    for c in range(l_c):
        county +=1        
        new_graph = []
        for i in graphs[c]:
            dicy = {}
            for j in i:
                dicy[j] = i[j]
            new_graph.append(dicy)
        new_graph_2 = deepcopy(new_graph)
        new_graph_3 = deepcopy(new_graph)

        new_in = []
        for i in in_graph:
            sety = set()
            for j in i:
                sety.add(j)
            new_in.append(sety)
        

        new_out = []
        for i in out_graph:
            sety = set()
            for j in i:
                sety.add(j)
            new_out.append(sety)
        
        
        for i in range(len(scc)):
            counte = 0
            
            for x in in_cycle_specs[i]:
                counte +=1
                new_out[x] = scc_out_specs[i].union(set(scc[i]))
                leny = int(np.sqrt(len(scc[i])))
                data, in_cyc_data, out_data = cycle_simulator_3_out(scc[i], x, graphs[c],out_graph,in_graph, [leny+40,2*(leny+40)],1e-6,set(scc[i]).union(scc_out_specs[i]),scc_out_specs[i])
                cycle_out_graphs[c][x] = data
                ful_dat = {}
                for p in data:
                    ful_dat[p] = data[p]
                ful_dat_2 = deepcopy(ful_dat)
                for p in in_cyc_data:
                    ful_dat[p] = in_cyc_data[p]
                for p in out_data:
                    ful_dat_2[p] = out_data[p]
                new_graph[x]= data
                new_graph_2[x]= ful_dat
                new_graph_3[x] = deepcopy(ful_dat_2)
                cycle_dag_graphs[c][x] = ful_dat
                cycle_dag_graphs_2[c][x] = ful_dat_2

                

            for y in scc[i]:
                if y not in in_cycle_specs[i]:

                    for s in out_graph[y]:
                        if y in new_in[s]:
                            new_in[s].remove(y)
                    new_in[y] = set(in_cycle_specs[i])
                    new_out[y] = set()
                    new_graph[y] = set()
                    new_graph_2[y]= set()
        new_graphs.append(new_graph)
        new_ins.append(new_in)
        new_outs.append(new_out)
        yi = get_yields(roots, 1e-8, new_graph, new_in, new_out, back_set,scc,scc_dict_2, scc_set, spec_len)
        yi_2 = get_yields(roots, 1e-8, new_graph_2, new_in, new_out, back_set,scc,scc_dict_2, scc_set, spec_len)
        yi_3 = get_yields(roots, 1e-8, new_graph_3, new_in, new_out, back_set,scc,scc_dict_2, scc_set, spec_len)

        yields.append(yi)
        yields_for_order.append(yi_2)
        yields_for_order_2.append(yi_3)
    avg_cycle_out_graph = {}
    for c in range(l_c):
        for x in cycle_out_graphs[c]:
            
            if x in avg_cycle_out_graph:
                for j in cycle_out_graphs[c][x]:
                    if j in avg_cycle_out_graph[x]:
                        avg_cycle_out_graph[x][j]+=cycle_out_graphs[c][x][j]
                    else:
                        avg_cycle_out_graph[x][j] = cycle_out_graphs[c][x][j]
            else:
                avg_cycle_out_graph[x] = {}
                for j in cycle_out_graphs[c][x]:
                    if j in avg_cycle_out_graph[x]:
                        avg_cycle_out_graph[x][j]+=cycle_out_graphs[c][x][j]
                    else:
                        avg_cycle_out_graph[x][j] = cycle_out_graphs[c][x][j]
    
    for i in avg_cycle_out_graph:
        for j in avg_cycle_out_graph[i]:
            avg_cycle_out_graph[i][j]*=1/l_c




    avg_cycle_dag_graph = {}
    for c in range(l_c):
        for x in cycle_dag_graphs[c]:
            
            if x in avg_cycle_dag_graph:
                for j in cycle_dag_graphs[c][x]:
                    if j in avg_cycle_dag_graph[x]:
                        avg_cycle_dag_graph[x][j]+=cycle_dag_graphs[c][x][j]
                    else:
                        avg_cycle_dag_graph[x][j] = cycle_dag_graphs[c][x][j]
            else:
                avg_cycle_dag_graph[x] = {}
                for j in cycle_dag_graphs[c][x]:
                    if j in avg_cycle_dag_graph[x]:
                        avg_cycle_dag_graph[x][j]+=cycle_dag_graphs[c][x][j]
                    else:
                        avg_cycle_dag_graph[x][j] = cycle_dag_graphs[c][x][j]
    
    for i in avg_cycle_dag_graph:
        for j in avg_cycle_dag_graph[i]:
            avg_cycle_dag_graph[i][j]*=1/l_c



    

    avg_yield_o = {}
    for i in yields_for_order[0]:
        specyi = []
        for c in range(l_c):
            specyi.append(yields_for_order[c][i])
        avg_yield_o[i] = np.mean(specyi)


    avg_yield_o2 = {}
    for i in yields_for_order_2[0]:
        specyi = []
        for c in range(l_c):
            specyi.append(yields_for_order_2[c][i])
        avg_yield_o2[i] = np.mean(specyi)
        
    boost_factor = 1.5
    ordered_species = []
    ordered_yields = []
    yield_values = []
    for i in avg_yield_o:
        yield_values.append(avg_yield_o[i])
    max_yield = max(yield_values)
    for i in avg_yield_o:
        if i in protected:
            ordered_yields.append(max_yield)
        else:
            ordered_yields.append(avg_yield_o[i])
        ordered_species.append(i)


    
    avg_yield = {}
    for i in yields[0]:
        specyi = []
        for c in range(l_c):
            specyi.append(yields[c][i])
        avg_yield[i] = np.mean(specyi)
        
    
        
    ordered_species = [x for _, x in sorted(zip(ordered_yields, ordered_species))]
    ordered_yields.sort()
    spec_order_dict = {}
    count = 0
    for i in ordered_species:
        spec_order_dict[i] = count
        count+=1
    for i in groups:
        min_dict = {}
        for j in i:
            min_dict[spec_order_dict[j]] = j
        min_spec = min_dict[min(min_dict)]
        ordered_yields[spec_order_dict[min_spec]] *= boost_factor
    
    ordered_species = [x for _, x in sorted(zip(ordered_yields, ordered_species))]
    for i in protected:
        ordered_species.remove(i)
        ordered_species.append(i)
    ordered_yields.sort()

    for i in settings['Remove Species']:
        ordered_species.remove(dic[i])
        ordered_species.insert(0, dic[i])
    
    reduced_graph, reduced_in, reduced_out = copy_graph(avg_graph, in_graph, out_graph)
    reduced_reactions = []
    for i in test_mechanism.reactions:
        reduced_reactions.append(deepcopy(i))
    
    reduced_mech = Mechanism(deepcopy(test_mechanism.species),reduced_reactions)

    for i in range(reac_len):
        reduced_mech.reactions[i].prod_dict = prod_dict[i]
        reduced_mech.reactions[i].reactants = reac_list[i]
    
    avg_yield_full = avg_yield
    not_removed = set()
    remaining_specs = ordered_species[spec_len-red_mech_size:]
    
    remove_species = []
    for i in ordered_species[:spec_len-red_mech_size]:
        if i not in protected:
            remove_species.append(i)
        else:
            remaining_specs.append(i)

    if settings['Print Progress'] == True:
        print('Stage 4: Combining Grouped Species')

    new_groups = []
    man_group_set = set()
    for i in manual_groups:
        for j in i:
            man_group_set.add(j)
    for i in groups:
        new_group = []
        for j in i:
            if j not in man_group_set:
                new_group.append(j)
        if len(new_group)>1:
            new_groups.append(new_group)
    groups= deepcopy(new_groups)
    new_groups = []
    for i in groups:
        if not all([x in remaining_specs for x in i]):
            new_groups.append(i)
    groups = deepcopy(new_groups)
    

    remaining_scc = set()
    for i in remaining_specs:
        if i in scc_set:
            remaining_scc.add(scc_dict_2[i])

    removed_scc = set(range(len(scc))).difference(remaining_scc)
    scc_remove_species = set()

    for i in removed_scc:
        for j in scc[i]:
            scc_remove_species.add(j)
    
    new_groups = []
    for i in range(len(groups)):
        no_scc_group = []
        scc_groups = {}
        none_left = []
        for j in groups[i]:
            if j not in no_group:
                if j in scc_set:
                    if scc_dict_2[j] in scc_groups:
                        scc_groups[scc_dict_2[j]].append(j)
                    else:
                        scc_groups[scc_dict_2[j]] = [j]
                else:
                    no_scc_group.append(j)
        for g in scc_groups:
            scc_ind = scc_dict_2[scc_groups[g][0]]
 
            scc_specs = []
            for y in scc[scc_ind]:
                if y not in scc_groups[g]:
                    scc_specs.append(y)

            counted = 0

            if not any([x in remaining_specs for x in scc_specs]):

                for h in scc_groups:
                    if g!=h:
                        if any([x in remaining_specs for x in scc_groups[h]]) and counted==0:
                            counted+=1
                            
                            for f in scc_groups[g]:
                                scc_groups[h].append(f)
                                if f in remaining_specs:
                                    
                                    remaining_specs.remove(f)
                                    remove_species.append(f)
                if counted==0:
                    for f in scc_groups[g]:
                        none_left.append(f)
                scc_groups[g] = []
                        
        if len(no_scc_group)>1:
            new_groups.append(no_scc_group)
        for k in scc_groups:
            if len(scc_groups[k])>1:
                new_groups.append(scc_groups[k])
        if len(none_left)>1:
            new_groups.append(none_left)
    groups = deepcopy(new_groups)   
    
    for m in manual_groups:
        if any([x in remaining_specs for x in m]):
            if any([x in remove_species for x in m]):
                groups.append(m)     
    group_set = set()
    grouped = set()
    group_dict = {}
    rem_groups = []
    count = 0
    for i in groups:
        if any([j in remaining_specs for j in i]) and any([k in remove_species for k in i]):
            rem_groups.append(i)
            for j in i:
                group_set.add(j)
                group_dict[j] = count
            count+=1
    groups = deepcopy(rem_groups)
    group_mapping = []
    
    
    group_dict_check = {}

    for j in range(len(groups)):
        mapping = [0]
        group = groups[j]
        for i in ordered_species[spec_len-red_mech_size:]:
            if i in group:
                group_lead = i
                mapping[0] = group_lead
        out_reactions = {}
        removers = set()

        new_group = {group_lead}
        for i in group:
            if i not in remove_species:
                pass

            else:
                new_group.add(i)
        group = deepcopy(new_group) 
        for sp in new_group:
            group_dict_check[sp] = new_group
        total_yield = 0 
        grouped_map = []
        for i in group:
            grouped.add(i)
            grouped_map.append(i)
            total_yield += avg_yield_o2[i]
        mapping.append(grouped_map)
        group_mapping.append(mapping)

        
        
        new_graph_gl = {}
        within_spec_ratio = {}
        
        #mult * = base * spec_rat * rate_type[x]/rate_sum  /rate 
        for i in group:
            for r in rxn_reac[i]:
                if any([x in group for x in reduced_mech.reactions[r].prod_dict]):
                    new_prod_dict = {}
                    
                    for p in reduced_mech.reactions[r].prod_dict:
                        if p in group:
                            if p in reduced_graph[i]:
                                del reduced_graph[i][p]
                        else:
                            new_prod_dict[p] = reduced_mech.reactions[r].prod_dict[p]
                        
                    reduced_mech.reactions[r].prod_dict = deepcopy(new_prod_dict)                      

        
        base_spec = avg_yield_o2[group_lead]/max(1e-20,total_yield)
        base_rate_type = {}
        for g in groups[j]:
            for x in out_graph_type[g]:
                for y in out_graph_type[g][x]:
                    base_rate_type[y] = 0
        

        
        base_rate = 0
        for r in rxn_reac[group_lead]:
            if rxn_types[r] in base_rate_type:
                base_rate_type[rxn_types[r]]+= np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier
            else:
                pass
                #print('no match', groups[j], reduced_mech.reactions[r].reactants)
            base_rate+=np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier
            
        for r in rxn_reac[group_lead]:
            
            mult = base_spec

            reduced_mech.reactions[r].multiplier *= mult
        for i in group:
            if i != group_lead:
                spec_rat = avg_yield_o[i]/max(1e-20,total_yield)
                rate_type = {}
                for x in out_graph_type[i]:
                    for y in out_graph_type[i][x]:
                        rate_type[y] = 0
                rate_sum = 0
                for r in rxn_reac[i]:
                    rate_type[rxn_types[r]]+= np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier
                    rate_sum+=np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier
                for r in rxn_reac[i]:
                    if rate_sum>0:
                        mult = spec_rat*base_rate/rate_sum
                        
                    else:
                        mult = 0
                    new_reactants = []
                    for x in reduced_mech.reactions[r].reactants:
                        if x in group:
                            new_reactants.append(group_lead)
                        else:
                            new_reactants.append(x)
                    reduced_mech.reactions[r].reactants = new_reactants
                    reduced_mech.reactions[r].multiplier*=mult
                    rxn_reac[group_lead].add(r)

                    for p in reduced_mech.reactions[r].prod_dict:
                        
                        reduced_out[group_lead].add(p)
                        reduced_in[p].add(group_lead)
                for r in rxn_prod[i]:
                    if i in reduced_mech.reactions[r].prod_dict:
                        if group_lead in reduced_mech.reactions[r].prod_dict:
                            reduced_mech.reactions[r].prod_dict[group_lead] += reduced_mech.reactions[r].prod_dict[i]
                        else:
                            reduced_mech.reactions[r].prod_dict[group_lead] = reduced_mech.reactions[r].prod_dict[i]
                        del reduced_mech.reactions[r].prod_dict[i]
                        rxn_prod[group_lead].add(r)
                        for re in reduced_mech.reactions[r].reactants:
                            if re not in background_spc:
                                if i in reduced_graph[re]:
                                    if group_lead in reduced_graph[re]:
                                        reduced_graph[re][group_lead]+= reduced_graph[re][i]
                                    else:
                                        reduced_graph[re][group_lead] = reduced_graph[re][i]
                for x in reduced_out[i]:
                    if i in reduced_in[x]:
                        reduced_in[x].remove(i)
                for x in reduced_in[i]:
                    if i in reduced_out[x]:
                        reduced_out[x].remove(i)
                    if i in reduced_graph[x]:
                        del reduced_graph[x][i]
              
        total_rate = 0
        new_in_gl = set()
        for r in rxn_prod[group_lead]:
            for j in reduced_mech.reactions[r].reactants:
                if j not in background_spc:
                    new_in_gl.add(j)
                    
        for r in rxn_reac[group_lead]:
            total_rate+= np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier
        new_out_gl = set()
        for r in rxn_reac[group_lead]:
            for p in reduced_mech.reactions[r].prod_dict:
                new_out_gl.add(p)
                if p in new_graph_gl:
                    new_graph_gl[p]+= reduced_mech.reactions[r].prod_dict[p]*np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier/max(1e-20,total_rate)
                else:
                    new_graph_gl[p] = reduced_mech.reactions[r].prod_dict[p]*np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier/max(1e-20,total_rate)
        reduced_graph[group_lead] = new_graph_gl
        reduced_out[group_lead] = new_out_gl
        reduced_in[group_lead] = new_in_gl
    named_rxns = []
    for i in reduced_mech.reactions:
        new_rxn = deepcopy(i)
        new_reactants = []
        for j in i.reactants:
            new_reactants.append(species_list_names_2[j])
        new_prods = {}
        for j in i.prod_dict:
            new_prods[species_list_names_2[j]]= i.prod_dict[j]
        new_rxn.prod_dict = new_prods
        new_rxn.reactants = new_reactants
        named_rxns.append(new_rxn)
    reduced_mech_test = Mechanism(species_list_names_2,named_rxns)

    if settings['Print Progress'] == True:
        print('Stage 5: Converting Graph to Directed Acyclic Graph')

    removed_rxns = set()
    dont_remove = set()
    for i in remaining_specs:
        dont_remove.add(i)
    for i in grouped:
        dont_remove.add(i)
    scc_rem_removed = set()
    for i in remaining_scc:
        just_removed = set()
        for j in scc[i]:
            if j not in dont_remove:
                for r in rxn_reac[j]:
                    removed_rxns.add(r)
                for ins in reduced_in[j]:
                    
                    if ins in reduced_graph[j]:
                        scale_factor = abs(1/max(1e-20,1-reduced_graph[j][ins]))
                    else:
                        scale_factor = 1
                    
                    if j in reduced_graph[ins]: 
                        mark = False
                        if ins in group_dict_check:
                            if any([t in scc[i] for t in group_dict_check[ins]]):
                                mark = True

                        if ins in scc[i] or mark==True:
                            for sp in reduced_graph[j]:
                                if sp!= ins:
                                    if sp in reduced_graph[ins]:
                                        
                                        reduced_graph[ins][sp]+= reduced_graph[j][sp]*reduced_graph[ins][j]*scale_factor
                                    else:
                                        reduced_graph[ins][sp] = reduced_graph[j][sp]*reduced_graph[ins][j]*scale_factor
                                    reduced_out[ins].add(sp)
                                    reduced_in[sp].add(ins)
                        else:
                            for sp in avg_cycle_out_graph[j]:
 
                                if sp in reduced_graph[ins]:
                                    reduced_graph[ins][sp]+= avg_cycle_out_graph[j][sp]*reduced_graph[ins][j]
                                else:

                                    reduced_graph[ins][sp] = avg_cycle_out_graph[j][sp]*reduced_graph[ins][j]
                                reduced_out[ins].add(sp)
                                reduced_in[sp].add(ins)
            
                            
                                
                    reduced_out[ins].remove(j)
                    del reduced_graph[ins][j]
                for r in rxn_prod[j]:
                    if j in reduced_mech.reactions[r].prod_dict:
                        
                        mark = False
                        for spe in reduced_mech.reactions[r].reactants:
                            if spe in group_dict_check:
                                if any([t in scc[i] for t in group_dict_check[spe]]):
                                    mark = True
                        mark = True
                        if any([f in scc[i] for f in reduced_mech.reactions[r].reactants]) or mark==True:
                            scale_factor = 1
                            for rec in reduced_mech.reactions[r].reactants:
                                if rec in scc[i]:
                                    if rec in reduced_graph[j]:
                                        scale_factor = abs(1/max(1e-20,1-reduced_graph[j][rec]))
                            for pro in reduced_graph[j]:
                                if pro not in reduced_mech.reactions[r].reactants:
                                    if pro in reduced_mech.reactions[r].prod_dict:
                                        reduced_mech.reactions[r].prod_dict[pro] += reduced_graph[j][pro]*reduced_mech.reactions[r].prod_dict[j]*scale_factor
                                    else:
                                        reduced_mech.reactions[r].prod_dict[pro] = reduced_graph[j][pro]*reduced_mech.reactions[r].prod_dict[j]*scale_factor
                                    
                                    rxn_prod[pro].add(r)
                            
                        else:
                            for pro in avg_cycle_out_graph[j]:
                                if pro not in reduced_mech.reactions[r].reactants:
                                    if pro in reduced_mech.reactions[r].prod_dict:
                                        reduced_mech.reactions[r].prod_dict[pro] += avg_cycle_out_graph[j][pro]*reduced_mech.reactions[r].prod_dict[j]
                                    else:
                                        reduced_mech.reactions[r].prod_dict[pro] = avg_cycle_out_graph[j][pro]*reduced_mech.reactions[r].prod_dict[j]
                                    rxn_prod[pro].add(r)
                        del reduced_mech.reactions[r].prod_dict[j]
                scc_rem_removed.add(j)  

    
    
    marked = set(background_spc).union(remaining_specs)
    dag_graph = [{} for i in range(spec_len)]
    dag_out = [set() for i in range(spec_len)]
    dag_in = [set() for i in range(spec_len)]
    count = 0
    
    for i in in_cycle_specs:
        for j in i:
            dicy = {}
            for k in avg_cycle_out_graph[j]:
                if avg_cycle_out_graph[j][k]!=0:
                    dicy[k]=avg_cycle_out_graph[j][k]
                    dag_out[j].add(k)
                    dag_in[k].add(j)
            dag_graph[j] = dicy
    for i in range(spec_len):
        count+=1
        if i not in scc_set:
            dicy = {}
            for j in reduced_graph[i]:
                if reduced_graph[i][j]!=0:
                    dicy[j]=reduced_graph[i][j]
                    dag_out[i].add(j)
                    dag_in[j].add(i)
            dag_graph[i] = dicy

    remove_these_rxns = set()
    avg_cycle_out_graph_2 = {}
    for i in avg_cycle_out_graph:
        dicy = {}
        for j in avg_cycle_out_graph[i]:
            if avg_cycle_out_graph[i][j]!=0:
                dicy[j]= avg_cycle_out_graph[i][j]
        avg_cycle_out_graph_2[i] = dicy

    for i in range(len(reduced_graph)):
        dicy = {}
        for j in reduced_graph[i]:
            if reduced_graph[i][j]!=0:
                dicy[j]= reduced_graph[i][j]
        reduced_graph[i]=dicy

    if settings['Print Progress'] == True:
        print('Stage 6: Creating Category Species')

    
    new_dict = deepcopy(dic)
    
    categories = []
    cat_names = []
    new_species = deepcopy(mechanism.species)
    cat_species_indices = []
    for i in settings['Categories']:
        caty = []
        for j in i[1]:
            caty.append(dic[j])
        categories.append(caty)
        cat_names.append(i[0])
        new_species.append(i[0])
        new_dict[i[0]] = len(new_species)-1
        cat_species_indices.append(len(new_species)-1)
    cat_dict = {}
    cat_set = set()
    count = 0
    for i in categories:
        for j in i:
            cat_dict[j] = count
            cat_set.add(j)
        count+=1
    cat_specs = []
    cat_mapping = []
    
    #category rate multiplier: do the exact same method as the groups 
    #Basically, take the rate of reaction, multiply it by the % contribution of the species
    # and the % contribution of the rxn type compared to a base rate which is selected as the highest
    # contributor to the category
    # if a product in a reaction is in the category, what ever the moles to in-cat is the fraction
    # within cat, and the rate should be multiplied by 1 - frac, and all products should be multiplied 
    # by 1/(1-frac) except for the within cat which is removed
    all_cat_set = set()
    for i in categories:
        for j in i:
            all_cat_set.add(j)
    multses = []
    mult_list_list = []
    cat_data = []
    multy = []

    for j in range(len(categories)):
        mapping = []
        spec_len = spec_len+1
        dag_out.append(set())
        dag_in.append(set())
        dag_graph.append({})
        new_spec = cat_species_indices[j]
        mapping.append(new_spec)
        cat_specs.append(new_spec)
        all_cat_set.add(new_spec)
        caty = categories[j]
        reduced_mech.species.append(cat_names[j])
        removers = set()
        rxy_prod = set()
        rxy_reac = set()
        net_yield = 0
        net_out_yield = 0
        out_reactions = {}
        multies = []
        data_c = []
        for i in caty:
            
            if i in remaining_specs:
                removers.add(i)
                
            if i in grouped:
                removers.add(i)

        for i in removers:
            caty.remove(i)
        cat_map = []
        caty_yields = {}
        caty_yields_all = {}
        caty_yields_cyc = {}
        caty_yields_cyc_all = {}
        max_yi = 0
        if len(caty)==0:
            break
        for i in caty:
            spec_yi = 0
            spec_yi_c = 0
            for x in dag_in[i]:
                if x not in caty:
                    if i in dag_graph[x]:
                        
                        spec_yi += dag_graph[x][i]*avg_yield_o[x]
                        if x in avg_yield_o:
                            spec_yi_c += dag_graph[x][i]*avg_yield_o[x]
                        
            caty_yields[i] = spec_yi
            caty_yields_cyc[i] = spec_yi_c
            caty_yields_all[i] = avg_yield_o[i]
            caty_yields_cyc_all[i] = avg_yield_o[i]
            
            if spec_yi>max_yi:
                max_yi = deepcopy(spec_yi)
                cat_lead = i
        cat_full_yield = sum(caty_yields.values())
        if settings['Print Progress'] == True:
            print('Category Yield ' + str(j) + ': ', cat_full_yield)#, cat_full_yield_all, cat_full_yield_cyc, cat_full_yield_cyc_all)
        cat_rel_rate = 0
        avg_yield_full[new_spec] = cat_full_yield
        avg_yield_o[new_spec] = cat_full_yield
        cat_rel_out = 0
        cat_weight_list = []
        cat_num_list = []
        for i in caty:
            rates_c = []
            for r in rxn_reac[i]:
                delta = 1
                for x in reduced_mech.reactions[r].prod_dict:
                    if x in caty:
                        delta = delta - reduced_mech.reactions[r].prod_dict[x]
                cat_rel_rate+=np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier*caty_yields[i]/cat_full_yield
                cat_rel_out+= np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier*max(0,delta)*caty_yields[i]/cat_full_yield
                cat_weight_list.append(caty_yields[i]/max(1e-20,cat_full_yield))
                cat_num_list.append(np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier*max(0,delta))
                
                rates_c.append([i,r, reduced_mech.reactions[r].reactants,np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier])
            data_c.append(rates_c)
        #cat_rel_rate = cat_rel_rate/len(caty)
        #cat_rel_out = cat_rel_out/len(caty)
        cdata = pd.DataFrame({'num':cat_num_list,'weight':cat_weight_list})
        if len(cdata)>0:
            cat_median_rate = weighted_median(cdata,'num', 'weight')
        else:
            cat_median_rate = 1e-5
        data_c.append(cat_rel_rate)
        mult_list = []
        for i in caty:
            spec_yield_frac = caty_yields[i]/max(1e-20,cat_full_yield)
            species_rate = 0
            for r in rxn_reac[i]:
                species_rate+= np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier
            if species_rate>0:  
                multies.append([i, spec_yield_frac, cat_rel_rate, species_rate])
                mult = spec_yield_frac*(cat_median_rate)/species_rate
                mult_list.append([i,mult, cat_rel_rate,species_rate, spec_yield_frac])
                
            else:
                mult = 0            
            cat_map.append(i)
            for k in dag_in[i]:
                dag_out[k].add(new_spec)
                dag_in[new_spec].add(k)
                if i in dag_graph[k]:
                    if new_spec not in dag_graph[k]:
                        dag_graph[k][new_spec] = dag_graph[k][i]
                    else:
                        dag_graph[k][new_spec] += dag_graph[k][i]
                else:
                    pass
                    #print('matchning issue', k, i)
                if i in dag_out[k]:
                    dag_out[k].remove(i)
                    if i in dag_graph[k]:    
                        del dag_graph[k][i]
                    else:
                        pass
            skip_r = set()

            for r in rxn_prod[i]:
                
                
                if not any([x in caty for x in reduced_mech.reactions[r].reactants]) and r not in skip_r:
                    

                    rxy_prod.add(r)
                    
                    if i in reduced_mech.reactions[r].prod_dict:
                        if new_spec in reduced_mech.reactions[r].prod_dict:
                            reduced_mech.reactions[r].prod_dict[new_spec] += reduced_mech.reactions[r].prod_dict[i]
                        else:
                            reduced_mech.reactions[r].prod_dict[new_spec] = reduced_mech.reactions[r].prod_dict[i]
                else:
                    remove_these_rxns.add(r)
                
            for r in rxn_reac[i]:
                if any([x not in caty for x in reduced_mech.reactions[r].prod_dict]) and r not in skip_r:
                    delta = 1
                     
                    for x in reduced_mech.reactions[r].prod_dict:
                        if x in caty:
                            delta = delta - reduced_mech.reactions[r].prod_dict[x]
                    if delta>0:
                        out_reactions[r] = delta*avg_yield_full[i]
                        net_out_yield = delta*avg_yield_full[i]
                        rxy_reac.add(r)
                        reduced_mech.reactions[r].multiplier *= mult*delta
                        new_prods = {}
                        p_count = 0
                        for p in reduced_mech.reactions[r].prod_dict:
                            if p not in caty:
                                p_count+=reduced_mech.reactions[r].prod_dict[p]

                        for p in reduced_mech.reactions[r].prod_dict:
                            if p not in caty:
                                new_prods[p] = reduced_mech.reactions[r].prod_dict[p] + (1 - delta)*(reduced_mech.reactions[r].prod_dict[p]/max(p_count,1e-20))
                            if (1/delta)*reduced_mech.reactions[r].prod_dict[p]>3:
                                #print('maxed it')
                                pass
                        if p_count == 0 and 'XCLOST' in species_list_names:
                                
                            if dic['XCLOST'] in new_prods:
                                new_prods[dic['XCLOST']] += 1-delta
                            else:
                                new_prods[dic['XCLOST']] = 1-delta
                        reduced_mech.reactions[r].prod_dict = new_prods
                    else:
                        remove_these_rxns.add(r)
                    reduced_mech.reactions[r].reactants.append(new_spec)
                    if i in reduced_mech.reactions[r].reactants:
                        reduced_mech.reactions[r].reactants.remove(i)
                elif all([x not in caty for x in reduced_mech.reactions[r].prod_dict]) and r not in skip_r:
                    out_reactions[r] = avg_yield_full[i]
                    net_out_yield = avg_yield_full[i]
                    rxy_reac.add(r)
                    reduced_mech.reactions[r].multiplier *= mult
                    multy.append([r, reduced_mech.reactions[r].multiplier, mult])
                    new_prods = {}
                    if i in scc_set:
                        new_prods = dag_graph[i]
                        reduced_mech.reactions[r].prod_dict = new_prods
                    reduced_mech.reactions[r].reactants.append(new_spec)
                    if i in reduced_mech.reactions[r].reactants:
                        reduced_mech.reactions[r].reactants.remove(i)
                else:
                    
                    for p in reduced_mech.reactions[r].prod_dict:
                        if r in rxn_prod[p]:
                            rxn_prod[p].remove(r)
                    remove_these_rxns.add(r)
        mult_list_list.append(mult_list)
        mapping.append(cat_map)
        net_out_graph = {}
        multses.append(multies)
        # out reactions + graph outs
        tot_cat_rate = 0
        for r in out_reactions:
            tot_cat_rate += np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier
        for r in out_reactions:
            r_eff = np.mean(reduced_mech.reactions[r].rate)*reduced_mech.reactions[r].multiplier/max(1e-20,tot_cat_rate)
            for p in reduced_mech.reactions[r].prod_dict:
                if p in net_out_graph:
                    net_out_graph[p]+= r_eff*reduced_mech.reactions[r].prod_dict[p]
                else:
                    net_out_graph[p] = r_eff*reduced_mech.reactions[r].prod_dict[p]

        for i in caty:
            for k in dag_out[i]:
                if k not in caty:
                    dag_in[k].add(new_spec)
                    dag_out[new_spec].add(k)
                    if i in dag_in[k]:
                        dag_in[k].remove(i)
            for k in dag_in[i]:
                if i in dag_graph[k]:
                    del dag_graph[k][i]
                if i in dag_out[k]:
                    dag_out[k].remove(i)
            dag_in[i] = set()
        dag_graph[new_spec] = net_out_graph
        rxn_reac.append(rxy_reac)
        rxn_prod.append(rxy_prod)
        cat_mapping.append(mapping)
        cat_data.append(data_c)
  
    if settings['Print Progress'] == True:
        print('Stage 7: Creating Species Removal Tiers')


    group_set = set()
    group_dict = {}
    for i in range(len(groups)):
        for j in groups[i]:
            group_dict[j] = i
            group_set.add(j)
    remaining_scc = set()
    removed_scc = set()
    remaining_groups = set()
    removed_groups = set()
    group_lead = {}
    reduced_mech_1 = deepcopy(reduced_mech)
    
    for i in cat_specs:
        remaining_specs.append(i)
    
    for i in remaining_specs:
        if i in scc_set:
            remaining_scc.add(scc_dict_2[i])
        if i in group_set:
            remaining_groups.add(group_dict[i])
            group_lead[group_dict[i]] = i
    for i in grouped:
        if i in scc_set:
            remaining_scc.add(scc_dict_2[i])
    removed_scc = set(range(len(scc))).difference(remaining_scc)
    removed_groups = set(range(len(groups))).difference(remaining_groups)

    
    tiers = [[]]
    cat_set = set()
    group_set = set()
    tier_set = set(background_spc).union(remaining_specs)
    for i in grouped:
        tier_set.add(i)

    delta = 1
 
    in_cycle_spec_set = set()
    for i in in_cycle_specs:
        for j in i:
            in_cycle_spec_set.add(j)
    remaining_search = {i for i in range(spec_len) if i not in tier_set}
    for i in scc_remove_species:
        if i not in in_cycle_spec_set:
            if i in remaining_search:
                remaining_search.remove(i)
    for i in cat_set:
        if i not in not_removed and i in remaining_search:
            remaining_search.remove(i)
    for i in group_set:
        if i not in not_removed and i in remaining_search:
            remaining_search.remove(i)
    scc_remove_no_in = set()
    for i in scc_remove_species:
        if i not in in_cycle_spec_set:
            scc_remove_no_in.add(i)
    

    remove_1 = deepcopy(remove_these_rxns)
    
    while delta>0:
        new_to_add = set()
        tier = set()
        to_remove = set()
        for j in remaining_search:
            if all([x in tier_set for x in dag_out[j]]):
                new_to_add.add(j)
                to_remove.add(j)
                if j not in cat_set and j not in group_set and j not in scc_remove_no_in:
                    tier.add(j)
        for p in to_remove:
            remaining_search.remove(p)
        delta = len(tier)
        tiers.append(tier)
        tier_set = tier_set.union(new_to_add)
    if len(remaining_search)>0:
        tiers.append(remaining_search)
    
    dag_graph_save = deepcopy(dag_graph)
    
   
    count = 0
    dag_graph_save = deepcopy(dag_graph)

    dag_graph_save_2 = deepcopy(dag_graph)

    remove_2 = deepcopy(remove_these_rxns)
    normal_remove = deepcopy(remove_species)
    normal_remove = set(range(spec_len))
    removed_species = set()
    true_count = 0
    false_count =0
    remaining_specs = set(remaining_specs)


    remove_3 = deepcopy(remove_these_rxns)
    
    if settings['Print Progress'] == True:
        print('Stage 8: Removing Remaining Species')

    reduced_mech_1 = deepcopy(reduced_mech)
    for t in tiers:
        for i in t:
            if i in scc_set:
                scc_indy = scc_dict_2[i]
                if i not in grouped:
    
                    count+=1
                    if count%1000==0:
                        pass
                        #print(count)
                    for k in dag_in[i]:
                        if k not in scc[scc_indy]:
    
                            if i in dag_graph[k]:
                                mult = dag_graph[k][i]
                            else:
                                mult = 0
    
                            for p in dag_graph[i]:
                                dag_in[p].add(k)
                                dag_out[k].add(p)
                                if p not in dag_graph[k]:
                                    dag_graph[k][p] = mult*dag_graph[i][p]
                                else:
                                    dag_graph[k][p] += mult*dag_graph[i][p]
    
                            if i in dag_out[k]:
                                dag_out[k].remove(i)
                            if i in dag_graph[k]:
                                del dag_graph[k][i]    

    
                    for r in rxn_prod[i]:
                        if r not in removed_rxns:
                            if not any([x in scc[scc_indy] for x in reduced_mech.reactions[r].reactants]):
                                if i in reduced_mech.reactions[r].prod_dict:
                                    mult = reduced_mech.reactions[r].prod_dict[i]
                                else:
                                    mult = 0
                                for p in dag_graph[i]:
                                    if p not in reduced_mech.reactions[r].reactants and p not in scc[scc_indy]:
                                        if p in reduced_mech.reactions[r].prod_dict:
                                            reduced_mech.reactions[r].prod_dict[p]+= mult*dag_graph[i][p]
                                        else:
                                            reduced_mech.reactions[r].prod_dict[p] = mult*dag_graph[i][p]
                                        rxn_prod[p].add(r)
                    for r in rxn_reac[i]:
                        removed_rxns.add(r)
            else:

                removed_species.add(i)
                count+=1
    
                removers = set()
                for j in dag_in[i]:
                    if i in dag_out[j]:
                        dag_out[j].remove(i)
                    if i in dag_graph[j]:
                        mult = dag_graph[j][i]
                        del dag_graph[j][i]
                    else:
                        mult = 0
    
                    for k in dag_out[i]:
                        
                        if k!=j and dag_graph[i][k]*mult!=0:
                            dag_out[j].add(k)
                            dag_in[k].add(j)
                            if k in dag_graph[j]:
                                dag_graph[j][k] += dag_graph[i][k]*mult
                            else:
                                dag_graph[j][k] = dag_graph[i][k]*mult

     
                    removers.add(j)
                    
                    
    
                if any([x in remaining_specs for x in dag_in[i]]):
                    true_count+=1
                    
                    for r in rxn_prod[i]:
    
                        if r not in removed_rxns:
    
                            if i in reduced_mech.reactions[r].prod_dict:
                                mult = reduced_mech.reactions[r].prod_dict[i]
                            else:
                                mult = 0
                            for j in dag_graph[i]:
    
                                if j not in reduced_mech.reactions[r].reactants and mult*dag_graph[i][j]!=0:
    
                                    rxn_prod[j].add(r)
                                    if j in reduced_mech.reactions[r].prod_dict:
                                        reduced_mech.reactions[r].prod_dict[j] += mult*dag_graph[i][j]
                                    else:
                                        reduced_mech.reactions[r].prod_dict[j] = mult*dag_graph[i][j]
                    for r in rxn_reac[i]:
    
                        removed_rxns.add(r)
                for p in removers:
                    dag_in[i].remove(p)



    reduced_mech_3 = deepcopy(reduced_mech)
    
    remove_4 = deepcopy(remove_these_rxns)


    if settings['Print Progress'] == True:
        print('Stage 9: Mechanism Clean Up')
    remove_set = set(remove_species)
    back_set_set = set(back_set)
    full_set = remove_set.union(back_set_set)
    
    for i in range(len(reduced_mech.reactions)):
        
        for j in reduced_mech.reactions[i].reactants:
            if j in remove_set:
                remove_these_rxns.add(i)

    new_reactions = []
    count = 0
    for i in range(len(reduced_mech.reactions)):
        if i not in remove_these_rxns:
            
            new_reactions.append(deepcopy(reduced_mech.reactions[i]))
            count+=1


    remove_species = set(remove_species)

    for i in range(len(new_reactions)):

        new_reactants = deepcopy(new_reactions[i].reactants)
        new_prod_dict = {}
        for j in new_reactions[i].prod_dict:
            if new_reactions[i].prod_dict[j]>0.001 and j not in remove_species:
                # FLAG THIS 0.001 value
                new_prod_dict[j] = new_reactions[i].prod_dict[j]
        new_reactions[i].prod_dict = new_prod_dict



    new_new_reactions = []
    reduced_mech_1 = deepcopy(reduced_mech)
    for i in new_reactions:
        if i.multiplier>0:
            new_new_reactions.append(i)

    new_reactions = deepcopy(new_new_reactions)
    
    remove_5 = deepcopy(remove_these_rxns)

    reduced_mech.reactions = deepcopy(new_reactions)
   


    accounted_specs = set()
    new_cat_mapping = []
    map_dict = {}
    for i in background_spc:
        map_dict[i] = i
    for i in cat_mapping:
        if len(i[1])>0:
            accounted_specs.add(i[0])
            new_cat_mapping.append(i)
            for k in i[1]:
                map_dict[k] = i[0]
    new_group_mapping = []
    group_map_dict = {}
    for i in group_mapping:
        if len(i[1])>0:
            accounted_specs.add(i[0])
            new_group_mapping.append(i)
            group_map_dict[i[0]] = i[1]
            for k in i[1]:
                map_dict[k] = i[0]
        map_dict[i[0]] = i[0]
            
    normal_specs = []
    for i in remaining_specs:
        if i not in accounted_specs:
            normal_specs.append(i)
            map_dict[i] = i
    represented_specs = []
    for i in new_group_mapping:
        represented_specs.append(i[0])
        for j in i[1]:
            represented_specs.append(j)
    for i in new_cat_mapping:
        for j in i[1]:
            represented_specs.append(j)
    for i in normal_specs:
        represented_specs.append(i)
    full_map = [new_group_mapping, new_cat_mapping, normal_specs]
    map_yields = [[] for i in range(len(represented_specs))]
    inv_map_dict = {}
    for i in map_dict:
        if map_dict[i] in inv_map_dict:
            inv_map_dict[map_dict[i]].add(i)
        else:
            inv_map_dict[map_dict[i]] = {i}
    represented_set = set(represented_specs)
    
    if settings['Print Progress'] == True:
        print('Stage 10: Analyzing Reduced Mechanism')

        
    red_graphs = [[{} for i in range(spec_len)] for c in range(l_c)]
    red_in_graph = [set() for i in range(spec_len)]
    red_out_graph = [set() for i in range(spec_len)]

    
    red_dag_out = []
    new_rxns = []
    for i in reduced_mech.reactions:
        if i.prod_dict!={}:
            new_rxns.append(i)

    reduced_mech.reactions = deepcopy(new_rxns)
    red_specs = []
    for i in new_dict:
        if new_dict[i] in remaining_specs:
            red_specs.append(new_dict[i])

    red_rxn_reac = {}
    red_rxn_prod = {}
    red_spec_dict = {}
    for i in range(len(red_specs)):
        red_spec_dict[red_specs[i]] = i
    for i in range(len(reduced_mech.reactions)):
        for j in reduced_mech.reactions[i].reactants:
            if j not in back_set:
                if j in red_rxn_reac:  
                    red_rxn_reac[j].add(i)
                else:
                    red_rxn_reac[j] = {i}
        for p in reduced_mech.reactions[i].prod_dict:
            if p not in back_set:
                if p in red_rxn_prod:
                    red_rxn_prod[p].add(i)
                else:
                    red_rxn_prod[p] = {i}
    for i in red_rxn_reac:
        if i not in back_set:
            edges = [{} for c in range(l_c)]
            rate_sums = [0 for c in range(l_c)]
            
            for j in red_rxn_reac[i]:

                for c in range(l_c):
                    rate_sums[c] = rate_sums[c] + reduced_mech.reactions[j].rate[c]
            for j in red_rxn_reac[i]:
                mults = [0 for c in range(l_c)]
                for c in range(l_c):
                    mults[c] = reduced_mech.reactions[j].rate[c]/max(1e-20,rate_sums[c])
                if len(reduced_mech.reactions[j].reactants) == 1:
                    type_r = 'solo'
                else:
                    mark = True
                    for p in reduced_mech.reactions[j].reactants:
                        if p in background_spc:
                            type_r = p
                            mark = False
                    if mark:
                        type_r = 'double'
                for k in reduced_mech.reactions[j].prod_dict:
                    
                    for c in range(l_c):
                        if k in edges[c]:
                            if c==0:
                                edges[c][k] = edges[c][k] + reduced_mech.reactions[j].prod_dict[k]*mults[c]
                                red_out_graph[i].add(k)
                                red_in_graph[k].add(i)
                            else:
                                edges[c][k] = edges[c][k] + reduced_mech.reactions[j].prod_dict[k]*mults[c]

                        
                        else:
                            if c==0:
                                edges[c][k] = reduced_mech.reactions[j].prod_dict[k]*mults[c]
                                red_out_graph[i].add(k)
                                red_in_graph[k].add(i)
                            else:
                                edges[c][k] = reduced_mech.reactions[j].prod_dict[k]*mults[c]
            for c in range(l_c):
                red_graphs[c][i] = edges[c]
    
    
    

    for i in red_rxn_reac:
        if i not in back_set:
            edges = [{} for c in range(l_c)]
            rate_sums = [0 for c in range(l_c)]
            
            for j in red_rxn_reac[i]:

                for c in range(l_c):
                    rate_sums[c] = rate_sums[c] + reduced_mech.reactions[j].rate[c]*reduced_mech.reactions[j].multiplier
            for j in red_rxn_reac[i]:
                mults = [0 for c in range(l_c)]
                for c in range(l_c):
                    mults[c] = reduced_mech.reactions[j].multiplier*reduced_mech.reactions[j].rate[c]/max(1e-20,rate_sums[c])
                if len(reduced_mech.reactions[j].reactants) == 1:
                    type_r = 'solo'
                else:
                    mark = True
                    for p in reduced_mech.reactions[j].reactants:
                        if p in background_spc:
                            type_r = p
                            mark = False
                    if mark:
                        type_r = 'double'
                for k in reduced_mech.reactions[j].prod_dict:
                    
                    for c in range(l_c):
                        if k in edges[c]:
                            if c==0:
                                edges[c][k] = edges[c][k] + reduced_mech.reactions[j].prod_dict[k]*mults[c]
                                red_out_graph[i].add(k)
                                red_in_graph[k].add(i)
                            else:
                                edges[c][k] = edges[c][k] + reduced_mech.reactions[j].prod_dict[k]*mults[c]

                        
                        else:
                            if c==0:
                                edges[c][k] = reduced_mech.reactions[j].prod_dict[k]*mults[c]
                                red_out_graph[i].add(k)
                                red_in_graph[k].add(i)
                            else:
                                edges[c][k] = reduced_mech.reactions[j].prod_dict[k]*mults[c]
            for c in range(l_c):
                red_graphs[c][i] = edges[c]
    
    red_graph_mat_lil = lil_matrix((spec_len,spec_len), dtype=None)
    

        
    red_edges = []
    for i in range(len(red_out_graph)):
        for j in red_out_graph[i]:
            red_edges.append((i,j))
    count = 0
    for i,j in red_edges:
        red_graph_mat_lil[i,j] = 1
    reduced_mech_4 = deepcopy(reduced_mech)
    red_graph_mat = red_graph_mat_lil.tocsr(copy=False)

    red_scc_result = connected_components(red_graph_mat, directed=True, connection='strong', return_labels=True)

    red_scc_dic = {}
    for i in range(len(red_scc_result[1])):
        if red_scc_result[1][i] in red_scc_dic:
            red_scc_dic[red_scc_result[1][i]].append(i)
        else:
            red_scc_dic[red_scc_result[1][i]] = [i]
    
    red_scc = []
    
    for i in red_scc_dic:
        if len(red_scc_dic[i])>1:
            if not any([x in cat_specs for x in red_scc_dic[i]]):
                pass
            else:
                c_specs = []
                for x in red_scc_dic[i]:
                    if x in cat_specs:
                        c_specs.append(x)
                not_allowed = set()
                for x in red_scc_dic[i]:
                    if x not in c_specs:
                        not_allowed.add(x)
                for c in c_specs:
                    new_out_g = set()
                    for j in red_out_graph[c]:
                        if j not in not_allowed:
                            new_out_g.add(j)
                        else:
                            if c in red_in_graph[j]:
                                red_in_graph[j].remove(c)
                    red_out_graph[c] = new_out_g

                    for cond in range(l_c):
                        new_graph_c = {}
                        for j in red_graphs[cond][c]:
                            if j not in not_allowed:
                                new_graph_c[j] = red_graphs[cond][c][j]
                        red_graphs[cond][c] = deepcopy(new_graph_c)
                    
                    for r in red_rxn_reac[c]:
                        new_p = {}
                        for p in reduced_mech.reactions[r].prod_dict:
                            if p not in not_allowed:
                                new_p[p] = reduced_mech.reactions[r].prod_dict[p]
                            else:
                                red_rxn_prod[p].remove(r)
                        reduced_mech.reactions[r].prod_dict = new_p

    red_graph_mat_lil = lil_matrix((spec_len,spec_len), dtype=None)
    red_edges = []
    for i in range(len(red_out_graph)):
        for j in red_out_graph[i]:
            red_edges.append((i,j))
    count = 0
    for i,j in red_edges:
        red_graph_mat_lil[i,j] = 1
    reduced_mech_4 = deepcopy(reduced_mech)
    red_graph_mat = red_graph_mat_lil.tocsr(copy=False)

    red_scc_result = connected_components(red_graph_mat, directed=True, connection='strong', return_labels=True)

    red_scc_dic = {}
    for i in range(len(red_scc_result[1])):
        if red_scc_result[1][i] in red_scc_dic:
            red_scc_dic[red_scc_result[1][i]].append(i)
        else:
            red_scc_dic[red_scc_result[1][i]] = [i]
    for i in red_scc_dic:
        if len(red_scc_dic[i])>1:
            red_scc.append(red_scc_dic[i])
            
 

    red_scc_dict_2 = {}
    red_scc_set = set()
    for i in range(len(red_scc)):
        for j in red_scc[i]:
            red_scc_dict_2[j] = i
            red_scc_set.add(j)
    red_scc_map = {}
    for i in red_scc_set:
        red_scc_map[i] = inv_map_dict[i]
    

    if settings['Print Progress'] == True:
        print('Stage 11: Optimizing Cyclical Rate Constants')

    scc_ref_data = {}
    represented_scc = set()
    for i in red_scc_map:
        for j in red_scc_map[i]:
            if j in scc_dict_2:
                represented_scc.add(scc_dict_2[j])
    if settings['Iterations']>0:
        for i in represented_scc:
            ref_data = []
            for c in range(l_c):
                in_specs = []
                for k in in_cycle_specs[i]:
                    k_start = 0
                    for s in in_graph[k]:
                       
                        if s not in scc[i]:
                            if k in graphs[c][s]:
                                k_start+=graphs[c][s][k]*yields[c][s]
                    in_specs.append([k,k_start])
                    
                data, within_data, out_data = cycle_simulator_3_modified(scc[i], in_specs, graphs[c],out_graph,in_graph, [leny+40,2*(leny+40)],1e-6,set(scc[i]).union(scc_out_specs[i]),scc_out_specs[i])
    
                
                new_data = {}
                for d in data:
                    if d not in represented_set:
                        
                        yi = get_yields_modified([d], represented_set, 1e-10,new_graphs[c], new_ins[c], new_outs[c], back_set,scc, scc_dict_2, scc_set)
                        for y in yi:
                            if yi[y]!=0:
                                if y in new_data:
                                    new_data[y]+=data[d]*yi[y]
                                else:
                                    new_data[y]=data[d]*yi[y]
                    else:
                        if d in new_data:
                            new_data[d]+= data[d]
                        else:
                            new_data[d]= data[d]
    
                ref_data.append([new_data, within_data, out_data])
            scc_ref_data[i] = ref_data

    red_dag_out = deepcopy(red_out_graph)
    red_dag_in = deepcopy(red_in_graph)
    xspecs = []
    for i in red_scc:
 
        xspec = i[0]
        xspecs.append(i[0])
        for j in i:
            if j != xspec:
                for x in red_dag_out[j]:
                    if x not in i:
                        red_dag_out[xspec].add(x)
                        red_dag_in[x].add(xspec)
                    if j in red_dag_in[x]:
                        red_dag_in[x].remove(j)
                for x in red_dag_in[j]:
                    red_dag_out[x].add(xspec)
                    red_dag_in[xspec].add(x)
                    red_dag_out[x].remove(j)
                red_dag_out[j] = set()
                red_dag_in[j] = set()

    for i in range(len(red_dag_in)):
        if i in red_dag_in[i]:
            red_dag_in[i].remove(i)
        if i in red_dag_out[i]:
            red_dag_out[i].remove(i)

    searcher = set(deepcopy(roots))
    dont_visit = set()
    for i in range(len(red_dag_in)):
        if len(red_dag_in[i])==0:
            searcher.add(i)
            dont_visit.add(i)
    visited = []
    visit_set = set()
    while len(searcher)>0:
        
        new_searcher = deepcopy(searcher)
        for i in searcher:
            if i not in visit_set:
                if i not in dont_visit:
                    visited.append(i)
                visit_set.add(i)
            new_searcher.remove(i)
            for j in red_dag_out[i]:
                if all([x in visit_set for x in red_dag_in[j]]):
                    if j not in back_set and j not in visit_set:
                        new_searcher.add(j)
        searcher = deepcopy(new_searcher)
    
 
    red_scc_ins = []
    for i in red_scc:
        in_specy = set()
        for k in i:
            for j in red_in_graph[k]:
                if j not in i:
                    in_specy.add(j)
        red_scc_ins.append(in_specy)
        
    in_specy_set = set()
    for i in range(len(red_scc_ins)):
        for j in red_scc_ins[i]:
            in_specy_set.add(j)

    red_scc_order = []

    count = 0

    for i in visited:
        if i in red_scc_set:
            red_scc_order.append(red_scc_dict_2[i])
    
    red_scc_ins_2 = []
    for i in red_scc:
        insy = set()
        for j in i:
            mark = False
            for k in red_in_graph[j]:
                if k not in i:
                    mark = True
            if mark:
                insy.add(j)
        red_scc_ins_2.append(insy)
    
    
    scc_ref_data_2 = {}

    new_red_scc_order = []
    for i in red_scc_order:
        if not any([x in cat_specs for x in red_scc[i]]):
            new_red_scc_order.append(i)
    red_scc_order = deepcopy(new_red_scc_order)

    for m in range(len(red_scc_order)):
        m_para = {}
        for n in range(len(reduced_mech.reactions)):
            if any([f in reduced_mech.reactions[n].reactants for f in red_scc[red_scc_order[m]]]):
                mark = True
                
                if mark:
                    m_para[n] = reduced_mech.reactions[n].multiplier
        if settings['Print Progress'] == True:
            print('Constants to Optimize: ',m,len(m_para))

    
    scc_sets = []
    for i in scc:
        scc_sets.append(set(i))
    red_scc_sets = []
    for i in red_scc:
        red_scc_sets.append(set(i))
    for i in range(len(red_scc)):
        if red_scc_sets[i] in scc_sets:
            if i in red_scc_order:
                red_scc_order.remove(i)
    if settings['Iterations']>0:
        for m in range(len(red_scc_order)):
            
            scc_rem = red_scc_order[m]
            stop_specs = set()
            for x in range(len(red_dag_out)):
                if len(red_dag_out[x])==0:
                    stop_specs.add(x)
            for x in red_scc_order[m:]:
                for j in red_scc[x]:
                    stop_specs.add(j)
            rep_sccs = set()
            for j in red_scc[red_scc_order[m]]:
                for k in red_scc_map[j]:
                    if k in scc_dict_2:
                        rep_sccs.add(scc_dict_2[k])
            data = []
            for c in range(l_c):
                daty = [{}, {}, {}]
                for f in rep_sccs:
                    for p in scc_ref_data[f][c][0]:
    
                        if p in daty:
                            daty[0][p]+= scc_ref_data[f][c][0][p]
                        else:
                            daty[0][p]= scc_ref_data[f][c][0][p] 
                    for p in scc_ref_data[f][c][1]:
    
                        if p in daty:
                            daty[1][p]+= scc_ref_data[f][c][1][p]
                        else:
                            daty[1][p]= scc_ref_data[f][c][1][p] 
                    for p in scc_ref_data[f][c][2]:
    
                        if p in daty:
                            daty[2][p]+= scc_ref_data[f][c][2][p]
                        else:
                            daty[2][p]= scc_ref_data[f][c][2][p] 
                for sp in red_scc[red_scc_order[m]]:
                    if sp not in daty[0]:
                        daty[0][sp] = 0
                    if sp not in daty[1]:
                        daty[1][sp] = 0
                    if sp not in daty[2]:
                        daty[2][sp] = 0
                data.append(daty)
            scc_ref_data_2[red_scc_order[m]] = data
    
            all_starts = []
            for c in range(l_c):
                new_graph = []
                for p in red_graphs[c]:
                    dicy = {}
                    for j in p:
                        dicy[j] = p[j]
                    new_graph.append(dicy)
    
    
                new_in = []
                for p in red_in_graph:
                    sety = set()
                    for j in p:
                        sety.add(j)
                    new_in.append(sety)
    
                new_out = []
                for p in red_out_graph:
                    sety = set()
                    for j in p:
                        sety.add(j)
                    new_out.append(sety)
    
    
                for n in range(len(red_scc)):
                    counte = 0
                    out_specs = set()
                    all_specs = set(red_scc[n])
                    for x in red_scc[n]:
                        for j in red_out_graph[x]:
                            if j not in red_scc[n]:
                                out_specs.add(j)
                                all_specs.add(j)
                    for x in red_scc_ins_2[n]:
                        counte +=1
                        new_out[x] = all_specs
                        leny = int(np.sqrt(len(red_scc[n])))
                        data, in_cyc_data = cycle_simulator_3(red_scc[n], x, red_graphs[c],red_out_graph,red_in_graph, [leny+40,2*(leny+40)],1e-6,all_specs,out_specs)
                        cycle_out_graphs[c][x] = data
                        ful_dat = {}
                        for p in data:
                            ful_dat[p] = data[p]
                        for p in in_cyc_data:
                            ful_dat[p] = in_cyc_data[p]
                        new_graph[x] = ful_dat
    
                    for y in red_scc[m]:
                        if y not in red_scc_ins_2[red_scc_order[m]]:
    
                            for s in out_graph[y]:
                                if y in new_in[s]:
                                    new_in[s].remove(y)
                            new_in[y] = set(red_scc_ins_2[red_scc_order[m]])
                            new_out[y] = set()
                            new_graph[y] = set()
                            
                graph_mat_lil = lil_matrix((spec_len,spec_len), dtype=None)
    
                edges = []
                for i in range(len(new_out)):
                    for j in new_out[i]:
                        edges.append((i,j))
                count = 0
                for i,j in edges:
                    graph_mat_lil[i,j] = 1
            
                graph_mat = graph_mat_lil.tocsr(copy=False)
            
                scc_result = connected_components(graph_mat, directed=True, connection='strong', return_labels=True)
            
                scc_dic = {}
                for i in range(len(scc_result[1])):
                    if scc_result[1][i] in scc_dic:
                        scc_dic[scc_result[1][i]].append(i)
                    else:
                        scc_dic[scc_result[1][i]] = [i]
    
                yi = get_yields_modified(roots, stop_specs, 1e-10, new_graph, new_in, new_out, back_set,red_scc, red_scc_dict_2, red_scc_set)
               

                starters = []
                for j in red_scc_ins_2[red_scc_order[m]]:
                    start_val = 0
                    for k in red_in_graph[j]:
                        if k not in red_scc[red_scc_order[m]]:
                            if j in red_graphs[c][k] and k in yi:
                                start_val+= red_graphs[c][k][j]*yi[k]
                    
                    starters.append([j, start_val])
                all_starts.append(starters)
            data_map = []
            for j in red_scc[red_scc_order[m]]:
    
                mapper1 = [1]
                mapper2 = [0]
                mapper1.append(j)
                mapper2.append(j)
                comps = []
                
                for k in red_scc_map[j]:
                    comps.append([k,1])

                mapper1.append(comps)
                mapper2.append(comps)
                data_map.append(mapper1)
            out_specs = set()
            all_specs = set(red_scc[red_scc_order[m]])
            for x in red_scc[red_scc_order[m]]:
                for j in red_out_graph[x]:
                    if j not in red_scc[red_scc_order[m]]:
                        out_specs.add(j)
                        all_specs.add(j)
    
            for j in out_specs:
    
                mapper2 = [0]
                mapper2.append(j)
                comps = []
                denom = 0
                for k in inv_map_dict[j]:
                    denom+=avg_yield_o[k]
                for k in inv_map_dict[j]:
                    comps.append([k,avg_yield_o[k]/max(1e-20,denom)])

                mapper2.append(comps)
                data_map.append(mapper2)
            
            cycle_info = [red_scc[red_scc_order[m]], all_starts, all_specs, out_specs]
            settings_des = {}
            settings_des['learn_rate'] = 0.0002
            settings_des['steps'] = settings['Iterations']
            settings_des['grad_step'] = 1.001

            m_para_map = {}
            m_para = {}
            for n in range(len(reduced_mech.reactions)):
                if any([f in reduced_mech.reactions[n].reactants for f in red_scc[red_scc_order[m]]]):
                    mark = True

                    if mark:
                        m_para_map[n] = [n]
                        m_para[n] = reduced_mech.reactions[n].multiplier
    
            reduced_mech = grad_descent_cycles(m_para, m_para_map, reduced_mech, scc_ref_data_2[red_scc_order[m]], data_map, cycle_info, settings_des, conditions, red_specs, back_set, len(new_species), red_scc[scc_rem])
            
            red_graphs = [[{} for i in range(spec_len)] for c in range(l_c)]
            red_in_graph = [set() for i in range(spec_len)]
            red_out_graph = [set() for i in range(spec_len)]
            for nn in red_rxn_reac:
                if nn not in back_set:
                    edges = [{} for c in range(l_c)]
                    rate_sums = [0 for c in range(l_c)]
    
                    for j in red_rxn_reac[nn]:
    
                        for c in range(l_c):
                            rate_sums[c] = rate_sums[c] + reduced_mech.reactions[j].multiplier*reduced_mech.reactions[j].rate[c]
                    for j in red_rxn_reac[nn]:
                        mults = [0 for c in range(l_c)]
                        for c in range(l_c):
                            mults[c] = reduced_mech.reactions[j].multiplier*reduced_mech.reactions[j].rate[c]/max(1e-20,rate_sums[c])
                        if len(reduced_mech.reactions[j].reactants) == 1:
                            type_r = 'solo'
                        else:
                            mark = True
                            for p in reduced_mech.reactions[j].reactants:
                                if p in background_spc:
                                    type_r = p
                                    mark = False
                            if mark:
                                type_r = 'double'
                        for k in reduced_mech.reactions[j].prod_dict:
    
                       
                            for c in range(l_c):
                                if k in edges[c]:
                                    if c==0:
                                        edges[c][k] = edges[c][k] + reduced_mech.reactions[j].prod_dict[k]*mults[c]
                                        red_out_graph[nn].add(k)
                                        red_in_graph[k].add(nn)
                          
                                    else:
                                        edges[c][k] = edges[c][k] + reduced_mech.reactions[j].prod_dict[k]*mults[c]
    
    
                                else:
                                    if c==0:
                                        edges[c][k] = reduced_mech.reactions[j].prod_dict[k]*mults[c]
                                        red_out_graph[nn].add(k)
                                        red_in_graph[k].add(nn)
                                      
                                    else:
                                        edges[c][k] = reduced_mech.reactions[j].prod_dict[k]*mults[c]
                    for c in range(l_c):
                        red_graphs[c][nn] = edges[c]
    
    if settings['Print Progress'] == True:
        print('Stage 12: Getting Reduced Mechanism Yield')

    
    red_yields = []     
    for c in range(l_c):
        new_graph = []
        for p in red_graphs[c]:
            dicy = {}
            for j in p:
                dicy[j] = p[j]
            new_graph.append(dicy)


        new_in = []
        for p in red_in_graph:
            sety = set()
            for j in p:
                sety.add(j)
            new_in.append(sety)

        new_out = []
        for p in red_out_graph:
            sety = set()
            for j in p:
                sety.add(j)
            new_out.append(sety)


        for n in range(len(red_scc)):
            counte = 0
            out_specs = set()
            all_specs = set(red_scc[n])
            for x in red_scc[n]:
                for j in red_out_graph[x]:
                    if j not in red_scc[n]:
                        out_specs.add(j)
                        all_specs.add(j)
            for x in red_scc_ins_2[n]:
                counte +=1
                new_out[x] = all_specs
                leny = int(np.sqrt(len(red_scc[n])))
                data, in_cyc_data = cycle_simulator_3(red_scc[n], x, red_graphs[c],red_out_graph,red_in_graph, [leny+40,2*(leny+40)],1e-6,all_specs,out_specs)
                cycle_out_graphs[c][x] = data
                ful_dat = {}
                for p in data:
                    ful_dat[p] = data[p]

                new_graph[x] = ful_dat

            for y in red_scc[n]:
                if y not in red_scc_ins_2[n]:

                    for s in red_out_graph[y]:
                        if y in new_in[s]:
                            new_in[s].remove(y)
                    new_in[y] = set(red_scc_ins_2[n])
                    new_out[y] = set()
                    new_graph[y] = set()


        yi = get_yields_modified(roots, [], 0, new_graph, new_in, new_out, back_set,red_scc, red_scc_dict_2, red_scc_set)
        red_yields.append(yi)

    if settings['Reduce Stiffness'] == True:
        
        real_rates = []
        count = 0
        for i in reduced_mech.reactions:
            real_rates.append([count, np.mean(i.rate)*i.multiplier])
            count+=1
        sorted_rates = sorted(real_rates, key=lambda x: x[1]) 
        sorted_rates_only = [i[1] for i in sorted_rates]
        if 'Reference Rate' in settings:
            ref = settings['Reference Rate']
        else:
            ref = np.median(sorted_rates_only)
        
        differences = [math.log10(max(i[1], 1e-20))- math.log10(ref) for i in sorted_rates]
        slow_these = []
        slow_these_rxns = set()
        accounted = set()
        for i in range(len(differences)):
            if differences[i]>settings['Stiffness Threshold']:
                slow_these.append(sorted_rates[i])
                slow_these_rxns.add(sorted_rates[i][0])
        numba = 10000000000000
        for s in red_rxn_reac:
            check = [x in slow_these_rxns for x in red_rxn_reac[s]]
            if all(check):
                rates_check = [real_rates[r][1] for r in red_rxn_reac[s]]
                rate_mult = (10**settings['Stiffness Threshold'])*ref/max(rates_check)
                for r in red_rxn_reac[s]:
                    if r not in accounted:
                        accounted.add(r)
                        reduced_mech.reactions[r].multiplier*= rate_mult
            '''elif any(check):
                pass
                print('checking')
                rates_check = []
                for x in red_rxn_reac[s]:
                    if x not in accounted:
                        rates_check.append(real_rates[x])
                        accounted.add(x)
                sorted_check = sorted(rates_check, key=lambda x: x[1], reverse = True) 
                marks = []
                multer = []
                print(sorted_check)
                for j in range(len(sorted_check)):
                    if sorted_check[j][0] in slow_these_rxns:
                        if j>0 and sorted_check[j][1]/max(1e-20,sorted_check[j-1][1])<1/numba:
                            marks.append(j)
                            min_mult = numba*sorted_check[j][1]/max(1e-20,sorted_check[j-1][1])
                            multer.append(min_mult)
                        print('in slow', sorted_check[j][1])
                    elif j>0 and sorted_check[j][1]/max(1e-20,sorted_check[j-1][1])>1/numba:
                        print('is close', sorted_check[j][1], sorted_check[j][1]/max(1e-20,sorted_check[j-1][1]))
                    else:
                        marks.append(j)
                        min_mult = numba*sorted_check[j][1]/max(1e-20,sorted_check[j-1][1])
                        multer.append(min_mult)
                        print('max mult', min_mult)
                        break
                multer_2 = []
                for i in range(len(multer)):
                    multer_2.append(np.prod(multer[i:]))
                multer = deepcopy(multer_2)
                print('multer', marks, multer)
                count = 0
                if len(multer)>0:
                    for j in range(len(sorted_check)):
                        print(j, count)
                        if j<marks[count]:
                            reduced_mech.reactions[sorted_check[j][0]].multiplier *= multer[count]
                            print('sorty', sorted_check[j][1], multer[count])
                        elif count+1<len(marks):
                            count+=1
                            reduced_mech.reactions[sorted_check[j][0]].multiplier *= multer[count]
                            print('sorty', sorted_check[j][1], multer[count])
                        else:
                            break'''

    
    if settings['Print Progress'] == True:
        print('Stage 13: Removing Reactions')

    cat_species_set = set(cat_species_indices)
    
    avoid_these_rxns = set()
    for i in range(len(reduced_mech.reactions)):
        for j in reduced_mech.reactions[i].reactants:
            if j in red_scc_set:
                if j not in cat_species_set:
                    scc_indy = red_scc_dict_2[j]
                    for k in reduced_mech.reactions[i].prod_dict:
                        if k in red_scc[scc_indy]:
                            avoid_these_rxns.add(i)
    
    
    
    shortened_reactions = []
    reaction_type_dict = {}
    count = 0
    for i in reduced_mech.reactions:
        if settings['Keep Cycle Reactions']==True:
            if count not in avoid_these_rxns:
                if len(i.reactants) == 2:
                    reac_str = str(i.reactants[0]) + ' ' + str(i.reactants[1])
                    if reac_str in reaction_type_dict:
                        reaction_type_dict[reac_str].append(i)
                    else:
                        reaction_type_dict[reac_str] = [i]
                elif 'HV' in i.rate_string:
                    reac_str = str(i.reactants[0]) + ' ' + 'HV'
                    if reac_str in reaction_type_dict:
                        reaction_type_dict[reac_str].append(i)
                    else:
                        reaction_type_dict[reac_str] = [i]
                else:
                    reac_str = str(i.reactants[0]) + ' ' + 'OTH'
                    if reac_str in reaction_type_dict:
                        reaction_type_dict[reac_str].append(i)
                    else:
                        reaction_type_dict[reac_str] = [i]
        else:
            if len(i.reactants) == 2:
                reac_str = str(i.reactants[0]) + ' ' + str(i.reactants[1])
                if reac_str in reaction_type_dict:
                    reaction_type_dict[reac_str].append(i)
                else:
                    reaction_type_dict[reac_str] = [i]
            elif 'HV' in i.rate_string:
                reac_str = str(i.reactants[0]) + ' ' + 'HV'
                if reac_str in reaction_type_dict:
                    reaction_type_dict[reac_str].append(i)
                else:
                    reaction_type_dict[reac_str] = [i]
            else:
                reac_str = str(i.reactants[0]) + ' ' + 'OTH'
                if reac_str in reaction_type_dict:
                    reaction_type_dict[reac_str].append(i)
                else:
                    reaction_type_dict[reac_str] = [i]
        count+=1
    
    for i in reaction_type_dict:
        total_rates = []
        multi = []
        for j in reaction_type_dict[i]:
            if j not in avoid_these_rxns:
                total_rates.append(np.mean(j.rate)*j.multiplier)
                multi.append(j.multiplier)
            
        total_rate = sum(total_rates)
        avg_rate = np.mean(total_rates)
        max_reac = total_rates.index(max(total_rates))
        base_rxn = reaction_type_dict[i][max_reac]
        base_rate = np.mean(reaction_type_dict[i][max_reac].rate)
        
        mult = total_rate/max(base_rate, 1e-20)
        new_prod_dict = {}
        for j in reaction_type_dict[i]:
            factor = np.mean(j.rate)*j.multiplier/max(1e-20,total_rate)
            for k in j.prod_dict:
                if k in new_prod_dict:
                    new_prod_dict[k] += j.prod_dict[k]*factor
                else:
                    new_prod_dict[k] = j.prod_dict[k]*factor
        new_rxn = deepcopy(base_rxn)
        new_rxn.prod_dict = new_prod_dict
        new_rxn.multiplier = mult
        shortened_reactions.append(new_rxn)
        new_name_reactions =[]
    if settings['Keep Cycle Reactions']==True:
        for i in avoid_these_rxns:
            shortened_reactions.append(reduced_mech.reactions[i])

    if settings['Remove Weak Reactions']==True:
        short_rxn_reac = {}
        for i in range(len(shortened_reactions)):
            for j in shortened_reactions[i].reactants:
                if j not in back_set:
                    if j in short_rxn_reac:
                        short_rxn_reac[j].add(i)
                    else:
                        short_rxn_reac[j] = set([i])
    
        include_set = set()
        for i in short_rxn_reac:
            total_rate = 0
            for j in short_rxn_reac[i]:
                total_rate+= np.mean(shortened_reactions[j].rate)*shortened_reactions[j].multiplier
            for j in short_rxn_reac[i]:
                if np.mean(shortened_reactions[j].rate)*shortened_reactions[j].multiplier > total_rate*settings['Weak Reaction Cutoff']:
                    include_set.add(j)
        red_shortened_reactions = [shortened_reactions[i] for i in include_set]
        shortened_reactions = deepcopy(red_shortened_reactions)
    new_name_reactions = []
    if settings['Remove Reactions']== True:
        
        spec_set = set()
        for i in shortened_reactions:
        
            new_reacs = []
            for j in i.reactants:
                if species_list_names_2[j] not in background_spc_n:
                    new_reacs.append(species_list_names_2[j])
                    spec_set.add(species_list_names_2[j])
            for j in i.reactants:
                if species_list_names_2[j] in background_spc_n:
                    new_reacs.append(species_list_names_2[j])
                    spec_set.add(species_list_names_2[j])
            new_prods = {}
            for j in i.prod_dict:
                if i.prod_dict[j]>0.001:
                    new_prods[species_list_names_2[j]] = i.prod_dict[j]
                    spec_set.add(species_list_names_2[j])
            j = deepcopy(i)
            j.prod_dict = new_prods
            j.reactants = new_reacs
            if len(new_prods)>0:
                new_name_reactions.append(j)
    else:
        spec_set = set()
        for i in reduced_mech.reactions:
        
            new_reacs = []
            for j in i.reactants:
                if species_list_names_2[j] not in background_spc_n:
                    new_reacs.append(species_list_names_2[j])
                    spec_set.add(species_list_names_2[j])
            for j in i.reactants:
                if species_list_names_2[j] in background_spc_n:
                    new_reacs.append(species_list_names_2[j])
                    spec_set.add(species_list_names_2[j])
            new_prods = {}
            for j in i.prod_dict:
                if i.prod_dict[j]>0.001:
                    new_prods[species_list_names_2[j]] = i.prod_dict[j]
                    spec_set.add(species_list_names_2[j])
            j = deepcopy(i)
            j.prod_dict = new_prods
            j.reactants = new_reacs
            if len(new_prods)>0:
                new_name_reactions.append(j)
 
    reduced_mech.reactions = new_name_reactions
    print('Reduction Complete!')

    return reduced_mech, categories, test_mechanism, remaining_specs, avg_yield, yields, graphs, red_yields, red_graphs, new_graph, scc, red_scc, groups, species_list, dic, tiers, dag_graph_save, dag_graph_save_2, inv_map_dict, mult_list_list, cat_data, multy, ordered_species, avg_yield_o, avg_yield_o2
    

    
                

In [7]:
import AMORE_v2 as am

# Isoprene

### Isoprene Inputs

In [11]:
#eq_list = read_eqns('/Users/woods/OneDrive/Documents/Python/Code/amore_apinene_full_eqn.txt')


In [12]:
#background rxns for f0am conversion
eq_list_addition = [('GLYX + hv = CO + CO + H2','J31'),
('GLYX + hv = CO + CO + HO2 + HO2','J33'),
('GLYX + hv = HCHO + CO','J32'),
('NO3 + GLYX = HCOCO + HNO3','KNO3AL'),
('OH + GLYX = HCOCO','3.1e-12.*exp(340./T)'),
('MGLY + hv = CH3CO3 + CO + HO2','J34'),
('NO3 + MGLY = CH3CO3 + CO + HNO3','KNO3AL.*2.4'),
('OH + MGLY = CH3CO3 + CO','1.9e-12.*exp(575./T)'),
('GLYC + NO3 = HOCH2CO3 + HNO3','KNO3AL'),
('GLYC + OH = GLYX + HO2','1.00e-11.*0.200'),
('GLYC + OH = HOCH2CO3','1.00e-11.*0.800'),
('GLYC + hv = HO2 + HCHO + HO2 + CO','J15'),
('HCHO + hv = CO + HO2 + HO2','J11'),
('HCHO + hv = H2 + CO','J12'),
('NO3 + HCHO = HNO3 + CO + HO2','5.5e-16'),
('OH + HCHO = HO2 + CO','5.4e-12.*exp(135./T)'),
('CO + OH = CO2 + HO2','K_OH_CO'), 
('O = O3','5.6e-34.*.78.*M.*(T./300).^-2.6.*.21.*M'),
('O = O3','6.0e-34.*.21.*M.*(T./300).^-2.6.*.21.*M'),
('O + O3 = ','8.0e-12.*exp(-2060./T)'),
('O + NO = NO2','KMT01'),
('O + NO2 = NO','5.5e-12.*exp(188./T)'),
('O + NO2 = NO3','KMT02'),
('O1D = O','3.2e-11.*exp(67./T).*.21.*M'),
('O1D = O','2.0e-11.*exp(130./T).*.78.*M'),
('NO + O3 = NO2','1.4e-12.*exp(-1310./T)'),
('NO2 + O3 = NO3','1.4e-13.*exp(-2470./T)'),
('NO + NO = NO2 + NO2','3.3e-39.*exp(530./T).*.21.*M'),
('NO + NO3 = NO2 + NO2','1.8e-11.*exp(110./T)'),
('NO2 + NO3 = NO + NO2','4.50e-14.*exp(-1260./T)'),
('NO2 + NO3 = N2O5','KMT03'),
('O1D = OH + OH','2.14e-10.*H2O'),
('OH + O3 = HO2','1.70e-12.*exp(-940./T)'),
('OH + H2 = HO2','7.7e-12.*exp(-2100./T)'),
('OH + CO = HO2','KMT05'),
('OH + H2O2 = HO2','2.9e-12.*exp(-160./T)'),
('HO2 + O3 = OH','2.03e-16.*(T./300).^4.57.*exp(693./T)'),
('OH + HO2 = ','4.8e-11.*exp(250./T)'),
('HO2 + HO2 = H2O2','2.20e-13.*KMT06.*exp(600./T)'),
('HO2 + HO2 = H2O2','1.90e-33.*M.*KMT06.*exp(980./T)'),
('OH + NO = HONO','KMT07'),
('OH + NO2 = HNO3','KMT08'),
('OH + NO3 = HO2 + NO2','2.0e-11'),
('HO2 + NO = OH + NO2','3.45e-12.*exp(270./T)'),
('HO2 + NO2 = HO2NO2','KMT09'),
('OH + HO2NO2 = NO2','3.2e-13.*exp(690./T).*1.0'),
('HO2 + NO3 = OH + NO2','4.0e-12'),
('OH + HONO = NO2','2.5e-12.*exp(260./T)'),
('OH + HNO3 = NO3','KMT11'),
('O + SO2 = SO3','4.0e-32.*exp(-1000./T).*M'),
('OH + SO2 = HSO3','KMT12'),
('HSO3 = HO2 + SO3','1.3e-12.*exp(-330./T).*.21.*M'),
('HNO3 = NA','6.00e-06'),
('N2O5 = NA + NA','4.00e-04'),
('SO3 = SA','1.20e-15.*H2O'),
('O3 + hv = O1D','J1'),
('O3 + hv = O','J2'),
('H2O2 + hv = OH + OH','SUN*2.15e-6'),
('NO2 + hv = NO + O','J4'),
('NO3 + hv = NO','J5'),
('NO3 + hv = NO2 + O','J6'),
('HONO + hv = OH + NO','J7'),
('HNO3 + hv = OH + NO2','J8'),
('N2O5 = NO2 + NO3','KMT04'),
('HO2NO2 = HO2 + NO2','KMT10'),
('CH3OO + HO2 = CH3OOH','3.8e-13.*exp(780./T).*(1-1./(1+498.*exp(-1160./T)))'),
('CH3OO + HO2 = HCHO','3.8e-13.*exp(780./T).*(1./(1+498.*exp(-1160./T)))'),
('CH3OO + NO = CH3NO3','2.3e-12.*exp(360./T).*0.001'),
('CH3OO + NO = CH3O + NO2','2.3e-12.*exp(360./T).*0.999'),
('CH3OO + NO2 = CH3OONO2','KMT13'),
('CH3OO + NO3 = CH3O + NO2','1.2e-12'),
('CH3OO + CH3OO = CH3O + CH3O','2.*1.03e-13.*exp(365./T).*7.18.*exp(-885./T)'),
('CH3OO + CH3OO = CH3OH + HCHO','2.*1.03e-13.*exp(365./T).*(1-7.18.*exp(-885./T))'),
('CH3OOH + hv = CH3O + OH','J41'),
('OH + CH3OOH = CH3OO','5.3e-12.*exp(190./T).*0.6'),
('OH + CH3OOH = HCHO + OH','5.3e-12.*exp(190./T).*0.4'),
('CH3NO3 + hv = CH3O + NO2','J51'),
('OH + CH3NO3 = HCHO + NO2','4.0e-13.*exp(-845./T)'),
('CH3O = HCHO + HO2','7.2e-14.*exp(-1080./T).*.21.*M'),
('CH3OONO2 = CH3OO + NO2','KMT14'),
('CH3OH + OH = HO2 + HCHO','2.85e-12.*exp(-345./T)'),
('CH3OO = CH3O','2.*KCH3O2.*0.1.*7.18.*exp(-885./T)'),
('CH3OO = CH3OH','2.*KCH3O2.*0.1.*0.5.*(1-7.18.*exp(-885./T))'),
('CH3OO = HCHO','2.*KCH3O2.*0.1.*0.5.*(1-7.18.*exp(-885./T))')]

In [13]:
#background rxn processing for f0am conversion
nums = ['0','1','2','3','4','5','6','7','8','9']
reac_list_a = [i[0].split(' = ')[0].split(' + ') for i in eq_list_addition]
prod_list_a = []
for i in eq_list_addition: 
    if i[0].split(' = ')[1]!='':
        prod_list_a.append(i[0].split(' = ')[1].split(' + '))
    else:
        prod_list_a.append([])
prod_list2_a = deepcopy(prod_list_a)
prod_coeff_list_a = deepcopy(prod_list_a)
reac_list2_a = deepcopy(reac_list_a)
reac_coeff_list_a = deepcopy(reac_list_a)
#splitting up prod from coeff using numericals. a is just a dummy variable.
a=[]
for i in range(len(prod_list2_a)):
    for j in range(len(prod_list2_a[i])):
        if prod_list2_a[i][j][:1] in nums:
            a=re.split('([a-zA-Z])',prod_list_a[i][j],1)
            prod_list2_a[i][j]=a[1]+a[2]
            prod_coeff_list_a[i][j]=a[0]
        else:
            prod_coeff_list_a[i][j]=1
b=[]
for i in range(len(reac_list2_a)):
    for j in range(len(reac_list2_a[i])):
        if reac_list2_a[i][j][:1] in nums:
            b=re.split('([a-zA-Z])',reac_list_a[i][j],1)
            reac_list2_a[i][j]=b[1]+b[2]
            reac_coeff_list_a[i][j]=b[0]
        else:
            reac_coeff_list_a[i][j]=1
for i in range(len(prod_coeff_list_a)):
    for j in range(len(prod_coeff_list_a[i])):
        prod_coeff_list_a[i][j] = float(prod_coeff_list_a[i][j])
for i in range(len(reac_coeff_list_a)):
    for j in range(len(reac_coeff_list_a[i])):
        reac_coeff_list_a[i][j] = float(reac_coeff_list_a[i][j])
#prod_list_a = deepcopy(prod_list2_a)
#reac_list_a = deepcopy(reac_list2_a)

In [14]:
eq_list = am.read_eqns('/Users/woods/OneDrive/Documents/Supporting Information/Caltech Full Mechanism (AMORE update)/caltech_amore_isoprene_full_eqn.txt')

In [15]:
species_list_names = am.read_spc('/Users/woods/OneDrive/Documents/Supporting Information/Caltech Full Mechanism (AMORE update)/caltech_amore_isoprene_full_spc.txt')  

In [16]:
#eq_list2 = read_eqns("C:/Users/woods/iCloudDrive/Documents/Documents - Woods MacBook Air/Jobs/Columbia/McNeill Lab/Woods_AMORE_code/isoprene_oxidation_model_v5_190415/isoprene_full_v5_eqn.txt")

In [17]:
species_list2 = am.read_spc('/Users/woods/OneDrive/Documents/Supporting Information/Caltech Full Mechanism (AMORE update)/caltech_amore_isoprene_full_spc.txt')  

In [18]:
species_list_names.append('MGLYOX')

In [19]:
species_list = list(range(len(species_list_names)))

In [20]:
spec_len = len(species_list)


In [21]:
len(species_list_names)

430

In [22]:
rate_list = [i[1] for i in eq_list]
rate_list_clean = [i[1] for i in eq_list]
#k_list = get_k_list(rate_list, )

In [23]:
#background_spc
#a list of background species to be excluded from our reaction network for visual clarity (and also because they will mess with the drg reduction method)
background_spc_0 = ['OH','NO','NO2','NO3','HO2','O3','CH3OO','CO2','CO','O2','N2O5','H2O','H2O2','H2Od','SO2',
                  'H2S','HF','H2S','N2H4','HN3','HI','HBr',
                  'HCl','HCN','H2Se','H2Te','NH2OH','HBrO','HClO','H3PO2','HPO3',
                  'H2O3','OF2','O2F2','NOHSO4','COS','N2F4','N2O4','N2O3','HNO3',
                  'HNO2','N2O','NF5','NI3','H2SO4','CS2','H2CO3','H2SO3','SO2Cl2','S4N4','H2SO5',
                  'H2S2O7','S2F10','H3NO3S','Br2S','SF6','SF4', 'CH3CO3']
background_spc = []
background_spc_n = []
for i in background_spc_0:
    if i in species_list_names:
        background_spc_n.append(i)
        background_spc.append(species_list_names.index(i))

In [24]:
reac_list_n,reac_coeff_list,prod_list_n,prod_coeff_list = am.get_prod_reac(eq_list)

In [25]:
dic = {species_list_names[i]:i for i in range(spec_len)}

In [26]:
reaction_list = []
for i in range(len(reac_list_n)):
    r = reac_list_n[i]
    p = {}
    for j in range(len(prod_list_n[i])):
        if prod_list_n[i][j] not in p:
            p[prod_list_n[i][j]] = prod_coeff_list[i][j]
        else:
            p[prod_list_n[i][j]] += prod_coeff_list[i][j]
    k1 = eq_list[i][1]
    k2 = 0 
    k3 = 1
    if any([x not in background_spc_n for x in r]):
        reaction_list.append(am.Reaction(r,p,k1,k2,k3))
    else:
        print(r)

['HO2', 'CH3CO3']
['NO', 'CH3CO3']
['NO2', 'CH3CO3']
['NO3', 'CH3CO3']
['CH3OO', 'CH3CO3']


In [27]:
isop_mech = am.Mechanism(species_list_names, reaction_list)

In [28]:
replacement_dic = {'ALK(':'ALK(TEMP, M,', 'NIT(':'NIT(TEMP, M,', 'ISO1(':'ISO1(TEMP,','TUN(':'TUN(TEMP,','ISO2(':'ISO2(TEMP,','EPO(':'EPO(TEMP,','FALL(':'FALL(TEMP,','TROE(':'TROE(TEMP, M,','ARR(':'ARR(TEMP,'}

In [29]:
replacement_set = set(replacement_dic.keys())

In [30]:
for i in range(len(isop_mech.reactions)):
    for j in replacement_set:
        if j in isop_mech.reactions[i].rate_law:
            print('hi')
            rate_string = deepcopy(isop_mech.reactions[i].rate_law)
            rate_string = rate_string.replace(j,replacement_dic[j])
            print(rate_string)
            isop_mech.reactions[i].rate_law = deepcopy(rate_string)

            print(isop_mech.reactions[i].rate_law)

hi
ALK(TEMP, M,2.7E-12,350.,1.190,6.,1.,0.)
ALK(TEMP, M,2.7E-12,350.,1.190,6.,1.,0.)
hi
NIT(TEMP, M,2.7E-12,350.,1.190,6.,1.,0.)
NIT(TEMP, M,2.7E-12,350.,1.190,6.,1.,0.)
hi
ALK(TEMP, M,1.215E-12,350.,1.421,6.,1.,0.)
ALK(TEMP, M,1.215E-12,350.,1.421,6.,1.,0.)
hi
ALK(TEMP, M,1.485E-12,350.,1.421,6.,1.,0.)
ALK(TEMP, M,1.485E-12,350.,1.421,6.,1.,0.)
hi
NIT(TEMP, M,2.7E-12,350.,1.421,6.,1.,0.)
NIT(TEMP, M,2.7E-12,350.,1.421,6.,1.,0.)
hi
ALK(TEMP, M,1.215E-12,350.,1.421,6.,1.,0.)
ALK(TEMP, M,1.215E-12,350.,1.421,6.,1.,0.)
hi
ALK(TEMP, M,1.485E-12,350.,1.421,6.,1.,0.)
ALK(TEMP, M,1.485E-12,350.,1.421,6.,1.,0.)
hi
NIT(TEMP, M,2.7E-12,350.,1.421,6.,1.,0.)
NIT(TEMP, M,2.7E-12,350.,1.421,6.,1.,0.)
hi
TUN(TEMP,4.21E13,8.93E3,1.55E8)
TUN(TEMP,4.21E13,8.93E3,1.55E8)
hi
TUN(TEMP,5.45E13,9.45E3,1.55E8)
TUN(TEMP,5.45E13,9.45E3,1.55E8)
hi
ALK(TEMP, M,2.7E-12,350.,28.237,9.,1.,0.)
ALK(TEMP, M,2.7E-12,350.,28.237,9.,1.,0.)
hi
NIT(TEMP, M,2.7E-12,350.,28.237,9.,1.,0.)
NIT(TEMP, M,2.7E-12,350.,28.237,9.,1.,

In [31]:
isopnit_noihn = ["ISOP1CO2N3OOH4OH", "ISOP1CO2OOH3N4OH", "ISOP1OH2OOH3N4CO", "ISOP1OH2N3OOH4CO", "ISOP1OH2N3CO4OH", "ISOP1CO2N3OOH4OOH", "ISOP1CO2OOH3N4OOH", "ISOP1CO2OOH3OOH4N", "ISOP1OOH2OOH3N4CO", "ISOP1OOH2N3OOH4CO", "ISOP1N2OOH3OOH4CO", "ISOP1OOH23O3OH4N", "ISOP1N2OOH34O4OH", "ISOP1OH12O3OOH4N", "ISOP1OH2OOH3OH4N", "ISOP1OH2N3OH4OOH", "ISOP1N2OH3OOH4OH", "ISOP1OOH2OH3N4OH", "ISOP1OH2OOH3N4OH", "ISOP1OH2N3OOH4OH", "ISOP1N2OOH3OH4CO", "ISOP1CO2OH3OOH4N", "ISOP1OOH2OH3CO4N", "ISOP1N4R4OH", "ISOP1OH1R4N", "ISOP3OH3R4N", "ISOP1OOH2R3CO4N", "ISOP1OH2OH3N4CO", "ISOP1CO2N3OH4OH", "ISOP1OH2N3R4OH", "ISOP1OH2N3OH4N", "ISOP1OH2N3N4OH", "ISOP1CO2N3OH4OOH", "ISOP1OH2R3N4OH", "ISOP1N2OH3N4OH", "ISOP1OOH2OH3N4CO", "ISOP1N2OOH", "ISOP1N4OOH", "ISOP3OOH4N", "ISOP1OOH4N", "ISOP3CO4N", "ISOP1CO4N", "ISOP1O4N", "ISOP1N4CO", "ISOP1N4O", "ISOP1N2N", "ISOP1N4N", "ISOP3N4N", "ISOP1N253O4OH", "ISOP1N253N4OH", "ISOP1N253OOH4OH", "ISOP1N253CO4OH", "ISOP1N253OH4OH", "ISOP1N2OOH3R4OH", "ISOP1N23O4OH", "ISOP1N2OH3R4OOH", "ISOP1N2OH34O", "ISOP1N2R3OH4OOH", "ISOP1OH2R3OOH4N", "ISOP1OH23O4N", "ISOP1OOH2R3OH4N", "ISOP12O3OH4N", "ISOP1OOH2OH3R4N", "ISOP1N2OOH3N4OH", "ISOP1N2OOH3OOH4OH", "ISOP1N2OOH3OH4N", "ISOP1N2OOH3OH4OOH", "ISOP1N2OH3N4OOH", "ISOP1N2OH3OOH4OOH", "ISOP1N2N3OH4OOH", "ISOP1OH2N3OOH4N", "ISOP1OH2OOH3OOH4N", "ISOP1N2OH3OOH4N", "ISOP1OOH2OH3OOH4N", "ISOP1OOH2N3OH4N", "ISOP1OOH2OOH3OH4N", "ISOP1OO2OOH3OH4N", "ISOP1N2N3OOH4OH", "ISOP1OH2OOH3N4N", "ISOP1N23O4OH4R", "ISOP1N23O4CO", "ISOP1OH1R23O4N", "ISOP1CO23O4N", "ISOP1N2OH3CO4OOH", "ISOP12O3OH3R4N", "ISOP12O3CO4N", "ISOP1N2OH3CO4N", "ISOP1OH2OOH3CO4N", "ISOP1N4R4CO", "ISOP1N4CO4OOH", "ISOP1N4CO4OH", "ISOP1CO1R4N", "ISOP1CO1OOH4N", "ISOP1CO1OH4N", "ISOP1N2R3OH4CO", "ISOP1CO2OH3R4N", "ISOP1N2OH3OOH4CO", "ISOP1CO2OOH3OH4N", "ISOP1N2R3OH4OH", "ISOP1OH2OH3R4N", "ISOP1N2OOH3OH4OH", "ISOP1OH2OH3OOH4N", "ISOP1N2OH3OH4OOH", "ISOP1OOH2OH3OH4N", "ISOP1OOH2OH3N4N", "ISOP1OH2N3CO4N", "ISOP1N2N3CO4OH", "ISOP1OH2OH3N4N", "ISOP1N2OH3OH4N", "ISOP1N2N3OH4OH", "ISOP1CO2N3OH4N", "ISOP1N2OH3N4CO", "ISOP1N4PAN", "ISOP1PAN4N", "ISOP1PAN4OH", "ISOP1OH4PAN"]

#### New Categories

In [33]:
isop_nit_tetra = ['ISOPN','ISOP1OH2N3OH4OOH','ISOP1CO23O4N','ISOP1N2OH3OOH4N','ISOP1OOH2OH3CO4N','IHNPE','ISOP1N23O4OH','ISOP1N2OH3N4OH','ISOP1N253N4OH','ISOP1OH2N3N4OH','ISOP1CO2N3OH4N','ISOP1N2OOH3OOH4CO','ISOP1OOH2OH3OOH4N','ISOP1OH2N3OOH4CO','ISOP1OOH2OH3N4CO','ISOP1OH12O3OOH4N','ISOP1OOH2N3OOH4CO','ISOP1OOH23O3OH4N','ISOP1OOH2OH3OH4N','ISOP1N2OOH3OH4OH','ISOP1OH2OH3N4N','ISOP1OH2N3CO4N','ISOP1OH2N3OH4N','ISOP1OOH2OOH3N4CO','ISOP1CO2OH3OOH4N','ISOP12O3OH4N','ISOP1N23O4CO','ISOP1OH2N3OOH4N','ISOP1N2OH3OOH4OH','ISOP1N2OH3CO4N','ISOP1OOH2OH3N4N','ISOP1OH2OOH3OOH4N','ISOP1OH23O4N','ISOP1CO2N3OOH4OOH','ISOP1CO2N3OH4OH','ISOP1N2OH3OOH4OOH','ITHN','ISOP1N2N3OOH4OH','ISOP1N2OH3N4OOH','ISOP1OOH2OH3N4OH','ISOP1N2OOH34O4OH','ISOP1OH2OH3OOH4N','IDHCN','ISOP1OH2N3OOH4OH','ISOP1N2OOH3N4OH','ISOP1N2OH3N4CO','ISOP1N2OH3OH4N','ISOP1CO2OOH3OH4N','IDHDN','ISOP1N2N3OH4OH','ISOP1N2N3OH4OOH','IHNDP','ISOP1N2OOH3OH4CO','ICHNP','ISOP1OOH2N3OH4N','ISOP1CO2N3OOH4OH','ISOP1OH2N3CO4OH','IHPDN','ISOP1OOH2OOH3OH4N','ISOP1CO2OOH3N4OH','ISOP1N2OH34O','ISOP12O3CO4N','ISOP1CO2N3OH4OOH','ISOP1N2N3CO4OH','ISOP1CO2OOH3N4OOH','ISOP1OH2OOH3CO4N','ISOP1N2OH3OH4OOH','ICNE','ISOP1N2OH3CO4OOH','IDHPN','ICHDN','ISOP1CO2OOH3OOH4N','ISOP1OH2OOH3N4OH','ISOP1N2OOH3OH4OOH','ISOP1OH2OOH3OH4N','ISOP1OH2OOH3N4CO','ISOP1N2OOH3OH4N','ISOP1OH2OOH3N4N','ISOP1N2OH3OOH4CO','ISOP1OH2OH3N4CO','ITCN','IHNE','ISOP1N2OOH3OOH4OH']

tetra = ['TETRA','ISOP1OH23O4CO','ISOP1OH12O3OOH4OH','ISOP1OH12O3OH4OOH','ISOP1CO2OH3OOH4CO','ISOP1OOH2OH3CO4OH','ISOP1CO2OOH34O4OH','ICHE','ISOP1CO2OOH3OOH4OOH','ISOP1OH12O3OOH4CO','ICPDH','ISOP1OH2OOH3CO4OH','ISOP1OOH2OH34O4OH','ISOP12O3OH4CO','ISOP1OH2OOH3CO4CO','ISOP1OOH2OH34O','ISOP1OOH2OH3CO4CO','IDCHP','ISOP1OOH23O4CO','ISOP1CO23O4OOH','ISOP1CO2OOH3OH4OH','ISOP1OH2OH3OOH4CO','ISOP12O3CO4OH','ICHDN','ISOP1CO2OH3OOH4OH','ISOP1CO2OOH3OOH4N','ISOP1OH2OOH3OH4OOH','ISOP1CO2OOH3OH4CO','ISOP1CO2OOH3OOH4OH','ISOP1OH2OOH3OOH4OH','ISOP1OOH2OOH3CO4CO','ISOP1OH2OOH3OOH4CO','IDHPE','ISOP1OOH2OOH3OOH4CO','ISOP1CO2OOH3OOH4CO','ISOP1OOH2OH3OOH4OH','IHPDN','IDHDP','ISOP12O3OH4OOH','ISOP1OH2OOH34O4OH','ISOP1CO23O4OH','ISOP1OH2OOH3OH4CO','ISOP1CO2OH34O']
ihn_plus = ['IHN','ISOP1OOH4N','ISOP3N4N','IHNB','ICN','ISOP1N4OOH','ISOP1OH4Nc','ISOP3N4OH','ISOP1CO4N','ISOP3CO4N','IDN','ISOP1OH2N','ISOP1N4CO','ISOP3OH4N','ISOP1OH4N','ISOP1N2OOH','ISOP1N4N','ISOP1PAN4OH','ISOP1N4OH','INPD','ISOP1N2OH','ISOP1N2N','ISOP1N4OHt','ISOP1OH4PAN','ISOP1N4PAN','ISOP1PAN4N','ISOP1N4OHc','ISOP1OH4Nt','INPB','ISOP3OOH4N']

In [34]:
new_nit_tet = []
for i in isop_nit_tetra:
    if i in species_list_names:
        new_nit_tet.append(i)
new_tet = []
for i in tetra:
    if i in species_list_names:
        new_tet.append(i)
new_ihn = []
for i in ihn_plus:
    if i in species_list_names:
        new_ihn.append(i)
isop_nit_tetra = deepcopy(new_nit_tet)
tetra = deepcopy(new_tet)
ihn_plus = deepcopy(new_ihn)

#### Rest

In [36]:
protected = deepcopy(background_spc)
for i in ['ISOP','ISOP1OH23O4OHt','ISOP1OH23O4OHc', 'ISOP1OH2OH34O', 'MVK','HCHO', 'HAC','PAN','GLYC','PYRAC','MGLY','HCOOH','MACR','MPAN','GLYX']:
    protected.append(isop_mech.species.index(i))

In [37]:
settings = {}
settings['roots'] = ['ISOP']
settings['Weak rxn cutoff'] = 0.000000
settings['Remove weak reactions'] = True
settings['Categories'] = [['TETRA',tetra], ['ISOPN', isop_nit_tetra], ['IHN', ihn_plus]]
settings['Protected'] = ['ISOP','ISOP1OH23O4OHt','ISOP1OH23O4OHc', 'ISOP1OH2OH34O','ISOP12O3OH4OH', 'MVK','HCHO', 'HAC','PAN','GLYC','PYRAC','MGLY','HCOOH','MACR','MPAN','GLYX','HMML']
settings['Aerosol Rxns'] = []
settings['Background Rxns'] = []

In [38]:
settings['Protected'] = ['ISOP','ISOP1OH23O4OHt','ISOP1OH23O4OHc', 'ISOP1OH2OH34O','ISOP12O3OH4OH', 'MVK','HCHO', 'HAC','PAN','GLYC','PYRAC','MGLY','HCOOH','MACR','MPAN','GLYX','HMML']


In [39]:
settings['Categories'] = [['TETRA',tetra], ['ISOPN', isop_nit_tetra], ['IHN', ihn_plus]]


### Conditions

In [41]:
def arrhenius(A,n,Ea,T):
    rate = A*(T**n)*math.exp(-Ea/T)
    return rate
def falloff(a1,a2,a3,f1,f2,f3,f4,f5,f6,f7,temp,m):
    e1 = f1*((temp/300)**f2)*math.exp(-f3/temp)*m
    r1 = e1/(a1*((temp/300)**a2))*math.exp(-a3/temp)
    fr1 = 1/(1+((math.log10(r1))**2))
    if f5 == 0 or f6 == 0:
        lnrate = math.log(e1/(1+r1)) + fr1*math.log(f4)
    else:
        fr2 = (1-f4)*math.exp(-temp/f5) + f4*math.exp(-temp/f6) + math.exp(-f7/temp)
        if f7== 0:
            fr2 = fr2 -1
        lnrate = math.log(e1/(1+r1)) + fr1*math.log(fr2)
    return math.exp(lnrate)
'''def get_rates(rate_numbers, rate_type,a_list_2,b_list_3,temp,press,M,sza):
    rates = []
    for i in range(len(rate_numbers)):
        if rate_type[i] == 'WALL' or rate_type[i] == 'EXTRA':
            rates.append(0)
        elif rate_type[i] == 'HV':
            ind = a_list_2.index(rate_numbers[i][1][0])
            ind2 = b_list_3[ind].index(sza)
            r = b_list_3[ind][ind2+1]*rate_numbers[i][1][1]
            rates.append(r)
        elif rate_type[i] == 'FALLOFF':
            rates.append(falloff(rate_numbers[i][0][0],rate_numbers[i][0][1],rate_numbers[i][0][2],rate_numbers[i][1][0],rate_numbers[i][1][1],rate_numbers[i][1][2],rate_numbers[i][1][3],rate_numbers[i][1][4],rate_numbers[i][1][5],rate_numbers[i][1][6],temp,M))
        elif rate_type[i] == 'NORMAL':
            r = arrhenius(rate_numbers[i][0],rate_numbers[i][1],rate_numbers[i][2],temp)
            rates.append(r)
        else:
            rates.append(0)
    return rates'''
CFACTOR = 0#2.5e+19;
RO2 = 0.01*10e-9;
SUN = 0.5;
TEMP = 295;
M = am.pressure_to_m(1000,295);
sza = 0;

In [42]:
def_cond_dict = {'OH':1, 'HO2':1, 'pressure':1000,'temp':295,'sza':0,'sun':1, 'ISOP':1}

In [43]:
for i in {'CH4','HNO2','HNO3','ISOP1OH2OO3OH4OOH', 'ISOP1N4OO', 'HOCH2COCHO', 'ISOP1OH2OOH3OH4OO', 'H2Od', 'NO3', 'ISOP1N4O', 'ISOP1OO4N', 'ISOP1OH4OOt', 'CH3CO3', 'ISOP1N2OH3OO4OH', 'HCOOH', 'MACR2OH3N', 'ISOP1OH4OOc', 'SO2', 'ISOP1OO4OHc', 'NO2', 'MACR2N3OH', 'ISOP1OH2OO3CO4OH', 'ISOP1N2OO3OH4OH', 'ETHLN', 'ISOP1N2OO', 'ISOP1OO4OHt', 'ISOP3OO4OH', 'NO', 'O2', 'O3', 'H2O', 'ISOP3OO4N', 'ISOP1OH2OO', 'CH3OO','PERO2', 'H2O2', 'PERO3', 'NO3', 'EXTRA', 'HO', 'MEPERO', 'H', 'FURON', 'CO', 'CO2', 'SO2', 'HV', 'PERO1', 'NO2', 'CH3O2', 'HO2', 'HNO2', 'NO', 'MALAHY', 'O2', 'O3', 'H2', 'PERO9', 'OXYGEN', 'PERO6', 'PERO5'}:
    def_cond_dict[i] = 0

In [44]:
def_cond_dict['ISOP1OH2OO3OH4OOH'] = 7.17554085621398e-06/2	
def_cond_dict['ISOP1N4OO'] = 5e-5/2
def_cond_dict['HOCH2COCHO'] = 0.0020581917381866/2
def_cond_dict['ISOP1OH2OOH3OH4OO'] = 4.10637313603636e-06/2
def_cond_dict['ISOP1N4O'] = 3.68808775192020e-58/2
def_cond_dict['ISOP1OO4N'] = 1e-5/2
def_cond_dict['ISOP1OH4OOt'] = 7.39265328797610e-05/2
def_cond_dict['ISOP1N2OH3OO4OH'] = 1.92960943798541e-07/2
def_cond_dict['MACR2OH3N'] = 4.97424761756957e-09/2
def_cond_dict['ISOP1OH4OOc'] = 3.65328929974487e-06/2
def_cond_dict['ISOP1OO4OHc'] = 9.69261827506265e-07/2
def_cond_dict['MACR2N3OH'] = 7.66047361804737e-06/2
def_cond_dict['ISOP1OH2OO3CO4OH'] = 6.41021238985632e-06/2
def_cond_dict['ISOP1N2OO3OH4OH'] = 2.09039993272612e-07/2
def_cond_dict['ETHLN'] = 0.000581804550193117/2
def_cond_dict['ISOP1N2OO'] = 5e-5/2
def_cond_dict['ISOP1OO4OHt'] = 1.94366247386168e-05/2
def_cond_dict['ISOP3OO4OH'] = 0.000346753677764705/2
def_cond_dict['ISOP3OO4N'] = 5e-6/2
def_cond_dict['ISOP1OH2OO'] = 0.00166812115663153/2

In [45]:
conditions = [deepcopy(def_cond_dict),deepcopy(def_cond_dict),deepcopy(def_cond_dict),deepcopy(def_cond_dict),deepcopy(def_cond_dict),deepcopy(def_cond_dict)
        ]

In [47]:
#input conditions
conditions[0]['OH'] = 0.0002
conditions[0]['HO2'] = 7.00E-03
conditions[0]['NO'] = 0.08
conditions[0]['NO2'] = 5
conditions[0]['O3'] = 20
conditions[0]['NO3'] = 0.007
conditions[0]['CH3OO'] = 0.001
conditions[0]['CH3CO3'] = 0.001
conditions[0]['O2'] = 210000000
conditions[0]['sza'] = 0
conditions[0]['sun'] = 1

#low nox
conditions[1]['OH'] = 0.0002
conditions[1]['HO2'] = 7.00E-03
conditions[1]['NO'] = 0.02
conditions[0]['NO2'] = 0.5
conditions[1]['O3'] = 0
conditions[1]['NO3'] = 0.000
conditions[1]['CH3OO'] = 0.000
conditions[1]['CH3CO3'] = 0.000
conditions[1]['O2'] = 210000000
conditions[1]['sza'] = 0
conditions[1]['sun'] = 1

#high nox
conditions[2]['OH'] = 0.0002
conditions[2]['HO2'] = 7.00E-03
conditions[2]['NO'] = 0.2
conditions[0]['NO2'] = 20
conditions[2]['O3'] = 0
conditions[2]['NO3'] = 0.000
conditions[2]['CH3OO'] = 0.000
conditions[2]['CH3CO3'] = 0.000
conditions[2]['O2'] = 210000000
conditions[2]['sza'] = 0
conditions[2]['sun'] = 1

#high o3
conditions[3]['OH'] = 0.0001
conditions[3]['HO2'] = 7.00E-03
conditions[3]['NO'] = 0.03
conditions[0]['NO2'] = 10
conditions[3]['O3'] = 80
conditions[3]['NO3'] = 0.000
conditions[3]['CH3OO'] = 0.001
conditions[3]['CH3CO3'] = 0.001
conditions[3]['O2'] = 210000000
conditions[3]['sza'] = 0
conditions[3]['sun'] = 1

#high no3 low hv
conditions[4]['OH'] = 0.00005
conditions[4]['HO2'] = 3.00E-03
conditions[4]['NO'] = 0.08
conditions[0]['NO2'] = 5
conditions[4]['O3'] = 0
conditions[4]['NO3'] = 0.007
conditions[4]['CH3OO'] = 0.001
conditions[4]['CH3CO3'] = 0.001
conditions[4]['O2'] = 210000000
conditions[4]['sza'] = 90
conditions[4]['sun'] = 0
# low hv
conditions[5]['OH'] = 0.00005
conditions[5]['HO2'] = 3.00E-03
conditions[5]['NO'] = 0.08
conditions[0]['NO2'] = 2
conditions[5]['O3'] = 20
conditions[5]['NO3'] = 0.000
conditions[5]['CH3OO'] = 0.001
conditions[5]['CH3CO3'] = 0.001
conditions[5]['O2'] = 210000000
conditions[5]['sza'] = 90
conditions[5]['sun'] = 0

### Running the Algorithm

Large Mechanism (60+ species)

In [51]:
settings['Reduce Stiffness'] = True

In [53]:
settings['Protected'] = ['ISOP','ISOP1OH23O4OHt','ISOP1OH2OOH','ISOP1OH2OO','ISOP3OO4OH','ISOP1OHt','ISOP1OHc','ISOP4OHt','ISOP4OHc','MVK','HCHO', 'HAC','PAN','GLYC','PYRAC','MGLY','HCOOH','MACR','MPAN','GLYX','HMML']

In [56]:
settings['No Group'] = ['ISOP', 'MVK','HCHO', 'HAC','PAN','GLYC','PYRAC','MGLY','HCOOH','MACR','MPAN','GLYX', 'ISOP1OH23O4OHt','ISOP1OH23O4OHc', 'ISOP1OH2OH34O']

In [89]:
settings['Iterations'] = 100

Small Mechanism (60-39 species)

In [94]:
settings['Reduce Stiffness'] = True

In [95]:
settings['Protected'] = ['ISOP','ISOP1OH23O4OHt','ISOP1OH2OOH','ISOP1OH2OO','ISOP3OO4OH','ISOP1OHt','ISOP1OHc','ISOP4OHt','ISOP4OHc','MVK','HCHO', 'HAC','PAN','GLYC','PYRAC','MGLY','HCOOH','MACR','MPAN','GLYX','HMML']

In [97]:
settings['Manual Groups'] = [['ISOP1OH23O4OHt','ISOP1OH23O4OHc', 'ISOP1OH2OH34O']]

In [101]:
settings['No Group'] = ['ISOP', 'MVK','HCHO', 'HAC','PAN','GLYC','PYRAC','MGLY','HCOOH','MACR','MPAN','GLYX']

In [103]:
settings['Iterations'] = 100

Very Small Mechanism (<39 species)

In [156]:
settings['Reduce Stiffness'] = True

In [168]:
settings['Protected'] = ['ISOP','ISOP1OH23O4OHt','ISOP1OH2OOH','ISOP1OH2OO','ISOP1OHt','MVK','HCHO', 'HAC','PAN','GLYC','PYRAC','MGLY','HCOOH','MACR','MPAN','GLYX','HMML']

In [170]:
settings['Manual Groups'] = [['ISOP1OH23O4OHt','ISOP1OH23O4OHc', 'ISOP1OH2OH34O'], ['ISOP1OHt','ISOP1OHc','ISOP4OHt','ISOP4OHc'], ['ISOP1OH2OO','ISOP3OO4OH','ISOP1OH4OOt','ISOP1OH4OOc','ISOP1OO4OHc','ISOP1OO4OHt']]

In [172]:
settings['No Group'] = ['ISOP', 'MVK','HCHO', 'HAC','PAN','GLYC','PYRAC','MGLY','HCOOH','MACR','MPAN','GLYX']

In [ ]:
settings['Remove Species

In [164]:
settings['Iterations'] = 100

In [140]:
settings['Protected'] = ['ISOP','ISOP1OH23O4OHt','ISOP1OH2OOH','ISOP1OH2OO','MVK','HCHO', 'HAC','PAN','GLYC','PYRAC','MGLY','HCOOH','MACR','MPAN','GLYX','HMML']

In [148]:
settings['Manual Groups'] = [['ISOP1OH23O4OHt','ISOP1OH23O4OHc', 'ISOP1OH2OH34O'], ['ISOP1OH2OOH','ISOP1OOH4OHt','ISOP1OOH4OHc','ISOP1OH4OOHt','ISOP1OH4OOHc'],['ISOP1OH2OO','ISOP3OO4OH','ISOP1OH4OOt','ISOP1OH4OOc','ISOP1OO4OHc','ISOP1OO4OHt']]

In [144]:
settings['Remove Species'] = ['ISOP1OHt', 'ISOP1OHc','ISOP4OHc','ISOP4OHt']

In [166]:
settings['Remove Species'] = []

In [117]:
settings['No Group'] = deepcopy(species_list_names)

In [119]:
settings['Protected'] = ['ISOP','ISOP1OH23O4OHt','ISOP1OH2OOH','ISOP1OH2OO','ISOP3OO4OH','ISOP1OHt','ISOP4OHt','MVK','HCHO', 'HAC','PAN','GLYC','PYRAC','MGLY','HCOOH','MACR','MPAN','GLYX','HMML']

In [121]:
settings['Protected'] = ['ISOP','ISOP1OH23O4OHt','ISOP1OH2OOH','ISOP1OH2OO','ISOP3OO4OH','MVK','HCHO', 'HAC','PAN','GLYC','PYRAC','MGLY','HCOOH','MACR','MPAN','GLYX','HMML']

In [123]:
settings['Remove Species'] = ['ISOP1OHt', 'ISOP1OHc','ISOP4OHc','ISOP4OHt']

#### RUN

In [105]:
settings['Iterations'] = 100

##### settings['']

In [107]:
def create_f0am_file(mech,name):
    spec_2_add = "SpeciesToAdd = {'ISOPN'; "
    count = 0
    for i in mech.species:
        count+=1
        if count%1000 == 0:
            print(count)
        spec_2_add = spec_2_add + "'" + i +"'"+ ';'
    spec_2_add = spec_2_add[:-1]   
    spec_2_add = spec_2_add  + "};"
   
    eq_str = ''
    for i in range(len(mech.reactions)):
        r_string = ''
        for j in mech.reactions[i].reactants:
            r_string = r_string + str(j) + ' + '
        r_string = r_string[:-2] + '= '
        for j in mech.reactions[i].prod_dict:
            r_string = r_string + str(j) + ' + '
        r_string = r_string[:-3]
        reac_str = ''
        reac_str = reac_str + "\ni=i+1;\nRnames{i} = '" + r_string + "';\nk(:,i) = "+ str(mech.reactions[i].rate_law)+ '*'+str(mech.reactions[i].multiplier) + ';\n'
        counter = 1
        for j in mech.reactions[i].reactants:
            reac_str = reac_str + 'Gstr{i,'+str(counter)+"} = '"+str(j)+"'; "
            counter = counter + 1
        reac_str = reac_str +'\n'
        for k in mech.reactions[i].reactants:
            reac_str = reac_str + 'f'+ str(k) +'(i)'+'='+'f'+ str(k) +'(i)'+'-1' + '; '
        for k in mech.reactions[i].prod_dict:
            reac_str = reac_str + 'f'+ str(k)+'(i)'+'='+'f'+ str(k)+'(i)'+'+'+str(mech.reactions[i].prod_dict[k]) + '; '
        
        reac_str = reac_str +'\n'
        reac_str = reac_str +'\n'
        eq_str = eq_str + reac_str

        
    #eq_str = eq_str.replace('CH2O','HCHO')
    eq_str = eq_str.replace('.*RO2',';%.*RO2')
    eq_str = eq_str.replace('J(22)','J22')
    eq_str = eq_str.replace('J(34)','J34')
    eq_str = eq_str.replace('J(41)','J41')
    eq_str = eq_str.replace('ALK(','F0AM_isop_ALK(')
    eq_str = eq_str.replace('EPO(','F0AM_isop_EPO(')
    eq_str = eq_str.replace('NIT(','F0AM_isop_NIT(')
    eq_str = eq_str.replace('TROE(','F0AM_isop_TROE2(')
    eq_str = eq_str.replace('TUN(','F0AM_isop_TUN_2(')
    eq_str = eq_str.replace('ISO1(','F0AM_isop_ISO1(')
    eq_str = eq_str.replace('ISO2(','F0AM_isop_ISO2(')
    eq_str = eq_str.replace('KCO(','F0AM_isop_KCO(')
    eq_str = eq_str.replace('FALL(','F0AM_isop_FALL(')
    eq_str = eq_str.replace('TEMP','T')
    eq_str = eq_str.replace('**','.^')
    eq_str = eq_str.replace('EXP','exp')
    eq_str = eq_str.replace('*','.*')
    eq_str = eq_str.replace('.*O2','.*0.21')
    eq_str = eq_str.replace('KAPHO2(T)','KAPHO2')
    eq_str = eq_str.replace('KFPAN(T,M)','KFPAN')
    eq_str = eq_str.replace('KNO3AL(T)','KNO3AL')
    eq_str = eq_str.replace('KAPNO(T)','KAPNO')
    eq_str = eq_str.replace('KBPAN(T,M)','KBPAN')
    eq_str = eq_str.replace('KRO2NO3()','KRO2NO3')
    full = spec_2_add + '\n'+'RO2ToAdd = {};'+'\n'+'AddSpecies'+'\n'+ eq_str
    f0am_file = open("f0am_"+name+".m","w+")
    f0am_file.write(full)

In [109]:
countex = 0
x_list = []
runtimes = []
settings['Print Progress'] = False
#for n in [300,275,250,225,200,180,160,150,140,130,120,110,90,80,70]:
for n in [42]: #,55,50,48,46,44,43,42,41,40,39,38,37,36,35]:

     
    settings['Mechanism Size'] = n
    print('STARTING')
    xs = time.time()
    x = am.AMORE_mechanism_reduction(isop_mech, background_spc_n,conditions, settings)
    #x = reduce_mechanism_full_group_ammendment_with_comments_new_fix(isop_mech, background_spc_n,conditions, settings)
    x_list.append(x)
    #x = reduce_mechanism_full_group_ammendment_with_comments_n/'ew_fix(camphene_mech, background_spc_n,conditions, settings)
    spec_set = set()
    xf = time.time()
    print('ENDING')
    print('Took ', xf-xs, ' seconds')
    new_mech = deepcopy(x['Reduced Mechanism'])
    runtimes.append(xf-xs)
    name = 'AMORE_newest_stuffn_'+str(n)
    create_f0am_file(new_mech,name)

STARTING
len mpara 78
{4: 1, 5: 1, 6: 1, 7: 1, 12: 0.011848562853466008, 13: 0.9065581591869888, 14: 0.9065581591869888, 15: 0.002723610136480029, 20: 0.9065581591869888, 21: 0.9065581591869888, 22: 0.002723610136480029, 23: 0.002723610136480029, 24: 0.002723610136480029, 25: 0.011848562853466008, 26: 0.011848562853466008, 27: 0.011848562853466008, 36: 0.9065581591869888, 39: 0.5030448157516383, 40: 0.5030448157516383, 41: 0.0065239314569537875, 42: 0.0065239314569537875, 43: 0.0065239314569537875, 44: 0.01105643645486914, 45: 0.01105643645486914, 46: 0.01105643645486914, 47: 0.0024691109916703583, 48: 0.0024691109916703583, 49: 0.0024691109916703583, 50: 0.01074141132944948, 51: 0.01074141132944948, 52: 0.01074141132944948, 61: 0.0015113183256919881, 62: 0.0015113183256919881, 63: 0.0015113183256919881, 64: 0.0015113183256919881, 65: 0.006574711238481189, 66: 0.006574711238481189, 67: 0.006574711238481189, 68: 0.006574711238481189, 69: 0.9065581591869888, 70: 0.9065581591869888, 74: 0

In [132]:
x

{'Reduced Mechanism': <AMORE_v2.Mechanism at 0x20ec8abb680>,
 'Remaining Species': ['ISOP',
  'ISOP1OHt',
  'ISOP1OH2OO',
  'ISOP1OH2OOH',
  'MVK',
  'MACR',
  'PAN',
  'MGLY',
  'HCHO',
  'CO',
  'CO2',
  'GLYC',
  'HNO3',
  'O2',
  'OH',
  'HO2',
  'NO',
  'NO2',
  'NO3',
  'HAC',
  'H2O',
  'H2Od',
  'O3',
  'SO2',
  'GLYX',
  'TETRA',
  'ISOPN',
  'IHN',
  'CH3OO',
  'PYRAC',
  'H2SO4',
  'H2O2',
  'HCOOH',
  'MVK3OO4OH',
  'CH3CO3',
  'MPAN',
  'HMML',
  'ISOP1OH23O4OHt'],
 'Removed Species': ['ISOP1OHc',
  'ISOP4OHt',
  'ISOP4OHc',
  'ISOP1OH4OOt',
  'ISOP1OH4OOc',
  'ISOP1OO4OHt',
  'ISOP1OO4OHc',
  'ISOP3OO4OH',
  'ISOP3OOH4OH',
  'ISOP1CO4OH',
  'ISOP1OH4CO',
  'ISOP1OH2N',
  'ISOP1OH4Nc',
  'ISOP1OH4Nt',
  'ISOP3N4OH',
  'ISOP1N4OHc',
  'ISOP1N4OHt',
  'ISOP1CO2OOH3OO4OH',
  'ISOP1CO2OO3OOH4OH',
  'ISOP1CO2N3OOH4OH',
  'ISOP1CO2OOH3N4OH',
  'MVK3OOH4OH',
  'MGLYOX',
  'ISOP1OH2OOH3OO4CO',
  'ISOP1OH2OO3OOH4CO',
  'ISOP1OH2OOH3N4CO',
  'ISOP1OH2N3OOH4CO',
  'ISOP1OH2OOH3OOH4CO

In [138]:
x.keys()

dict_keys(['Reduced Mechanism', 'Remaining Species', 'Removed Species', 'Removal Order', 'Full Graphs', 'Full Yields', 'Average Full Yield', 'Reduced Graphs', 'Reduced Yields', 'Tiers', 'Strongly Connected Components', 'Reduced Strongly Connected Components', 'Groups', 'Species Dictionary', 'Species'])

In [170]:
x['Species']

['ISOP',
 'ISOP1OHt',
 'ISOP1OHc',
 'ISOP4OHt',
 'ISOP4OHc',
 'ISOP1OH2OO',
 'ISOP1OH4OOt',
 'ISOP1OH4OOc',
 'ISOP1OO4OHt',
 'ISOP1OO4OHc',
 'ISOP3OO4OH',
 'ISOP1OH2OOH',
 'ISOP3OOH4OH',
 'MVK',
 'MACR',
 'ISOP1CO4OH',
 'ISOP1OH4CO',
 'ISOP1OH2N',
 'ISOP1OH4Nc',
 'ISOP1OH4Nt',
 'ISOP3N4OH',
 'ISOP1N4OHc',
 'ISOP1N4OHt',
 'ISOP1CO2OOH3OO4OH',
 'ISOP1CO2OO3OOH4OH',
 'ISOP1CO2N3OOH4OH',
 'ISOP1CO2OOH3N4OH',
 'MVK3OOH4OH',
 'MGLY',
 'MGLYOX',
 'GLYC',
 'HCHO',
 'GLYX',
 'HAC',
 'ISOP1OH2OOH3OO4CO',
 'ISOP1OH2OO3OOH4CO',
 'ISOP1OH2OOH3N4CO',
 'ISOP1OH2N3OOH4CO',
 'ISOP1OH2OOH3OOH4CO',
 'ISOP1CO2OOH3OOH4OH',
 'MACR2OOH3OH',
 'MVK3OH4OH',
 'MACR3CO',
 'MACR2OH3OH',
 'ISOP3CO4OH',
 'ISOP1OH2OO3CO4OH',
 'MVK3CO4OH',
 'HPA',
 'ISOP1OH2N3CO4OH',
 'ISOP3OH4OH',
 'ISOP1OH2OH',
 'ISOP1OH2OOH3CO4OH',
 'ISOP1OH4OH',
 'CH3OO',
 'CH3OH',
 'ISOP1OOH4OHc',
 'ISOP1OH4OOHc',
 'ISOP1OOH4OHt',
 'ISOP1OH4OOHt',
 'ISOP1CO4OOHc',
 'ISOP1CO2R3OOH4OOH',
 'ISOP1CO2OO3OOH4OOH',
 'ISOP1CO23O4OOH',
 'ISOP1CO2OOH3OO4OO